In [2]:
import pandas as pd
from pandas import DataFrame
import csv
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import datetime
%matplotlib inline
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 40)

In [4]:
file2 = 'rugged_data.csv'
terrain_df = pd.read_csv(file2, delimiter=',')

terrain_df.sort_values('isocode', inplace=True)
terrain_df

,isocode,isonum,country,rugged,rugged_popw,rugged_slope,rugged_lsd,rugged_pc,land_area,lat,...,slave_exports,dist_slavemkt_atlantic,dist_slavemkt_indian,dist_slavemkt_saharan,dist_slavemkt_redsea,pop_1400,european_descent,island,landlocked,geoscheme
0,ABW,533,Aruba,0.462000,0.380000,1.226000,0.144000,0.00000,18.000000,12.508000,...,0.000,NaN,NaN,NaN,NaN,6.140000e+02,NaN,1,0,NaN
1,AFG,4,Afghanistan,2.518000,1.469000,7.414000,0.720000,39.00400,65209.000000,33.833000,...,0.000,NaN,NaN,NaN,NaN,1.870829e+06,0.000000,0,1,34.0
2,AGO,24,Angola,0.858000,0.714000,2.274000,0.228000,4.90600,124670.000000,-12.299000,...,3610000.000,5.669,6.981,4.926,3.872,1.223208e+06,2.000000,0,0,17.0
3,AIA,660,Anguilla,0.013000,0.010000,0.026000,0.006000,0.00000,9.000000,18.231000,...,0.000,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN
4,ALB,8,Albania,3.427000,1.597000,10.451000,1.006000,62.13300,2740.000000,41.143000,...,0.000,NaN,NaN,NaN,NaN,2.000000e+05,100.000000,0,0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,YEM,887,Yemen,2.323000,2.197000,6.155000,0.591000,33.06000,52797.000000,15.863000,...,0.000,NaN,NaN,NaN,NaN,2.250000e+06,0.000000,0,0,145.0
244,YUG,890,Yugoslavia,1.875685,0.839372,5.751834,0.540533,30.83074,6747.389474,44.134539,...,0.000,0.000,0.000,0.000,0.000,6.153843e+05,99.002857,0,0,39.0
231,ZAF,710,South Africa,1.761000,1.318000,4.942000,0.450000,23.95700,121447.000000,-29.004000,...,2030.663,6.766,3.457,6.584,4.895,4.865270e+05,18.000000,0,0,18.0
232,ZMB,894,Zambia,0.533000,0.506000,1.422000,0.145000,2.45500,74339.000000,-13.458000,...,27957.102,9.027,2.389,4.849,3.253,3.482200e+05,1.000000,0,1,14.0


In [7]:
i = 1949
while i < 2020:
    file = 'Gravity_%s.csv' %  i
    source_df = pd.read_csv(file, delimiter=',')
    df2 = source_df[['year','iso3_o','iso3num_o','iso3_d','iso3num_d','tradeflow_comtrade_o','tradeflow_comtrade_d','tradeflow_imf_o','tradeflow_imf_d','contig','distw','comlang_off','comlang_ethno','comcol','col45','comrelig','pop_o','pop_d','gdp_o','gdp_d','gatt_o','gatt_d','wto_o','wto_d','eu_o','eu_d']]
    
    df2['rugged_popw_o'] = np.nan
    df2['rugged_popw_d'] = np.nan
    df2['desert_o'] = np.nan
    df2['desert_d'] = np.nan
    df2['tropical_o'] = np.nan
    df2['tropical_d'] = np.nan
    df2['island_o'] = np.nan
    df2['island_d'] = np.nan
    df2['landlocked_o'] = np.nan
    df2['landlocked_d'] = np.nan
    df2['geoscheme_o'] = np.nan
    df2['geoscheme_d'] = np.nan
    
    df2['geoscheme_o'][df2['iso3num_o'] == 533] = terrain_df.iloc[0,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 533] = terrain_df.iloc[0,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 4] = terrain_df.iloc[1,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 4] = terrain_df.iloc[1,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 24] = terrain_df.iloc[2,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 24] = terrain_df.iloc[2,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 660] = terrain_df.iloc[3,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 660] = terrain_df.iloc[3,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 8] = terrain_df.iloc[4,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 8] = terrain_df.iloc[4,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 20] = terrain_df.iloc[5,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 20] = terrain_df.iloc[5,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 530] = terrain_df.iloc[6,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 530] = terrain_df.iloc[6,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 784] = terrain_df.iloc[7,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 784] = terrain_df.iloc[7,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 32] = terrain_df.iloc[8,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 32] = terrain_df.iloc[8,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 51] = terrain_df.iloc[9,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 51] = terrain_df.iloc[9,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 16] = terrain_df.iloc[10,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 16] = terrain_df.iloc[10,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 28] = terrain_df.iloc[11,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 28] = terrain_df.iloc[11,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 36] = terrain_df.iloc[12,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 36] = terrain_df.iloc[12,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 40] = terrain_df.iloc[13,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 40] = terrain_df.iloc[13,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 31] = terrain_df.iloc[14,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 31] = terrain_df.iloc[14,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 108] = terrain_df.iloc[15,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 108] = terrain_df.iloc[15,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 56] = terrain_df.iloc[16,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 56] = terrain_df.iloc[16,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 204] = terrain_df.iloc[17,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 204] = terrain_df.iloc[17,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 854] = terrain_df.iloc[18,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 854] = terrain_df.iloc[18,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 50] = terrain_df.iloc[19,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 50] = terrain_df.iloc[19,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 100] = terrain_df.iloc[20,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 100] = terrain_df.iloc[20,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 48] = terrain_df.iloc[21,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 48] = terrain_df.iloc[21,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 44] = terrain_df.iloc[22,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 44] = terrain_df.iloc[22,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 70] = terrain_df.iloc[23,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 70] = terrain_df.iloc[23,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 112] = terrain_df.iloc[24,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 112] = terrain_df.iloc[24,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 84] = terrain_df.iloc[25,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 84] = terrain_df.iloc[25,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 60] = terrain_df.iloc[26,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 60] = terrain_df.iloc[26,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 68] = terrain_df.iloc[27,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 68] = terrain_df.iloc[27,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 76] = terrain_df.iloc[28,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 76] = terrain_df.iloc[28,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 52] = terrain_df.iloc[29,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 52] = terrain_df.iloc[29,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 96] = terrain_df.iloc[30,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 96] = terrain_df.iloc[30,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 64] = terrain_df.iloc[31,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 64] = terrain_df.iloc[31,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 72] = terrain_df.iloc[32,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 72] = terrain_df.iloc[32,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 140] = terrain_df.iloc[33,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 140] = terrain_df.iloc[33,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 124] = terrain_df.iloc[34,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 124] = terrain_df.iloc[34,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 166] = terrain_df.iloc[35,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 166] = terrain_df.iloc[35,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 756] = terrain_df.iloc[36,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 756] = terrain_df.iloc[36,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 152] = terrain_df.iloc[37,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 152] = terrain_df.iloc[37,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 156] = terrain_df.iloc[38,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 156] = terrain_df.iloc[38,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 384] = terrain_df.iloc[39,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 384] = terrain_df.iloc[39,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 120] = terrain_df.iloc[40,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 120] = terrain_df.iloc[40,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 180] = terrain_df.iloc[41,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 180] = terrain_df.iloc[41,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 178] = terrain_df.iloc[42,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 178] = terrain_df.iloc[42,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 184] = terrain_df.iloc[43,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 184] = terrain_df.iloc[43,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 170] = terrain_df.iloc[44,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 170] = terrain_df.iloc[44,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 174] = terrain_df.iloc[45,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 174] = terrain_df.iloc[45,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 132] = terrain_df.iloc[46,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 132] = terrain_df.iloc[46,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 188] = terrain_df.iloc[47,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 188] = terrain_df.iloc[47,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 192] = terrain_df.iloc[48,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 192] = terrain_df.iloc[48,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 162] = terrain_df.iloc[49,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 162] = terrain_df.iloc[49,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 136] = terrain_df.iloc[50,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 136] = terrain_df.iloc[50,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 196] = terrain_df.iloc[51,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 196] = terrain_df.iloc[51,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 203] = terrain_df.iloc[52,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 203] = terrain_df.iloc[52,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 276] = terrain_df.iloc[53,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 276] = terrain_df.iloc[53,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 262] = terrain_df.iloc[54,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 262] = terrain_df.iloc[54,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 212] = terrain_df.iloc[55,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 212] = terrain_df.iloc[55,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 208] = terrain_df.iloc[56,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 208] = terrain_df.iloc[56,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 214] = terrain_df.iloc[57,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 214] = terrain_df.iloc[57,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 12] = terrain_df.iloc[58,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 12] = terrain_df.iloc[58,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 218] = terrain_df.iloc[59,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 218] = terrain_df.iloc[59,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 818] = terrain_df.iloc[60,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 818] = terrain_df.iloc[60,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 232] = terrain_df.iloc[61,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 232] = terrain_df.iloc[61,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 732] = terrain_df.iloc[62,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 732] = terrain_df.iloc[62,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 724] = terrain_df.iloc[63,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 724] = terrain_df.iloc[63,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 233] = terrain_df.iloc[64,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 233] = terrain_df.iloc[64,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 231] = terrain_df.iloc[65,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 231] = terrain_df.iloc[65,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 246] = terrain_df.iloc[66,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 246] = terrain_df.iloc[66,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 242] = terrain_df.iloc[67,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 242] = terrain_df.iloc[67,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 238] = terrain_df.iloc[68,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 238] = terrain_df.iloc[68,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 250] = terrain_df.iloc[69,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 250] = terrain_df.iloc[69,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 234] = terrain_df.iloc[70,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 234] = terrain_df.iloc[70,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 583] = terrain_df.iloc[71,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 583] = terrain_df.iloc[71,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 266] = terrain_df.iloc[72,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 266] = terrain_df.iloc[72,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 826] = terrain_df.iloc[73,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 826] = terrain_df.iloc[73,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 268] = terrain_df.iloc[74,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 268] = terrain_df.iloc[74,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 288] = terrain_df.iloc[75,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 288] = terrain_df.iloc[75,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 292] = terrain_df.iloc[76,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 292] = terrain_df.iloc[76,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 324] = terrain_df.iloc[77,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 324] = terrain_df.iloc[77,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 312] = terrain_df.iloc[78,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 312] = terrain_df.iloc[78,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 270] = terrain_df.iloc[79,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 270] = terrain_df.iloc[79,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 624] = terrain_df.iloc[80,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 624] = terrain_df.iloc[80,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 226] = terrain_df.iloc[81,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 226] = terrain_df.iloc[81,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 300] = terrain_df.iloc[82,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 300] = terrain_df.iloc[82,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 308] = terrain_df.iloc[83,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 308] = terrain_df.iloc[83,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 304] = terrain_df.iloc[84,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 304] = terrain_df.iloc[84,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 320] = terrain_df.iloc[85,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 320] = terrain_df.iloc[85,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 254] = terrain_df.iloc[86,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 254] = terrain_df.iloc[86,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 316] = terrain_df.iloc[87,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 316] = terrain_df.iloc[87,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 328] = terrain_df.iloc[88,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 328] = terrain_df.iloc[88,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 344] = terrain_df.iloc[89,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 344] = terrain_df.iloc[89,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 340] = terrain_df.iloc[90,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 340] = terrain_df.iloc[90,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 191] = terrain_df.iloc[91,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 191] = terrain_df.iloc[91,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 332] = terrain_df.iloc[92,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 332] = terrain_df.iloc[92,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 348] = terrain_df.iloc[93,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 348] = terrain_df.iloc[93,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 360] = terrain_df.iloc[94,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 360] = terrain_df.iloc[94,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 356] = terrain_df.iloc[95,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 356] = terrain_df.iloc[95,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 86] = terrain_df.iloc[96,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 86] = terrain_df.iloc[96,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 372] = terrain_df.iloc[97,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 372] = terrain_df.iloc[97,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 364] = terrain_df.iloc[98,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 364] = terrain_df.iloc[98,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 368] = terrain_df.iloc[99,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 368] = terrain_df.iloc[99,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 352] = terrain_df.iloc[100,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 352] = terrain_df.iloc[100,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 376] = terrain_df.iloc[101,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 376] = terrain_df.iloc[101,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 380] = terrain_df.iloc[102,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 380] = terrain_df.iloc[102,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 388] = terrain_df.iloc[103,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 388] = terrain_df.iloc[103,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 400] = terrain_df.iloc[104,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 400] = terrain_df.iloc[104,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 392] = terrain_df.iloc[105,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 392] = terrain_df.iloc[105,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 398] = terrain_df.iloc[106,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 398] = terrain_df.iloc[106,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 404] = terrain_df.iloc[107,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 404] = terrain_df.iloc[107,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 417] = terrain_df.iloc[108,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 417] = terrain_df.iloc[108,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 116] = terrain_df.iloc[109,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 116] = terrain_df.iloc[109,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 296] = terrain_df.iloc[110,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 296] = terrain_df.iloc[110,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 659] = terrain_df.iloc[111,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 659] = terrain_df.iloc[111,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 410] = terrain_df.iloc[112,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 410] = terrain_df.iloc[112,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 414] = terrain_df.iloc[113,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 414] = terrain_df.iloc[113,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 418] = terrain_df.iloc[114,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 418] = terrain_df.iloc[114,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 422] = terrain_df.iloc[115,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 422] = terrain_df.iloc[115,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 430] = terrain_df.iloc[116,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 430] = terrain_df.iloc[116,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 434] = terrain_df.iloc[117,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 434] = terrain_df.iloc[117,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 662] = terrain_df.iloc[118,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 662] = terrain_df.iloc[118,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 438] = terrain_df.iloc[119,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 438] = terrain_df.iloc[119,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 144] = terrain_df.iloc[120,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 144] = terrain_df.iloc[120,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 426] = terrain_df.iloc[121,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 426] = terrain_df.iloc[121,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 440] = terrain_df.iloc[122,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 440] = terrain_df.iloc[122,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 442] = terrain_df.iloc[123,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 442] = terrain_df.iloc[123,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 428] = terrain_df.iloc[124,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 428] = terrain_df.iloc[124,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 446] = terrain_df.iloc[125,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 446] = terrain_df.iloc[125,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 504] = terrain_df.iloc[126,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 504] = terrain_df.iloc[126,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 492] = terrain_df.iloc[127,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 492] = terrain_df.iloc[127,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 498] = terrain_df.iloc[128,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 498] = terrain_df.iloc[128,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 450] = terrain_df.iloc[129,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 450] = terrain_df.iloc[129,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 462] = terrain_df.iloc[130,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 462] = terrain_df.iloc[130,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 484] = terrain_df.iloc[131,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 484] = terrain_df.iloc[131,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 584] = terrain_df.iloc[132,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 584] = terrain_df.iloc[132,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 807] = terrain_df.iloc[133,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 807] = terrain_df.iloc[133,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 466] = terrain_df.iloc[134,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 466] = terrain_df.iloc[134,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 470] = terrain_df.iloc[135,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 470] = terrain_df.iloc[135,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 104] = terrain_df.iloc[136,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 104] = terrain_df.iloc[136,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 496] = terrain_df.iloc[137,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 496] = terrain_df.iloc[137,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 580] = terrain_df.iloc[138,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 580] = terrain_df.iloc[138,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 508] = terrain_df.iloc[139,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 508] = terrain_df.iloc[139,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 478] = terrain_df.iloc[140,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 478] = terrain_df.iloc[140,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 500] = terrain_df.iloc[141,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 500] = terrain_df.iloc[141,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 474] = terrain_df.iloc[142,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 474] = terrain_df.iloc[142,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 480] = terrain_df.iloc[143,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 480] = terrain_df.iloc[143,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 454] = terrain_df.iloc[144,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 454] = terrain_df.iloc[144,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 458] = terrain_df.iloc[145,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 458] = terrain_df.iloc[145,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 175] = terrain_df.iloc[146,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 175] = terrain_df.iloc[146,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 516] = terrain_df.iloc[147,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 516] = terrain_df.iloc[147,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 540] = terrain_df.iloc[148,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 540] = terrain_df.iloc[148,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 562] = terrain_df.iloc[149,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 562] = terrain_df.iloc[149,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 574] = terrain_df.iloc[150,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 574] = terrain_df.iloc[150,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 566] = terrain_df.iloc[151,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 566] = terrain_df.iloc[151,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 558] = terrain_df.iloc[152,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 558] = terrain_df.iloc[152,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 570] = terrain_df.iloc[153,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 570] = terrain_df.iloc[153,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 528] = terrain_df.iloc[154,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 528] = terrain_df.iloc[154,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 578] = terrain_df.iloc[155,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 578] = terrain_df.iloc[155,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 524] = terrain_df.iloc[156,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 524] = terrain_df.iloc[156,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 520] = terrain_df.iloc[157,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 520] = terrain_df.iloc[157,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 554] = terrain_df.iloc[158,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 554] = terrain_df.iloc[158,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 512] = terrain_df.iloc[159,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 512] = terrain_df.iloc[159,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 586] = terrain_df.iloc[160,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 586] = terrain_df.iloc[160,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 591] = terrain_df.iloc[161,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 591] = terrain_df.iloc[161,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 612] = terrain_df.iloc[162,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 612] = terrain_df.iloc[162,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 604] = terrain_df.iloc[163,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 604] = terrain_df.iloc[163,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 608] = terrain_df.iloc[164,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 608] = terrain_df.iloc[164,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 585] = terrain_df.iloc[165,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 585] = terrain_df.iloc[165,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 598] = terrain_df.iloc[166,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 598] = terrain_df.iloc[166,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 616] = terrain_df.iloc[167,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 616] = terrain_df.iloc[167,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 630] = terrain_df.iloc[168,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 630] = terrain_df.iloc[168,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 408] = terrain_df.iloc[169,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 408] = terrain_df.iloc[169,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 620] = terrain_df.iloc[170,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 620] = terrain_df.iloc[170,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 600] = terrain_df.iloc[171,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 600] = terrain_df.iloc[171,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 275] = terrain_df.iloc[172,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 275] = terrain_df.iloc[172,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 258] = terrain_df.iloc[173,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 258] = terrain_df.iloc[173,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 634] = terrain_df.iloc[174,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 634] = terrain_df.iloc[174,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 638] = terrain_df.iloc[175,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 638] = terrain_df.iloc[175,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 642] = terrain_df.iloc[176,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 642] = terrain_df.iloc[176,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 643] = terrain_df.iloc[177,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 643] = terrain_df.iloc[177,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 646] = terrain_df.iloc[178,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 646] = terrain_df.iloc[178,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 682] = terrain_df.iloc[179,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 682] = terrain_df.iloc[179,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 891] = terrain_df.iloc[180,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 891] = terrain_df.iloc[180,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 736] = terrain_df.iloc[181,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 736] = terrain_df.iloc[181,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 686] = terrain_df.iloc[182,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 686] = terrain_df.iloc[182,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 702] = terrain_df.iloc[183,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 702] = terrain_df.iloc[183,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 654] = terrain_df.iloc[184,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 654] = terrain_df.iloc[184,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 744] = terrain_df.iloc[185,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 744] = terrain_df.iloc[185,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 90] = terrain_df.iloc[186,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 90] = terrain_df.iloc[186,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 694] = terrain_df.iloc[187,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 694] = terrain_df.iloc[187,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 222] = terrain_df.iloc[188,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 222] = terrain_df.iloc[188,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 674] = terrain_df.iloc[189,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 674] = terrain_df.iloc[189,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 706] = terrain_df.iloc[190,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 706] = terrain_df.iloc[190,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 666] = terrain_df.iloc[191,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 666] = terrain_df.iloc[191,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 678] = terrain_df.iloc[192,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 678] = terrain_df.iloc[192,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 740] = terrain_df.iloc[193,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 740] = terrain_df.iloc[193,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 703] = terrain_df.iloc[194,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 703] = terrain_df.iloc[194,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 705] = terrain_df.iloc[195,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 705] = terrain_df.iloc[195,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 752] = terrain_df.iloc[196,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 752] = terrain_df.iloc[196,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 748] = terrain_df.iloc[197,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 748] = terrain_df.iloc[197,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 690] = terrain_df.iloc[198,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 690] = terrain_df.iloc[198,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 760] = terrain_df.iloc[199,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 760] = terrain_df.iloc[199,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 796] = terrain_df.iloc[200,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 796] = terrain_df.iloc[200,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 148] = terrain_df.iloc[201,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 148] = terrain_df.iloc[201,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 768] = terrain_df.iloc[202,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 768] = terrain_df.iloc[202,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 764] = terrain_df.iloc[203,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 764] = terrain_df.iloc[203,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 762] = terrain_df.iloc[204,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 762] = terrain_df.iloc[204,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 772] = terrain_df.iloc[205,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 772] = terrain_df.iloc[205,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 795] = terrain_df.iloc[206,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 795] = terrain_df.iloc[206,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 626] = terrain_df.iloc[207,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 626] = terrain_df.iloc[207,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 776] = terrain_df.iloc[208,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 776] = terrain_df.iloc[208,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 780] = terrain_df.iloc[209,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 780] = terrain_df.iloc[209,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 788] = terrain_df.iloc[210,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 788] = terrain_df.iloc[210,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 792] = terrain_df.iloc[211,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 792] = terrain_df.iloc[211,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 798] = terrain_df.iloc[212,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 798] = terrain_df.iloc[212,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 158] = terrain_df.iloc[213,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 158] = terrain_df.iloc[213,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 834] = terrain_df.iloc[214,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 834] = terrain_df.iloc[214,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 800] = terrain_df.iloc[215,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 800] = terrain_df.iloc[215,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 804] = terrain_df.iloc[216,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 804] = terrain_df.iloc[216,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 581] = terrain_df.iloc[217,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 581] = terrain_df.iloc[217,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 858] = terrain_df.iloc[218,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 858] = terrain_df.iloc[218,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 840] = terrain_df.iloc[219,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 840] = terrain_df.iloc[219,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 860] = terrain_df.iloc[220,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 860] = terrain_df.iloc[220,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 336] = terrain_df.iloc[221,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 336] = terrain_df.iloc[221,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 670] = terrain_df.iloc[222,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 670] = terrain_df.iloc[222,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 862] = terrain_df.iloc[223,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 862] = terrain_df.iloc[223,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 92] = terrain_df.iloc[224,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 92] = terrain_df.iloc[224,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 850] = terrain_df.iloc[225,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 850] = terrain_df.iloc[225,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 704] = terrain_df.iloc[226,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 704] = terrain_df.iloc[226,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 548] = terrain_df.iloc[227,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 548] = terrain_df.iloc[227,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 876] = terrain_df.iloc[228,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 876] = terrain_df.iloc[228,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 882] = terrain_df.iloc[229,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 882] = terrain_df.iloc[229,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 887] = terrain_df.iloc[230,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 887] = terrain_df.iloc[230,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 710] = terrain_df.iloc[231,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 710] = terrain_df.iloc[231,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 894] = terrain_df.iloc[232,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 894] = terrain_df.iloc[232,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 716] = terrain_df.iloc[233,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 716] = terrain_df.iloc[233,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 200] = terrain_df.iloc[234,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 200] = terrain_df.iloc[234,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 278] = terrain_df.iloc[235,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 278] = terrain_df.iloc[235,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 280] = terrain_df.iloc[236,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 280] = terrain_df.iloc[236,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 230] = terrain_df.iloc[237,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 230] = terrain_df.iloc[237,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 499] = terrain_df.iloc[238,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 499] = terrain_df.iloc[238,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 590] = terrain_df.iloc[239,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 590] = terrain_df.iloc[239,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 729] = terrain_df.iloc[240,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 729] = terrain_df.iloc[240,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 688] = terrain_df.iloc[241,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 688] = terrain_df.iloc[241,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 728] = terrain_df.iloc[242,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 728] = terrain_df.iloc[242,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 810] = terrain_df.iloc[243,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 810] = terrain_df.iloc[243,53]
    df2['geoscheme_o'][df2['iso3num_o'] == 890] = terrain_df.iloc[244,53]
    df2['geoscheme_d'][df2['iso3num_d'] == 890] = terrain_df.iloc[244,53]
    
    df2['rugged_popw_o'][df2['iso3num_o'] == 533] = terrain_df.iloc[0,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 533] = terrain_df.iloc[0,4]
    df2['desert_o'][df2['iso3num_o'] == 533] = terrain_df.iloc[0,12]
    df2['desert_d'][df2['iso3num_d'] == 533] = terrain_df.iloc[0,12]
    df2['tropical_o'][df2['iso3num_o'] == 533] = terrain_df.iloc[0,13]
    df2['tropical_d'][df2['iso3num_d'] == 533] = terrain_df.iloc[0,13]
    df2['island_o'][df2['iso3num_o'] == 533] = terrain_df.iloc[0,51]
    df2['island_d'][df2['iso3num_d'] == 533] = terrain_df.iloc[0,51]
    df2['landlocked_o'][df2['iso3num_o'] == 533] = terrain_df.iloc[0,52]
    df2['landlocked_d'][df2['iso3num_d'] == 533] = terrain_df.iloc[0,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 4] = terrain_df.iloc[1,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 4] = terrain_df.iloc[1,4]
    df2['desert_o'][df2['iso3num_o'] == 4] = terrain_df.iloc[1,12]
    df2['desert_d'][df2['iso3num_d'] == 4] = terrain_df.iloc[1,12]
    df2['tropical_o'][df2['iso3num_o'] == 4] = terrain_df.iloc[1,13]
    df2['tropical_d'][df2['iso3num_d'] == 4] = terrain_df.iloc[1,13]
    df2['island_o'][df2['iso3num_o'] == 4] = terrain_df.iloc[1,51]
    df2['island_d'][df2['iso3num_d'] == 4] = terrain_df.iloc[1,51]
    df2['landlocked_o'][df2['iso3num_o'] == 4] = terrain_df.iloc[1,52]
    df2['landlocked_d'][df2['iso3num_d'] == 4] = terrain_df.iloc[1,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 24] = terrain_df.iloc[2,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 24] = terrain_df.iloc[2,4]
    df2['desert_o'][df2['iso3num_o'] == 24] = terrain_df.iloc[2,12]
    df2['desert_d'][df2['iso3num_d'] == 24] = terrain_df.iloc[2,12]
    df2['tropical_o'][df2['iso3num_o'] == 24] = terrain_df.iloc[2,13]
    df2['tropical_d'][df2['iso3num_d'] == 24] = terrain_df.iloc[2,13]
    df2['island_o'][df2['iso3num_o'] == 24] = terrain_df.iloc[2,51]
    df2['island_d'][df2['iso3num_d'] == 24] = terrain_df.iloc[2,51]
    df2['landlocked_o'][df2['iso3num_o'] == 24] = terrain_df.iloc[2,52]
    df2['landlocked_d'][df2['iso3num_d'] == 24] = terrain_df.iloc[2,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 660] = terrain_df.iloc[3,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 660] = terrain_df.iloc[3,4]
    df2['desert_o'][df2['iso3num_o'] == 660] = terrain_df.iloc[3,12]
    df2['desert_d'][df2['iso3num_d'] == 660] = terrain_df.iloc[3,12]
    df2['tropical_o'][df2['iso3num_o'] == 660] = terrain_df.iloc[3,13]
    df2['tropical_d'][df2['iso3num_d'] == 660] = terrain_df.iloc[3,13]
    df2['island_o'][df2['iso3num_o'] == 660] = terrain_df.iloc[3,51]
    df2['island_d'][df2['iso3num_d'] == 660] = terrain_df.iloc[3,51]
    df2['landlocked_o'][df2['iso3num_o'] == 660] = terrain_df.iloc[3,52]
    df2['landlocked_d'][df2['iso3num_d'] == 660] = terrain_df.iloc[3,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 8] = terrain_df.iloc[4,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 8] = terrain_df.iloc[4,4]
    df2['desert_o'][df2['iso3num_o'] == 8] = terrain_df.iloc[4,12]
    df2['desert_d'][df2['iso3num_d'] == 8] = terrain_df.iloc[4,12]
    df2['tropical_o'][df2['iso3num_o'] == 8] = terrain_df.iloc[4,13]
    df2['tropical_d'][df2['iso3num_d'] == 8] = terrain_df.iloc[4,13]
    df2['island_o'][df2['iso3num_o'] == 8] = terrain_df.iloc[4,51]
    df2['island_d'][df2['iso3num_d'] == 8] = terrain_df.iloc[4,51]
    df2['landlocked_o'][df2['iso3num_o'] == 8] = terrain_df.iloc[4,52]
    df2['landlocked_d'][df2['iso3num_d'] == 8] = terrain_df.iloc[4,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 20] = terrain_df.iloc[5,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 20] = terrain_df.iloc[5,4]
    df2['desert_o'][df2['iso3num_o'] == 20] = terrain_df.iloc[5,12]
    df2['desert_d'][df2['iso3num_d'] == 20] = terrain_df.iloc[5,12]
    df2['tropical_o'][df2['iso3num_o'] == 20] = terrain_df.iloc[5,13]
    df2['tropical_d'][df2['iso3num_d'] == 20] = terrain_df.iloc[5,13]
    df2['island_o'][df2['iso3num_o'] == 20] = terrain_df.iloc[5,51]
    df2['island_d'][df2['iso3num_d'] == 20] = terrain_df.iloc[5,51]
    df2['landlocked_o'][df2['iso3num_o'] == 20] = terrain_df.iloc[5,52]
    df2['landlocked_d'][df2['iso3num_d'] == 20] = terrain_df.iloc[5,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 530] = terrain_df.iloc[6,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 530] = terrain_df.iloc[6,4]
    df2['desert_o'][df2['iso3num_o'] == 530] = terrain_df.iloc[6,12]
    df2['desert_d'][df2['iso3num_d'] == 530] = terrain_df.iloc[6,12]
    df2['tropical_o'][df2['iso3num_o'] == 530] = terrain_df.iloc[6,13]
    df2['tropical_d'][df2['iso3num_d'] == 530] = terrain_df.iloc[6,13]
    df2['island_o'][df2['iso3num_o'] == 530] = terrain_df.iloc[6,51]
    df2['island_d'][df2['iso3num_d'] == 530] = terrain_df.iloc[6,51]
    df2['landlocked_o'][df2['iso3num_o'] == 530] = terrain_df.iloc[6,52]
    df2['landlocked_d'][df2['iso3num_d'] == 530] = terrain_df.iloc[6,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 784] = terrain_df.iloc[7,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 784] = terrain_df.iloc[7,4]
    df2['desert_o'][df2['iso3num_o'] == 784] = terrain_df.iloc[7,12]
    df2['desert_d'][df2['iso3num_d'] == 784] = terrain_df.iloc[7,12]
    df2['tropical_o'][df2['iso3num_o'] == 784] = terrain_df.iloc[7,13]
    df2['tropical_d'][df2['iso3num_d'] == 784] = terrain_df.iloc[7,13]
    df2['island_o'][df2['iso3num_o'] == 784] = terrain_df.iloc[7,51]
    df2['island_d'][df2['iso3num_d'] == 784] = terrain_df.iloc[7,51]
    df2['landlocked_o'][df2['iso3num_o'] == 784] = terrain_df.iloc[7,52]
    df2['landlocked_d'][df2['iso3num_d'] == 784] = terrain_df.iloc[7,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 32] = terrain_df.iloc[8,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 32] = terrain_df.iloc[8,4]
    df2['desert_o'][df2['iso3num_o'] == 32] = terrain_df.iloc[8,12]
    df2['desert_d'][df2['iso3num_d'] == 32] = terrain_df.iloc[8,12]
    df2['tropical_o'][df2['iso3num_o'] == 32] = terrain_df.iloc[8,13]
    df2['tropical_d'][df2['iso3num_d'] == 32] = terrain_df.iloc[8,13]
    df2['island_o'][df2['iso3num_o'] == 32] = terrain_df.iloc[8,51]
    df2['island_d'][df2['iso3num_d'] == 32] = terrain_df.iloc[8,51]
    df2['landlocked_o'][df2['iso3num_o'] == 32] = terrain_df.iloc[8,52]
    df2['landlocked_d'][df2['iso3num_d'] == 32] = terrain_df.iloc[8,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 51] = terrain_df.iloc[9,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 51] = terrain_df.iloc[9,4]
    df2['desert_o'][df2['iso3num_o'] == 51] = terrain_df.iloc[9,12]
    df2['desert_d'][df2['iso3num_d'] == 51] = terrain_df.iloc[9,12]
    df2['tropical_o'][df2['iso3num_o'] == 51] = terrain_df.iloc[9,13]
    df2['tropical_d'][df2['iso3num_d'] == 51] = terrain_df.iloc[9,13]
    df2['island_o'][df2['iso3num_o'] == 51] = terrain_df.iloc[9,51]
    df2['island_d'][df2['iso3num_d'] == 51] = terrain_df.iloc[9,51]
    df2['landlocked_o'][df2['iso3num_o'] == 51] = terrain_df.iloc[9,52]
    df2['landlocked_d'][df2['iso3num_d'] == 51] = terrain_df.iloc[9,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 16] = terrain_df.iloc[10,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 16] = terrain_df.iloc[10,4]
    df2['desert_o'][df2['iso3num_o'] == 16] = terrain_df.iloc[10,12]
    df2['desert_d'][df2['iso3num_d'] == 16] = terrain_df.iloc[10,12]
    df2['tropical_o'][df2['iso3num_o'] == 16] = terrain_df.iloc[10,13]
    df2['tropical_d'][df2['iso3num_d'] == 16] = terrain_df.iloc[10,13]
    df2['island_o'][df2['iso3num_o'] == 16] = terrain_df.iloc[10,51]
    df2['island_d'][df2['iso3num_d'] == 16] = terrain_df.iloc[10,51]
    df2['landlocked_o'][df2['iso3num_o'] == 16] = terrain_df.iloc[10,52]
    df2['landlocked_d'][df2['iso3num_d'] == 16] = terrain_df.iloc[10,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 28] = terrain_df.iloc[11,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 28] = terrain_df.iloc[11,4]
    df2['desert_o'][df2['iso3num_o'] == 28] = terrain_df.iloc[11,12]
    df2['desert_d'][df2['iso3num_d'] == 28] = terrain_df.iloc[11,12]
    df2['tropical_o'][df2['iso3num_o'] == 28] = terrain_df.iloc[11,13]
    df2['tropical_d'][df2['iso3num_d'] == 28] = terrain_df.iloc[11,13]
    df2['island_o'][df2['iso3num_o'] == 28] = terrain_df.iloc[11,51]
    df2['island_d'][df2['iso3num_d'] == 28] = terrain_df.iloc[11,51]
    df2['landlocked_o'][df2['iso3num_o'] == 28] = terrain_df.iloc[11,52]
    df2['landlocked_d'][df2['iso3num_d'] == 28] = terrain_df.iloc[11,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 36] = terrain_df.iloc[12,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 36] = terrain_df.iloc[12,4]
    df2['desert_o'][df2['iso3num_o'] == 36] = terrain_df.iloc[12,12]
    df2['desert_d'][df2['iso3num_d'] == 36] = terrain_df.iloc[12,12]
    df2['tropical_o'][df2['iso3num_o'] == 36] = terrain_df.iloc[12,13]
    df2['tropical_d'][df2['iso3num_d'] == 36] = terrain_df.iloc[12,13]
    df2['island_o'][df2['iso3num_o'] == 36] = terrain_df.iloc[12,51]
    df2['island_d'][df2['iso3num_d'] == 36] = terrain_df.iloc[12,51]
    df2['landlocked_o'][df2['iso3num_o'] == 36] = terrain_df.iloc[12,52]
    df2['landlocked_d'][df2['iso3num_d'] == 36] = terrain_df.iloc[12,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 40] = terrain_df.iloc[13,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 40] = terrain_df.iloc[13,4]
    df2['desert_o'][df2['iso3num_o'] == 40] = terrain_df.iloc[13,12]
    df2['desert_d'][df2['iso3num_d'] == 40] = terrain_df.iloc[13,12]
    df2['tropical_o'][df2['iso3num_o'] == 40] = terrain_df.iloc[13,13]
    df2['tropical_d'][df2['iso3num_d'] == 40] = terrain_df.iloc[13,13]
    df2['island_o'][df2['iso3num_o'] == 40] = terrain_df.iloc[13,51]
    df2['island_d'][df2['iso3num_d'] == 40] = terrain_df.iloc[13,51]
    df2['landlocked_o'][df2['iso3num_o'] == 40] = terrain_df.iloc[13,52]
    df2['landlocked_d'][df2['iso3num_d'] == 40] = terrain_df.iloc[13,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 31] = terrain_df.iloc[14,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 31] = terrain_df.iloc[14,4]
    df2['desert_o'][df2['iso3num_o'] == 31] = terrain_df.iloc[14,12]
    df2['desert_d'][df2['iso3num_d'] == 31] = terrain_df.iloc[14,12]
    df2['tropical_o'][df2['iso3num_o'] == 31] = terrain_df.iloc[14,13]
    df2['tropical_d'][df2['iso3num_d'] == 31] = terrain_df.iloc[14,13]
    df2['island_o'][df2['iso3num_o'] == 31] = terrain_df.iloc[14,51]
    df2['island_d'][df2['iso3num_d'] == 31] = terrain_df.iloc[14,51]
    df2['landlocked_o'][df2['iso3num_o'] == 31] = terrain_df.iloc[14,52]
    df2['landlocked_d'][df2['iso3num_d'] == 31] = terrain_df.iloc[14,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 108] = terrain_df.iloc[15,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 108] = terrain_df.iloc[15,4]
    df2['desert_o'][df2['iso3num_o'] == 108] = terrain_df.iloc[15,12]
    df2['desert_d'][df2['iso3num_d'] == 108] = terrain_df.iloc[15,12]
    df2['tropical_o'][df2['iso3num_o'] == 108] = terrain_df.iloc[15,13]
    df2['tropical_d'][df2['iso3num_d'] == 108] = terrain_df.iloc[15,13]
    df2['island_o'][df2['iso3num_o'] == 108] = terrain_df.iloc[15,51]
    df2['island_d'][df2['iso3num_d'] == 108] = terrain_df.iloc[15,51]
    df2['landlocked_o'][df2['iso3num_o'] == 108] = terrain_df.iloc[15,52]
    df2['landlocked_d'][df2['iso3num_d'] == 108] = terrain_df.iloc[15,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 56] = terrain_df.iloc[16,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 56] = terrain_df.iloc[16,4]
    df2['desert_o'][df2['iso3num_o'] == 56] = terrain_df.iloc[16,12]
    df2['desert_d'][df2['iso3num_d'] == 56] = terrain_df.iloc[16,12]
    df2['tropical_o'][df2['iso3num_o'] == 56] = terrain_df.iloc[16,13]
    df2['tropical_d'][df2['iso3num_d'] == 56] = terrain_df.iloc[16,13]
    df2['island_o'][df2['iso3num_o'] == 56] = terrain_df.iloc[16,51]
    df2['island_d'][df2['iso3num_d'] == 56] = terrain_df.iloc[16,51]
    df2['landlocked_o'][df2['iso3num_o'] == 56] = terrain_df.iloc[16,52]
    df2['landlocked_d'][df2['iso3num_d'] == 56] = terrain_df.iloc[16,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 204] = terrain_df.iloc[17,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 204] = terrain_df.iloc[17,4]
    df2['desert_o'][df2['iso3num_o'] == 204] = terrain_df.iloc[17,12]
    df2['desert_d'][df2['iso3num_d'] == 204] = terrain_df.iloc[17,12]
    df2['tropical_o'][df2['iso3num_o'] == 204] = terrain_df.iloc[17,13]
    df2['tropical_d'][df2['iso3num_d'] == 204] = terrain_df.iloc[17,13]
    df2['island_o'][df2['iso3num_o'] == 204] = terrain_df.iloc[17,51]
    df2['island_d'][df2['iso3num_d'] == 204] = terrain_df.iloc[17,51]
    df2['landlocked_o'][df2['iso3num_o'] == 204] = terrain_df.iloc[17,52]
    df2['landlocked_d'][df2['iso3num_d'] == 204] = terrain_df.iloc[17,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 854] = terrain_df.iloc[18,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 854] = terrain_df.iloc[18,4]
    df2['desert_o'][df2['iso3num_o'] == 854] = terrain_df.iloc[18,12]
    df2['desert_d'][df2['iso3num_d'] == 854] = terrain_df.iloc[18,12]
    df2['tropical_o'][df2['iso3num_o'] == 854] = terrain_df.iloc[18,13]
    df2['tropical_d'][df2['iso3num_d'] == 854] = terrain_df.iloc[18,13]
    df2['island_o'][df2['iso3num_o'] == 854] = terrain_df.iloc[18,51]
    df2['island_d'][df2['iso3num_d'] == 854] = terrain_df.iloc[18,51]
    df2['landlocked_o'][df2['iso3num_o'] == 854] = terrain_df.iloc[18,52]
    df2['landlocked_d'][df2['iso3num_d'] == 854] = terrain_df.iloc[18,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 50] = terrain_df.iloc[19,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 50] = terrain_df.iloc[19,4]
    df2['desert_o'][df2['iso3num_o'] == 50] = terrain_df.iloc[19,12]
    df2['desert_d'][df2['iso3num_d'] == 50] = terrain_df.iloc[19,12]
    df2['tropical_o'][df2['iso3num_o'] == 50] = terrain_df.iloc[19,13]
    df2['tropical_d'][df2['iso3num_d'] == 50] = terrain_df.iloc[19,13]
    df2['island_o'][df2['iso3num_o'] == 50] = terrain_df.iloc[19,51]
    df2['island_d'][df2['iso3num_d'] == 50] = terrain_df.iloc[19,51]
    df2['landlocked_o'][df2['iso3num_o'] == 50] = terrain_df.iloc[19,52]
    df2['landlocked_d'][df2['iso3num_d'] == 50] = terrain_df.iloc[19,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 100] = terrain_df.iloc[20,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 100] = terrain_df.iloc[20,4]
    df2['desert_o'][df2['iso3num_o'] == 100] = terrain_df.iloc[20,12]
    df2['desert_d'][df2['iso3num_d'] == 100] = terrain_df.iloc[20,12]
    df2['tropical_o'][df2['iso3num_o'] == 100] = terrain_df.iloc[20,13]
    df2['tropical_d'][df2['iso3num_d'] == 100] = terrain_df.iloc[20,13]
    df2['island_o'][df2['iso3num_o'] == 100] = terrain_df.iloc[20,51]
    df2['island_d'][df2['iso3num_d'] == 100] = terrain_df.iloc[20,51]
    df2['landlocked_o'][df2['iso3num_o'] == 100] = terrain_df.iloc[20,52]
    df2['landlocked_d'][df2['iso3num_d'] == 100] = terrain_df.iloc[20,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 48] = terrain_df.iloc[21,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 48] = terrain_df.iloc[21,4]
    df2['desert_o'][df2['iso3num_o'] == 48] = terrain_df.iloc[21,12]
    df2['desert_d'][df2['iso3num_d'] == 48] = terrain_df.iloc[21,12]
    df2['tropical_o'][df2['iso3num_o'] == 48] = terrain_df.iloc[21,13]
    df2['tropical_d'][df2['iso3num_d'] == 48] = terrain_df.iloc[21,13]
    df2['island_o'][df2['iso3num_o'] == 48] = terrain_df.iloc[21,51]
    df2['island_d'][df2['iso3num_d'] == 48] = terrain_df.iloc[21,51]
    df2['landlocked_o'][df2['iso3num_o'] == 48] = terrain_df.iloc[21,52]
    df2['landlocked_d'][df2['iso3num_d'] == 48] = terrain_df.iloc[21,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 44] = terrain_df.iloc[22,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 44] = terrain_df.iloc[22,4]
    df2['desert_o'][df2['iso3num_o'] == 44] = terrain_df.iloc[22,12]
    df2['desert_d'][df2['iso3num_d'] == 44] = terrain_df.iloc[22,12]
    df2['tropical_o'][df2['iso3num_o'] == 44] = terrain_df.iloc[22,13]
    df2['tropical_d'][df2['iso3num_d'] == 44] = terrain_df.iloc[22,13]
    df2['island_o'][df2['iso3num_o'] == 44] = terrain_df.iloc[22,51]
    df2['island_d'][df2['iso3num_d'] == 44] = terrain_df.iloc[22,51]
    df2['landlocked_o'][df2['iso3num_o'] == 44] = terrain_df.iloc[22,52]
    df2['landlocked_d'][df2['iso3num_d'] == 44] = terrain_df.iloc[22,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 70] = terrain_df.iloc[23,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 70] = terrain_df.iloc[23,4]
    df2['desert_o'][df2['iso3num_o'] == 70] = terrain_df.iloc[23,12]
    df2['desert_d'][df2['iso3num_d'] == 70] = terrain_df.iloc[23,12]
    df2['tropical_o'][df2['iso3num_o'] == 70] = terrain_df.iloc[23,13]
    df2['tropical_d'][df2['iso3num_d'] == 70] = terrain_df.iloc[23,13]
    df2['island_o'][df2['iso3num_o'] == 70] = terrain_df.iloc[23,51]
    df2['island_d'][df2['iso3num_d'] == 70] = terrain_df.iloc[23,51]
    df2['landlocked_o'][df2['iso3num_o'] == 70] = terrain_df.iloc[23,52]
    df2['landlocked_d'][df2['iso3num_d'] == 70] = terrain_df.iloc[23,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 112] = terrain_df.iloc[24,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 112] = terrain_df.iloc[24,4]
    df2['desert_o'][df2['iso3num_o'] == 112] = terrain_df.iloc[24,12]
    df2['desert_d'][df2['iso3num_d'] == 112] = terrain_df.iloc[24,12]
    df2['tropical_o'][df2['iso3num_o'] == 112] = terrain_df.iloc[24,13]
    df2['tropical_d'][df2['iso3num_d'] == 112] = terrain_df.iloc[24,13]
    df2['island_o'][df2['iso3num_o'] == 112] = terrain_df.iloc[24,51]
    df2['island_d'][df2['iso3num_d'] == 112] = terrain_df.iloc[24,51]
    df2['landlocked_o'][df2['iso3num_o'] == 112] = terrain_df.iloc[24,52]
    df2['landlocked_d'][df2['iso3num_d'] == 112] = terrain_df.iloc[24,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 84] = terrain_df.iloc[25,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 84] = terrain_df.iloc[25,4]
    df2['desert_o'][df2['iso3num_o'] == 84] = terrain_df.iloc[25,12]
    df2['desert_d'][df2['iso3num_d'] == 84] = terrain_df.iloc[25,12]
    df2['tropical_o'][df2['iso3num_o'] == 84] = terrain_df.iloc[25,13]
    df2['tropical_d'][df2['iso3num_d'] == 84] = terrain_df.iloc[25,13]
    df2['island_o'][df2['iso3num_o'] == 84] = terrain_df.iloc[25,51]
    df2['island_d'][df2['iso3num_d'] == 84] = terrain_df.iloc[25,51]
    df2['landlocked_o'][df2['iso3num_o'] == 84] = terrain_df.iloc[25,52]
    df2['landlocked_d'][df2['iso3num_d'] == 84] = terrain_df.iloc[25,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 60] = terrain_df.iloc[26,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 60] = terrain_df.iloc[26,4]
    df2['desert_o'][df2['iso3num_o'] == 60] = terrain_df.iloc[26,12]
    df2['desert_d'][df2['iso3num_d'] == 60] = terrain_df.iloc[26,12]
    df2['tropical_o'][df2['iso3num_o'] == 60] = terrain_df.iloc[26,13]
    df2['tropical_d'][df2['iso3num_d'] == 60] = terrain_df.iloc[26,13]
    df2['island_o'][df2['iso3num_o'] == 60] = terrain_df.iloc[26,51]
    df2['island_d'][df2['iso3num_d'] == 60] = terrain_df.iloc[26,51]
    df2['landlocked_o'][df2['iso3num_o'] == 60] = terrain_df.iloc[26,52]
    df2['landlocked_d'][df2['iso3num_d'] == 60] = terrain_df.iloc[26,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 68] = terrain_df.iloc[27,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 68] = terrain_df.iloc[27,4]
    df2['desert_o'][df2['iso3num_o'] == 68] = terrain_df.iloc[27,12]
    df2['desert_d'][df2['iso3num_d'] == 68] = terrain_df.iloc[27,12]
    df2['tropical_o'][df2['iso3num_o'] == 68] = terrain_df.iloc[27,13]
    df2['tropical_d'][df2['iso3num_d'] == 68] = terrain_df.iloc[27,13]
    df2['island_o'][df2['iso3num_o'] == 68] = terrain_df.iloc[27,51]
    df2['island_d'][df2['iso3num_d'] == 68] = terrain_df.iloc[27,51]
    df2['landlocked_o'][df2['iso3num_o'] == 68] = terrain_df.iloc[27,52]
    df2['landlocked_d'][df2['iso3num_d'] == 68] = terrain_df.iloc[27,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 76] = terrain_df.iloc[28,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 76] = terrain_df.iloc[28,4]
    df2['desert_o'][df2['iso3num_o'] == 76] = terrain_df.iloc[28,12]
    df2['desert_d'][df2['iso3num_d'] == 76] = terrain_df.iloc[28,12]
    df2['tropical_o'][df2['iso3num_o'] == 76] = terrain_df.iloc[28,13]
    df2['tropical_d'][df2['iso3num_d'] == 76] = terrain_df.iloc[28,13]
    df2['island_o'][df2['iso3num_o'] == 76] = terrain_df.iloc[28,51]
    df2['island_d'][df2['iso3num_d'] == 76] = terrain_df.iloc[28,51]
    df2['landlocked_o'][df2['iso3num_o'] == 76] = terrain_df.iloc[28,52]
    df2['landlocked_d'][df2['iso3num_d'] == 76] = terrain_df.iloc[28,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 52] = terrain_df.iloc[29,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 52] = terrain_df.iloc[29,4]
    df2['desert_o'][df2['iso3num_o'] == 52] = terrain_df.iloc[29,12]
    df2['desert_d'][df2['iso3num_d'] == 52] = terrain_df.iloc[29,12]
    df2['tropical_o'][df2['iso3num_o'] == 52] = terrain_df.iloc[29,13]
    df2['tropical_d'][df2['iso3num_d'] == 52] = terrain_df.iloc[29,13]
    df2['island_o'][df2['iso3num_o'] == 52] = terrain_df.iloc[29,51]
    df2['island_d'][df2['iso3num_d'] == 52] = terrain_df.iloc[29,51]
    df2['landlocked_o'][df2['iso3num_o'] == 52] = terrain_df.iloc[29,52]
    df2['landlocked_d'][df2['iso3num_d'] == 52] = terrain_df.iloc[29,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 96] = terrain_df.iloc[30,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 96] = terrain_df.iloc[30,4]
    df2['desert_o'][df2['iso3num_o'] == 96] = terrain_df.iloc[30,12]
    df2['desert_d'][df2['iso3num_d'] == 96] = terrain_df.iloc[30,12]
    df2['tropical_o'][df2['iso3num_o'] == 96] = terrain_df.iloc[30,13]
    df2['tropical_d'][df2['iso3num_d'] == 96] = terrain_df.iloc[30,13]
    df2['island_o'][df2['iso3num_o'] == 96] = terrain_df.iloc[30,51]
    df2['island_d'][df2['iso3num_d'] == 96] = terrain_df.iloc[30,51]
    df2['landlocked_o'][df2['iso3num_o'] == 96] = terrain_df.iloc[30,52]
    df2['landlocked_d'][df2['iso3num_d'] == 96] = terrain_df.iloc[30,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 64] = terrain_df.iloc[31,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 64] = terrain_df.iloc[31,4]
    df2['desert_o'][df2['iso3num_o'] == 64] = terrain_df.iloc[31,12]
    df2['desert_d'][df2['iso3num_d'] == 64] = terrain_df.iloc[31,12]
    df2['tropical_o'][df2['iso3num_o'] == 64] = terrain_df.iloc[31,13]
    df2['tropical_d'][df2['iso3num_d'] == 64] = terrain_df.iloc[31,13]
    df2['island_o'][df2['iso3num_o'] == 64] = terrain_df.iloc[31,51]
    df2['island_d'][df2['iso3num_d'] == 64] = terrain_df.iloc[31,51]
    df2['landlocked_o'][df2['iso3num_o'] == 64] = terrain_df.iloc[31,52]
    df2['landlocked_d'][df2['iso3num_d'] == 64] = terrain_df.iloc[31,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 72] = terrain_df.iloc[32,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 72] = terrain_df.iloc[32,4]
    df2['desert_o'][df2['iso3num_o'] == 72] = terrain_df.iloc[32,12]
    df2['desert_d'][df2['iso3num_d'] == 72] = terrain_df.iloc[32,12]
    df2['tropical_o'][df2['iso3num_o'] == 72] = terrain_df.iloc[32,13]
    df2['tropical_d'][df2['iso3num_d'] == 72] = terrain_df.iloc[32,13]
    df2['island_o'][df2['iso3num_o'] == 72] = terrain_df.iloc[32,51]
    df2['island_d'][df2['iso3num_d'] == 72] = terrain_df.iloc[32,51]
    df2['landlocked_o'][df2['iso3num_o'] == 72] = terrain_df.iloc[32,52]
    df2['landlocked_d'][df2['iso3num_d'] == 72] = terrain_df.iloc[32,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 140] = terrain_df.iloc[33,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 140] = terrain_df.iloc[33,4]
    df2['desert_o'][df2['iso3num_o'] == 140] = terrain_df.iloc[33,12]
    df2['desert_d'][df2['iso3num_d'] == 140] = terrain_df.iloc[33,12]
    df2['tropical_o'][df2['iso3num_o'] == 140] = terrain_df.iloc[33,13]
    df2['tropical_d'][df2['iso3num_d'] == 140] = terrain_df.iloc[33,13]
    df2['island_o'][df2['iso3num_o'] == 140] = terrain_df.iloc[33,51]
    df2['island_d'][df2['iso3num_d'] == 140] = terrain_df.iloc[33,51]
    df2['landlocked_o'][df2['iso3num_o'] == 140] = terrain_df.iloc[33,52]
    df2['landlocked_d'][df2['iso3num_d'] == 140] = terrain_df.iloc[33,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 124] = terrain_df.iloc[34,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 124] = terrain_df.iloc[34,4]
    df2['desert_o'][df2['iso3num_o'] == 124] = terrain_df.iloc[34,12]
    df2['desert_d'][df2['iso3num_d'] == 124] = terrain_df.iloc[34,12]
    df2['tropical_o'][df2['iso3num_o'] == 124] = terrain_df.iloc[34,13]
    df2['tropical_d'][df2['iso3num_d'] == 124] = terrain_df.iloc[34,13]
    df2['island_o'][df2['iso3num_o'] == 124] = terrain_df.iloc[34,51]
    df2['island_d'][df2['iso3num_d'] == 124] = terrain_df.iloc[34,51]
    df2['landlocked_o'][df2['iso3num_o'] == 124] = terrain_df.iloc[34,52]
    df2['landlocked_d'][df2['iso3num_d'] == 124] = terrain_df.iloc[34,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 166] = terrain_df.iloc[35,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 166] = terrain_df.iloc[35,4]
    df2['desert_o'][df2['iso3num_o'] == 166] = terrain_df.iloc[35,12]
    df2['desert_d'][df2['iso3num_d'] == 166] = terrain_df.iloc[35,12]
    df2['tropical_o'][df2['iso3num_o'] == 166] = terrain_df.iloc[35,13]
    df2['tropical_d'][df2['iso3num_d'] == 166] = terrain_df.iloc[35,13]
    df2['island_o'][df2['iso3num_o'] == 166] = terrain_df.iloc[35,51]
    df2['island_d'][df2['iso3num_d'] == 166] = terrain_df.iloc[35,51]
    df2['landlocked_o'][df2['iso3num_o'] == 166] = terrain_df.iloc[35,52]
    df2['landlocked_d'][df2['iso3num_d'] == 166] = terrain_df.iloc[35,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 756] = terrain_df.iloc[36,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 756] = terrain_df.iloc[36,4]
    df2['desert_o'][df2['iso3num_o'] == 756] = terrain_df.iloc[36,12]
    df2['desert_d'][df2['iso3num_d'] == 756] = terrain_df.iloc[36,12]
    df2['tropical_o'][df2['iso3num_o'] == 756] = terrain_df.iloc[36,13]
    df2['tropical_d'][df2['iso3num_d'] == 756] = terrain_df.iloc[36,13]
    df2['island_o'][df2['iso3num_o'] == 756] = terrain_df.iloc[36,51]
    df2['island_d'][df2['iso3num_d'] == 756] = terrain_df.iloc[36,51]
    df2['landlocked_o'][df2['iso3num_o'] == 756] = terrain_df.iloc[36,52]
    df2['landlocked_d'][df2['iso3num_d'] == 756] = terrain_df.iloc[36,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 152] = terrain_df.iloc[37,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 152] = terrain_df.iloc[37,4]
    df2['desert_o'][df2['iso3num_o'] == 152] = terrain_df.iloc[37,12]
    df2['desert_d'][df2['iso3num_d'] == 152] = terrain_df.iloc[37,12]
    df2['tropical_o'][df2['iso3num_o'] == 152] = terrain_df.iloc[37,13]
    df2['tropical_d'][df2['iso3num_d'] == 152] = terrain_df.iloc[37,13]
    df2['island_o'][df2['iso3num_o'] == 152] = terrain_df.iloc[37,51]
    df2['island_d'][df2['iso3num_d'] == 152] = terrain_df.iloc[37,51]
    df2['landlocked_o'][df2['iso3num_o'] == 152] = terrain_df.iloc[37,52]
    df2['landlocked_d'][df2['iso3num_d'] == 152] = terrain_df.iloc[37,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 156] = terrain_df.iloc[38,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 156] = terrain_df.iloc[38,4]
    df2['desert_o'][df2['iso3num_o'] == 156] = terrain_df.iloc[38,12]
    df2['desert_d'][df2['iso3num_d'] == 156] = terrain_df.iloc[38,12]
    df2['tropical_o'][df2['iso3num_o'] == 156] = terrain_df.iloc[38,13]
    df2['tropical_d'][df2['iso3num_d'] == 156] = terrain_df.iloc[38,13]
    df2['island_o'][df2['iso3num_o'] == 156] = terrain_df.iloc[38,51]
    df2['island_d'][df2['iso3num_d'] == 156] = terrain_df.iloc[38,51]
    df2['landlocked_o'][df2['iso3num_o'] == 156] = terrain_df.iloc[38,52]
    df2['landlocked_d'][df2['iso3num_d'] == 156] = terrain_df.iloc[38,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 384] = terrain_df.iloc[39,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 384] = terrain_df.iloc[39,4]
    df2['desert_o'][df2['iso3num_o'] == 384] = terrain_df.iloc[39,12]
    df2['desert_d'][df2['iso3num_d'] == 384] = terrain_df.iloc[39,12]
    df2['tropical_o'][df2['iso3num_o'] == 384] = terrain_df.iloc[39,13]
    df2['tropical_d'][df2['iso3num_d'] == 384] = terrain_df.iloc[39,13]
    df2['island_o'][df2['iso3num_o'] == 384] = terrain_df.iloc[39,51]
    df2['island_d'][df2['iso3num_d'] == 384] = terrain_df.iloc[39,51]
    df2['landlocked_o'][df2['iso3num_o'] == 384] = terrain_df.iloc[39,52]
    df2['landlocked_d'][df2['iso3num_d'] == 384] = terrain_df.iloc[39,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 120] = terrain_df.iloc[40,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 120] = terrain_df.iloc[40,4]
    df2['desert_o'][df2['iso3num_o'] == 120] = terrain_df.iloc[40,12]
    df2['desert_d'][df2['iso3num_d'] == 120] = terrain_df.iloc[40,12]
    df2['tropical_o'][df2['iso3num_o'] == 120] = terrain_df.iloc[40,13]
    df2['tropical_d'][df2['iso3num_d'] == 120] = terrain_df.iloc[40,13]
    df2['island_o'][df2['iso3num_o'] == 120] = terrain_df.iloc[40,51]
    df2['island_d'][df2['iso3num_d'] == 120] = terrain_df.iloc[40,51]
    df2['landlocked_o'][df2['iso3num_o'] == 120] = terrain_df.iloc[40,52]
    df2['landlocked_d'][df2['iso3num_d'] == 120] = terrain_df.iloc[40,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 180] = terrain_df.iloc[41,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 180] = terrain_df.iloc[41,4]
    df2['desert_o'][df2['iso3num_o'] == 180] = terrain_df.iloc[41,12]
    df2['desert_d'][df2['iso3num_d'] == 180] = terrain_df.iloc[41,12]
    df2['tropical_o'][df2['iso3num_o'] == 180] = terrain_df.iloc[41,13]
    df2['tropical_d'][df2['iso3num_d'] == 180] = terrain_df.iloc[41,13]
    df2['island_o'][df2['iso3num_o'] == 180] = terrain_df.iloc[41,51]
    df2['island_d'][df2['iso3num_d'] == 180] = terrain_df.iloc[41,51]
    df2['landlocked_o'][df2['iso3num_o'] == 180] = terrain_df.iloc[41,52]
    df2['landlocked_d'][df2['iso3num_d'] == 180] = terrain_df.iloc[41,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 178] = terrain_df.iloc[42,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 178] = terrain_df.iloc[42,4]
    df2['desert_o'][df2['iso3num_o'] == 178] = terrain_df.iloc[42,12]
    df2['desert_d'][df2['iso3num_d'] == 178] = terrain_df.iloc[42,12]
    df2['tropical_o'][df2['iso3num_o'] == 178] = terrain_df.iloc[42,13]
    df2['tropical_d'][df2['iso3num_d'] == 178] = terrain_df.iloc[42,13]
    df2['island_o'][df2['iso3num_o'] == 178] = terrain_df.iloc[42,51]
    df2['island_d'][df2['iso3num_d'] == 178] = terrain_df.iloc[42,51]
    df2['landlocked_o'][df2['iso3num_o'] == 178] = terrain_df.iloc[42,52]
    df2['landlocked_d'][df2['iso3num_d'] == 178] = terrain_df.iloc[42,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 184] = terrain_df.iloc[43,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 184] = terrain_df.iloc[43,4]
    df2['desert_o'][df2['iso3num_o'] == 184] = terrain_df.iloc[43,12]
    df2['desert_d'][df2['iso3num_d'] == 184] = terrain_df.iloc[43,12]
    df2['tropical_o'][df2['iso3num_o'] == 184] = terrain_df.iloc[43,13]
    df2['tropical_d'][df2['iso3num_d'] == 184] = terrain_df.iloc[43,13]
    df2['island_o'][df2['iso3num_o'] == 184] = terrain_df.iloc[43,51]
    df2['island_d'][df2['iso3num_d'] == 184] = terrain_df.iloc[43,51]
    df2['landlocked_o'][df2['iso3num_o'] == 184] = terrain_df.iloc[43,52]
    df2['landlocked_d'][df2['iso3num_d'] == 184] = terrain_df.iloc[43,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 170] = terrain_df.iloc[44,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 170] = terrain_df.iloc[44,4]
    df2['desert_o'][df2['iso3num_o'] == 170] = terrain_df.iloc[44,12]
    df2['desert_d'][df2['iso3num_d'] == 170] = terrain_df.iloc[44,12]
    df2['tropical_o'][df2['iso3num_o'] == 170] = terrain_df.iloc[44,13]
    df2['tropical_d'][df2['iso3num_d'] == 170] = terrain_df.iloc[44,13]
    df2['island_o'][df2['iso3num_o'] == 170] = terrain_df.iloc[44,51]
    df2['island_d'][df2['iso3num_d'] == 170] = terrain_df.iloc[44,51]
    df2['landlocked_o'][df2['iso3num_o'] == 170] = terrain_df.iloc[44,52]
    df2['landlocked_d'][df2['iso3num_d'] == 170] = terrain_df.iloc[44,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 174] = terrain_df.iloc[45,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 174] = terrain_df.iloc[45,4]
    df2['desert_o'][df2['iso3num_o'] == 174] = terrain_df.iloc[45,12]
    df2['desert_d'][df2['iso3num_d'] == 174] = terrain_df.iloc[45,12]
    df2['tropical_o'][df2['iso3num_o'] == 174] = terrain_df.iloc[45,13]
    df2['tropical_d'][df2['iso3num_d'] == 174] = terrain_df.iloc[45,13]
    df2['island_o'][df2['iso3num_o'] == 174] = terrain_df.iloc[45,51]
    df2['island_d'][df2['iso3num_d'] == 174] = terrain_df.iloc[45,51]
    df2['landlocked_o'][df2['iso3num_o'] == 174] = terrain_df.iloc[45,52]
    df2['landlocked_d'][df2['iso3num_d'] == 174] = terrain_df.iloc[45,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 132] = terrain_df.iloc[46,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 132] = terrain_df.iloc[46,4]
    df2['desert_o'][df2['iso3num_o'] == 132] = terrain_df.iloc[46,12]
    df2['desert_d'][df2['iso3num_d'] == 132] = terrain_df.iloc[46,12]
    df2['tropical_o'][df2['iso3num_o'] == 132] = terrain_df.iloc[46,13]
    df2['tropical_d'][df2['iso3num_d'] == 132] = terrain_df.iloc[46,13]
    df2['island_o'][df2['iso3num_o'] == 132] = terrain_df.iloc[46,51]
    df2['island_d'][df2['iso3num_d'] == 132] = terrain_df.iloc[46,51]
    df2['landlocked_o'][df2['iso3num_o'] == 132] = terrain_df.iloc[46,52]
    df2['landlocked_d'][df2['iso3num_d'] == 132] = terrain_df.iloc[46,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 188] = terrain_df.iloc[47,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 188] = terrain_df.iloc[47,4]
    df2['desert_o'][df2['iso3num_o'] == 188] = terrain_df.iloc[47,12]
    df2['desert_d'][df2['iso3num_d'] == 188] = terrain_df.iloc[47,12]
    df2['tropical_o'][df2['iso3num_o'] == 188] = terrain_df.iloc[47,13]
    df2['tropical_d'][df2['iso3num_d'] == 188] = terrain_df.iloc[47,13]
    df2['island_o'][df2['iso3num_o'] == 188] = terrain_df.iloc[47,51]
    df2['island_d'][df2['iso3num_d'] == 188] = terrain_df.iloc[47,51]
    df2['landlocked_o'][df2['iso3num_o'] == 188] = terrain_df.iloc[47,52]
    df2['landlocked_d'][df2['iso3num_d'] == 188] = terrain_df.iloc[47,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 192] = terrain_df.iloc[48,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 192] = terrain_df.iloc[48,4]
    df2['desert_o'][df2['iso3num_o'] == 192] = terrain_df.iloc[48,12]
    df2['desert_d'][df2['iso3num_d'] == 192] = terrain_df.iloc[48,12]
    df2['tropical_o'][df2['iso3num_o'] == 192] = terrain_df.iloc[48,13]
    df2['tropical_d'][df2['iso3num_d'] == 192] = terrain_df.iloc[48,13]
    df2['island_o'][df2['iso3num_o'] == 192] = terrain_df.iloc[48,51]
    df2['island_d'][df2['iso3num_d'] == 192] = terrain_df.iloc[48,51]
    df2['landlocked_o'][df2['iso3num_o'] == 192] = terrain_df.iloc[48,52]
    df2['landlocked_d'][df2['iso3num_d'] == 192] = terrain_df.iloc[48,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 162] = terrain_df.iloc[49,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 162] = terrain_df.iloc[49,4]
    df2['desert_o'][df2['iso3num_o'] == 162] = terrain_df.iloc[49,12]
    df2['desert_d'][df2['iso3num_d'] == 162] = terrain_df.iloc[49,12]
    df2['tropical_o'][df2['iso3num_o'] == 162] = terrain_df.iloc[49,13]
    df2['tropical_d'][df2['iso3num_d'] == 162] = terrain_df.iloc[49,13]
    df2['island_o'][df2['iso3num_o'] == 162] = terrain_df.iloc[49,51]
    df2['island_d'][df2['iso3num_d'] == 162] = terrain_df.iloc[49,51]
    df2['landlocked_o'][df2['iso3num_o'] == 162] = terrain_df.iloc[49,52]
    df2['landlocked_d'][df2['iso3num_d'] == 162] = terrain_df.iloc[49,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 136] = terrain_df.iloc[50,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 136] = terrain_df.iloc[50,4]
    df2['desert_o'][df2['iso3num_o'] == 136] = terrain_df.iloc[50,12]
    df2['desert_d'][df2['iso3num_d'] == 136] = terrain_df.iloc[50,12]
    df2['tropical_o'][df2['iso3num_o'] == 136] = terrain_df.iloc[50,13]
    df2['tropical_d'][df2['iso3num_d'] == 136] = terrain_df.iloc[50,13]
    df2['island_o'][df2['iso3num_o'] == 136] = terrain_df.iloc[50,51]
    df2['island_d'][df2['iso3num_d'] == 136] = terrain_df.iloc[50,51]
    df2['landlocked_o'][df2['iso3num_o'] == 136] = terrain_df.iloc[50,52]
    df2['landlocked_d'][df2['iso3num_d'] == 136] = terrain_df.iloc[50,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 196] = terrain_df.iloc[51,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 196] = terrain_df.iloc[51,4]
    df2['desert_o'][df2['iso3num_o'] == 196] = terrain_df.iloc[51,12]
    df2['desert_d'][df2['iso3num_d'] == 196] = terrain_df.iloc[51,12]
    df2['tropical_o'][df2['iso3num_o'] == 196] = terrain_df.iloc[51,13]
    df2['tropical_d'][df2['iso3num_d'] == 196] = terrain_df.iloc[51,13]
    df2['island_o'][df2['iso3num_o'] == 196] = terrain_df.iloc[51,51]
    df2['island_d'][df2['iso3num_d'] == 196] = terrain_df.iloc[51,51]
    df2['landlocked_o'][df2['iso3num_o'] == 196] = terrain_df.iloc[51,52]
    df2['landlocked_d'][df2['iso3num_d'] == 196] = terrain_df.iloc[51,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 203] = terrain_df.iloc[52,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 203] = terrain_df.iloc[52,4]
    df2['desert_o'][df2['iso3num_o'] == 203] = terrain_df.iloc[52,12]
    df2['desert_d'][df2['iso3num_d'] == 203] = terrain_df.iloc[52,12]
    df2['tropical_o'][df2['iso3num_o'] == 203] = terrain_df.iloc[52,13]
    df2['tropical_d'][df2['iso3num_d'] == 203] = terrain_df.iloc[52,13]
    df2['island_o'][df2['iso3num_o'] == 203] = terrain_df.iloc[52,51]
    df2['island_d'][df2['iso3num_d'] == 203] = terrain_df.iloc[52,51]
    df2['landlocked_o'][df2['iso3num_o'] == 203] = terrain_df.iloc[52,52]
    df2['landlocked_d'][df2['iso3num_d'] == 203] = terrain_df.iloc[52,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 276] = terrain_df.iloc[53,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 276] = terrain_df.iloc[53,4]
    df2['desert_o'][df2['iso3num_o'] == 276] = terrain_df.iloc[53,12]
    df2['desert_d'][df2['iso3num_d'] == 276] = terrain_df.iloc[53,12]
    df2['tropical_o'][df2['iso3num_o'] == 276] = terrain_df.iloc[53,13]
    df2['tropical_d'][df2['iso3num_d'] == 276] = terrain_df.iloc[53,13]
    df2['island_o'][df2['iso3num_o'] == 276] = terrain_df.iloc[53,51]
    df2['island_d'][df2['iso3num_d'] == 276] = terrain_df.iloc[53,51]
    df2['landlocked_o'][df2['iso3num_o'] == 276] = terrain_df.iloc[53,52]
    df2['landlocked_d'][df2['iso3num_d'] == 276] = terrain_df.iloc[53,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 262] = terrain_df.iloc[54,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 262] = terrain_df.iloc[54,4]
    df2['desert_o'][df2['iso3num_o'] == 262] = terrain_df.iloc[54,12]
    df2['desert_d'][df2['iso3num_d'] == 262] = terrain_df.iloc[54,12]
    df2['tropical_o'][df2['iso3num_o'] == 262] = terrain_df.iloc[54,13]
    df2['tropical_d'][df2['iso3num_d'] == 262] = terrain_df.iloc[54,13]
    df2['island_o'][df2['iso3num_o'] == 262] = terrain_df.iloc[54,51]
    df2['island_d'][df2['iso3num_d'] == 262] = terrain_df.iloc[54,51]
    df2['landlocked_o'][df2['iso3num_o'] == 262] = terrain_df.iloc[54,52]
    df2['landlocked_d'][df2['iso3num_d'] == 262] = terrain_df.iloc[54,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 212] = terrain_df.iloc[55,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 212] = terrain_df.iloc[55,4]
    df2['desert_o'][df2['iso3num_o'] == 212] = terrain_df.iloc[55,12]
    df2['desert_d'][df2['iso3num_d'] == 212] = terrain_df.iloc[55,12]
    df2['tropical_o'][df2['iso3num_o'] == 212] = terrain_df.iloc[55,13]
    df2['tropical_d'][df2['iso3num_d'] == 212] = terrain_df.iloc[55,13]
    df2['island_o'][df2['iso3num_o'] == 212] = terrain_df.iloc[55,51]
    df2['island_d'][df2['iso3num_d'] == 212] = terrain_df.iloc[55,51]
    df2['landlocked_o'][df2['iso3num_o'] == 212] = terrain_df.iloc[55,52]
    df2['landlocked_d'][df2['iso3num_d'] == 212] = terrain_df.iloc[55,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 208] = terrain_df.iloc[56,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 208] = terrain_df.iloc[56,4]
    df2['desert_o'][df2['iso3num_o'] == 208] = terrain_df.iloc[56,12]
    df2['desert_d'][df2['iso3num_d'] == 208] = terrain_df.iloc[56,12]
    df2['tropical_o'][df2['iso3num_o'] == 208] = terrain_df.iloc[56,13]
    df2['tropical_d'][df2['iso3num_d'] == 208] = terrain_df.iloc[56,13]
    df2['island_o'][df2['iso3num_o'] == 208] = terrain_df.iloc[56,51]
    df2['island_d'][df2['iso3num_d'] == 208] = terrain_df.iloc[56,51]
    df2['landlocked_o'][df2['iso3num_o'] == 208] = terrain_df.iloc[56,52]
    df2['landlocked_d'][df2['iso3num_d'] == 208] = terrain_df.iloc[56,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 214] = terrain_df.iloc[57,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 214] = terrain_df.iloc[57,4]
    df2['desert_o'][df2['iso3num_o'] == 214] = terrain_df.iloc[57,12]
    df2['desert_d'][df2['iso3num_d'] == 214] = terrain_df.iloc[57,12]
    df2['tropical_o'][df2['iso3num_o'] == 214] = terrain_df.iloc[57,13]
    df2['tropical_d'][df2['iso3num_d'] == 214] = terrain_df.iloc[57,13]
    df2['island_o'][df2['iso3num_o'] == 214] = terrain_df.iloc[57,51]
    df2['island_d'][df2['iso3num_d'] == 214] = terrain_df.iloc[57,51]
    df2['landlocked_o'][df2['iso3num_o'] == 214] = terrain_df.iloc[57,52]
    df2['landlocked_d'][df2['iso3num_d'] == 214] = terrain_df.iloc[57,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 12] = terrain_df.iloc[58,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 12] = terrain_df.iloc[58,4]
    df2['desert_o'][df2['iso3num_o'] == 12] = terrain_df.iloc[58,12]
    df2['desert_d'][df2['iso3num_d'] == 12] = terrain_df.iloc[58,12]
    df2['tropical_o'][df2['iso3num_o'] == 12] = terrain_df.iloc[58,13]
    df2['tropical_d'][df2['iso3num_d'] == 12] = terrain_df.iloc[58,13]
    df2['island_o'][df2['iso3num_o'] == 12] = terrain_df.iloc[58,51]
    df2['island_d'][df2['iso3num_d'] == 12] = terrain_df.iloc[58,51]
    df2['landlocked_o'][df2['iso3num_o'] == 12] = terrain_df.iloc[58,52]
    df2['landlocked_d'][df2['iso3num_d'] == 12] = terrain_df.iloc[58,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 218] = terrain_df.iloc[59,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 218] = terrain_df.iloc[59,4]
    df2['desert_o'][df2['iso3num_o'] == 218] = terrain_df.iloc[59,12]
    df2['desert_d'][df2['iso3num_d'] == 218] = terrain_df.iloc[59,12]
    df2['tropical_o'][df2['iso3num_o'] == 218] = terrain_df.iloc[59,13]
    df2['tropical_d'][df2['iso3num_d'] == 218] = terrain_df.iloc[59,13]
    df2['island_o'][df2['iso3num_o'] == 218] = terrain_df.iloc[59,51]
    df2['island_d'][df2['iso3num_d'] == 218] = terrain_df.iloc[59,51]
    df2['landlocked_o'][df2['iso3num_o'] == 218] = terrain_df.iloc[59,52]
    df2['landlocked_d'][df2['iso3num_d'] == 218] = terrain_df.iloc[59,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 818] = terrain_df.iloc[60,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 818] = terrain_df.iloc[60,4]
    df2['desert_o'][df2['iso3num_o'] == 818] = terrain_df.iloc[60,12]
    df2['desert_d'][df2['iso3num_d'] == 818] = terrain_df.iloc[60,12]
    df2['tropical_o'][df2['iso3num_o'] == 818] = terrain_df.iloc[60,13]
    df2['tropical_d'][df2['iso3num_d'] == 818] = terrain_df.iloc[60,13]
    df2['island_o'][df2['iso3num_o'] == 818] = terrain_df.iloc[60,51]
    df2['island_d'][df2['iso3num_d'] == 818] = terrain_df.iloc[60,51]
    df2['landlocked_o'][df2['iso3num_o'] == 818] = terrain_df.iloc[60,52]
    df2['landlocked_d'][df2['iso3num_d'] == 818] = terrain_df.iloc[60,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 232] = terrain_df.iloc[61,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 232] = terrain_df.iloc[61,4]
    df2['desert_o'][df2['iso3num_o'] == 232] = terrain_df.iloc[61,12]
    df2['desert_d'][df2['iso3num_d'] == 232] = terrain_df.iloc[61,12]
    df2['tropical_o'][df2['iso3num_o'] == 232] = terrain_df.iloc[61,13]
    df2['tropical_d'][df2['iso3num_d'] == 232] = terrain_df.iloc[61,13]
    df2['island_o'][df2['iso3num_o'] == 232] = terrain_df.iloc[61,51]
    df2['island_d'][df2['iso3num_d'] == 232] = terrain_df.iloc[61,51]
    df2['landlocked_o'][df2['iso3num_o'] == 232] = terrain_df.iloc[61,52]
    df2['landlocked_d'][df2['iso3num_d'] == 232] = terrain_df.iloc[61,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 732] = terrain_df.iloc[62,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 732] = terrain_df.iloc[62,4]
    df2['desert_o'][df2['iso3num_o'] == 732] = terrain_df.iloc[62,12]
    df2['desert_d'][df2['iso3num_d'] == 732] = terrain_df.iloc[62,12]
    df2['tropical_o'][df2['iso3num_o'] == 732] = terrain_df.iloc[62,13]
    df2['tropical_d'][df2['iso3num_d'] == 732] = terrain_df.iloc[62,13]
    df2['island_o'][df2['iso3num_o'] == 732] = terrain_df.iloc[62,51]
    df2['island_d'][df2['iso3num_d'] == 732] = terrain_df.iloc[62,51]
    df2['landlocked_o'][df2['iso3num_o'] == 732] = terrain_df.iloc[62,52]
    df2['landlocked_d'][df2['iso3num_d'] == 732] = terrain_df.iloc[62,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 724] = terrain_df.iloc[63,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 724] = terrain_df.iloc[63,4]
    df2['desert_o'][df2['iso3num_o'] == 724] = terrain_df.iloc[63,12]
    df2['desert_d'][df2['iso3num_d'] == 724] = terrain_df.iloc[63,12]
    df2['tropical_o'][df2['iso3num_o'] == 724] = terrain_df.iloc[63,13]
    df2['tropical_d'][df2['iso3num_d'] == 724] = terrain_df.iloc[63,13]
    df2['island_o'][df2['iso3num_o'] == 724] = terrain_df.iloc[63,51]
    df2['island_d'][df2['iso3num_d'] == 724] = terrain_df.iloc[63,51]
    df2['landlocked_o'][df2['iso3num_o'] == 724] = terrain_df.iloc[63,52]
    df2['landlocked_d'][df2['iso3num_d'] == 724] = terrain_df.iloc[63,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 233] = terrain_df.iloc[64,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 233] = terrain_df.iloc[64,4]
    df2['desert_o'][df2['iso3num_o'] == 233] = terrain_df.iloc[64,12]
    df2['desert_d'][df2['iso3num_d'] == 233] = terrain_df.iloc[64,12]
    df2['tropical_o'][df2['iso3num_o'] == 233] = terrain_df.iloc[64,13]
    df2['tropical_d'][df2['iso3num_d'] == 233] = terrain_df.iloc[64,13]
    df2['island_o'][df2['iso3num_o'] == 233] = terrain_df.iloc[64,51]
    df2['island_d'][df2['iso3num_d'] == 233] = terrain_df.iloc[64,51]
    df2['landlocked_o'][df2['iso3num_o'] == 233] = terrain_df.iloc[64,52]
    df2['landlocked_d'][df2['iso3num_d'] == 233] = terrain_df.iloc[64,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 231] = terrain_df.iloc[65,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 231] = terrain_df.iloc[65,4]
    df2['desert_o'][df2['iso3num_o'] == 231] = terrain_df.iloc[65,12]
    df2['desert_d'][df2['iso3num_d'] == 231] = terrain_df.iloc[65,12]
    df2['tropical_o'][df2['iso3num_o'] == 231] = terrain_df.iloc[65,13]
    df2['tropical_d'][df2['iso3num_d'] == 231] = terrain_df.iloc[65,13]
    df2['island_o'][df2['iso3num_o'] == 231] = terrain_df.iloc[65,51]
    df2['island_d'][df2['iso3num_d'] == 231] = terrain_df.iloc[65,51]
    df2['landlocked_o'][df2['iso3num_o'] == 231] = terrain_df.iloc[65,52]
    df2['landlocked_d'][df2['iso3num_d'] == 231] = terrain_df.iloc[65,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 246] = terrain_df.iloc[66,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 246] = terrain_df.iloc[66,4]
    df2['desert_o'][df2['iso3num_o'] == 246] = terrain_df.iloc[66,12]
    df2['desert_d'][df2['iso3num_d'] == 246] = terrain_df.iloc[66,12]
    df2['tropical_o'][df2['iso3num_o'] == 246] = terrain_df.iloc[66,13]
    df2['tropical_d'][df2['iso3num_d'] == 246] = terrain_df.iloc[66,13]
    df2['island_o'][df2['iso3num_o'] == 246] = terrain_df.iloc[66,51]
    df2['island_d'][df2['iso3num_d'] == 246] = terrain_df.iloc[66,51]
    df2['landlocked_o'][df2['iso3num_o'] == 246] = terrain_df.iloc[66,52]
    df2['landlocked_d'][df2['iso3num_d'] == 246] = terrain_df.iloc[66,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 242] = terrain_df.iloc[67,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 242] = terrain_df.iloc[67,4]
    df2['desert_o'][df2['iso3num_o'] == 242] = terrain_df.iloc[67,12]
    df2['desert_d'][df2['iso3num_d'] == 242] = terrain_df.iloc[67,12]
    df2['tropical_o'][df2['iso3num_o'] == 242] = terrain_df.iloc[67,13]
    df2['tropical_d'][df2['iso3num_d'] == 242] = terrain_df.iloc[67,13]
    df2['island_o'][df2['iso3num_o'] == 242] = terrain_df.iloc[67,51]
    df2['island_d'][df2['iso3num_d'] == 242] = terrain_df.iloc[67,51]
    df2['landlocked_o'][df2['iso3num_o'] == 242] = terrain_df.iloc[67,52]
    df2['landlocked_d'][df2['iso3num_d'] == 242] = terrain_df.iloc[67,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 238] = terrain_df.iloc[68,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 238] = terrain_df.iloc[68,4]
    df2['desert_o'][df2['iso3num_o'] == 238] = terrain_df.iloc[68,12]
    df2['desert_d'][df2['iso3num_d'] == 238] = terrain_df.iloc[68,12]
    df2['tropical_o'][df2['iso3num_o'] == 238] = terrain_df.iloc[68,13]
    df2['tropical_d'][df2['iso3num_d'] == 238] = terrain_df.iloc[68,13]
    df2['island_o'][df2['iso3num_o'] == 238] = terrain_df.iloc[68,51]
    df2['island_d'][df2['iso3num_d'] == 238] = terrain_df.iloc[68,51]
    df2['landlocked_o'][df2['iso3num_o'] == 238] = terrain_df.iloc[68,52]
    df2['landlocked_d'][df2['iso3num_d'] == 238] = terrain_df.iloc[68,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 250] = terrain_df.iloc[69,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 250] = terrain_df.iloc[69,4]
    df2['desert_o'][df2['iso3num_o'] == 250] = terrain_df.iloc[69,12]
    df2['desert_d'][df2['iso3num_d'] == 250] = terrain_df.iloc[69,12]
    df2['tropical_o'][df2['iso3num_o'] == 250] = terrain_df.iloc[69,13]
    df2['tropical_d'][df2['iso3num_d'] == 250] = terrain_df.iloc[69,13]
    df2['island_o'][df2['iso3num_o'] == 250] = terrain_df.iloc[69,51]
    df2['island_d'][df2['iso3num_d'] == 250] = terrain_df.iloc[69,51]
    df2['landlocked_o'][df2['iso3num_o'] == 250] = terrain_df.iloc[69,52]
    df2['landlocked_d'][df2['iso3num_d'] == 250] = terrain_df.iloc[69,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 234] = terrain_df.iloc[70,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 234] = terrain_df.iloc[70,4]
    df2['desert_o'][df2['iso3num_o'] == 234] = terrain_df.iloc[70,12]
    df2['desert_d'][df2['iso3num_d'] == 234] = terrain_df.iloc[70,12]
    df2['tropical_o'][df2['iso3num_o'] == 234] = terrain_df.iloc[70,13]
    df2['tropical_d'][df2['iso3num_d'] == 234] = terrain_df.iloc[70,13]
    df2['island_o'][df2['iso3num_o'] == 234] = terrain_df.iloc[70,51]
    df2['island_d'][df2['iso3num_d'] == 234] = terrain_df.iloc[70,51]
    df2['landlocked_o'][df2['iso3num_o'] == 234] = terrain_df.iloc[70,52]
    df2['landlocked_d'][df2['iso3num_d'] == 234] = terrain_df.iloc[70,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 583] = terrain_df.iloc[71,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 583] = terrain_df.iloc[71,4]
    df2['desert_o'][df2['iso3num_o'] == 583] = terrain_df.iloc[71,12]
    df2['desert_d'][df2['iso3num_d'] == 583] = terrain_df.iloc[71,12]
    df2['tropical_o'][df2['iso3num_o'] == 583] = terrain_df.iloc[71,13]
    df2['tropical_d'][df2['iso3num_d'] == 583] = terrain_df.iloc[71,13]
    df2['island_o'][df2['iso3num_o'] == 583] = terrain_df.iloc[71,51]
    df2['island_d'][df2['iso3num_d'] == 583] = terrain_df.iloc[71,51]
    df2['landlocked_o'][df2['iso3num_o'] == 583] = terrain_df.iloc[71,52]
    df2['landlocked_d'][df2['iso3num_d'] == 583] = terrain_df.iloc[71,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 266] = terrain_df.iloc[72,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 266] = terrain_df.iloc[72,4]
    df2['desert_o'][df2['iso3num_o'] == 266] = terrain_df.iloc[72,12]
    df2['desert_d'][df2['iso3num_d'] == 266] = terrain_df.iloc[72,12]
    df2['tropical_o'][df2['iso3num_o'] == 266] = terrain_df.iloc[72,13]
    df2['tropical_d'][df2['iso3num_d'] == 266] = terrain_df.iloc[72,13]
    df2['island_o'][df2['iso3num_o'] == 266] = terrain_df.iloc[72,51]
    df2['island_d'][df2['iso3num_d'] == 266] = terrain_df.iloc[72,51]
    df2['landlocked_o'][df2['iso3num_o'] == 266] = terrain_df.iloc[72,52]
    df2['landlocked_d'][df2['iso3num_d'] == 266] = terrain_df.iloc[72,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 826] = terrain_df.iloc[73,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 826] = terrain_df.iloc[73,4]
    df2['desert_o'][df2['iso3num_o'] == 826] = terrain_df.iloc[73,12]
    df2['desert_d'][df2['iso3num_d'] == 826] = terrain_df.iloc[73,12]
    df2['tropical_o'][df2['iso3num_o'] == 826] = terrain_df.iloc[73,13]
    df2['tropical_d'][df2['iso3num_d'] == 826] = terrain_df.iloc[73,13]
    df2['island_o'][df2['iso3num_o'] == 826] = terrain_df.iloc[73,51]
    df2['island_d'][df2['iso3num_d'] == 826] = terrain_df.iloc[73,51]
    df2['landlocked_o'][df2['iso3num_o'] == 826] = terrain_df.iloc[73,52]
    df2['landlocked_d'][df2['iso3num_d'] == 826] = terrain_df.iloc[73,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 268] = terrain_df.iloc[74,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 268] = terrain_df.iloc[74,4]
    df2['desert_o'][df2['iso3num_o'] == 268] = terrain_df.iloc[74,12]
    df2['desert_d'][df2['iso3num_d'] == 268] = terrain_df.iloc[74,12]
    df2['tropical_o'][df2['iso3num_o'] == 268] = terrain_df.iloc[74,13]
    df2['tropical_d'][df2['iso3num_d'] == 268] = terrain_df.iloc[74,13]
    df2['island_o'][df2['iso3num_o'] == 268] = terrain_df.iloc[74,51]
    df2['island_d'][df2['iso3num_d'] == 268] = terrain_df.iloc[74,51]
    df2['landlocked_o'][df2['iso3num_o'] == 268] = terrain_df.iloc[74,52]
    df2['landlocked_d'][df2['iso3num_d'] == 268] = terrain_df.iloc[74,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 288] = terrain_df.iloc[75,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 288] = terrain_df.iloc[75,4]
    df2['desert_o'][df2['iso3num_o'] == 288] = terrain_df.iloc[75,12]
    df2['desert_d'][df2['iso3num_d'] == 288] = terrain_df.iloc[75,12]
    df2['tropical_o'][df2['iso3num_o'] == 288] = terrain_df.iloc[75,13]
    df2['tropical_d'][df2['iso3num_d'] == 288] = terrain_df.iloc[75,13]
    df2['island_o'][df2['iso3num_o'] == 288] = terrain_df.iloc[75,51]
    df2['island_d'][df2['iso3num_d'] == 288] = terrain_df.iloc[75,51]
    df2['landlocked_o'][df2['iso3num_o'] == 288] = terrain_df.iloc[75,52]
    df2['landlocked_d'][df2['iso3num_d'] == 288] = terrain_df.iloc[75,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 292] = terrain_df.iloc[76,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 292] = terrain_df.iloc[76,4]
    df2['desert_o'][df2['iso3num_o'] == 292] = terrain_df.iloc[76,12]
    df2['desert_d'][df2['iso3num_d'] == 292] = terrain_df.iloc[76,12]
    df2['tropical_o'][df2['iso3num_o'] == 292] = terrain_df.iloc[76,13]
    df2['tropical_d'][df2['iso3num_d'] == 292] = terrain_df.iloc[76,13]
    df2['island_o'][df2['iso3num_o'] == 292] = terrain_df.iloc[76,51]
    df2['island_d'][df2['iso3num_d'] == 292] = terrain_df.iloc[76,51]
    df2['landlocked_o'][df2['iso3num_o'] == 292] = terrain_df.iloc[76,52]
    df2['landlocked_d'][df2['iso3num_d'] == 292] = terrain_df.iloc[76,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 324] = terrain_df.iloc[77,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 324] = terrain_df.iloc[77,4]
    df2['desert_o'][df2['iso3num_o'] == 324] = terrain_df.iloc[77,12]
    df2['desert_d'][df2['iso3num_d'] == 324] = terrain_df.iloc[77,12]
    df2['tropical_o'][df2['iso3num_o'] == 324] = terrain_df.iloc[77,13]
    df2['tropical_d'][df2['iso3num_d'] == 324] = terrain_df.iloc[77,13]
    df2['island_o'][df2['iso3num_o'] == 324] = terrain_df.iloc[77,51]
    df2['island_d'][df2['iso3num_d'] == 324] = terrain_df.iloc[77,51]
    df2['landlocked_o'][df2['iso3num_o'] == 324] = terrain_df.iloc[77,52]
    df2['landlocked_d'][df2['iso3num_d'] == 324] = terrain_df.iloc[77,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 312] = terrain_df.iloc[78,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 312] = terrain_df.iloc[78,4]
    df2['desert_o'][df2['iso3num_o'] == 312] = terrain_df.iloc[78,12]
    df2['desert_d'][df2['iso3num_d'] == 312] = terrain_df.iloc[78,12]
    df2['tropical_o'][df2['iso3num_o'] == 312] = terrain_df.iloc[78,13]
    df2['tropical_d'][df2['iso3num_d'] == 312] = terrain_df.iloc[78,13]
    df2['island_o'][df2['iso3num_o'] == 312] = terrain_df.iloc[78,51]
    df2['island_d'][df2['iso3num_d'] == 312] = terrain_df.iloc[78,51]
    df2['landlocked_o'][df2['iso3num_o'] == 312] = terrain_df.iloc[78,52]
    df2['landlocked_d'][df2['iso3num_d'] == 312] = terrain_df.iloc[78,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 270] = terrain_df.iloc[79,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 270] = terrain_df.iloc[79,4]
    df2['desert_o'][df2['iso3num_o'] == 270] = terrain_df.iloc[79,12]
    df2['desert_d'][df2['iso3num_d'] == 270] = terrain_df.iloc[79,12]
    df2['tropical_o'][df2['iso3num_o'] == 270] = terrain_df.iloc[79,13]
    df2['tropical_d'][df2['iso3num_d'] == 270] = terrain_df.iloc[79,13]
    df2['island_o'][df2['iso3num_o'] == 270] = terrain_df.iloc[79,51]
    df2['island_d'][df2['iso3num_d'] == 270] = terrain_df.iloc[79,51]
    df2['landlocked_o'][df2['iso3num_o'] == 270] = terrain_df.iloc[79,52]
    df2['landlocked_d'][df2['iso3num_d'] == 270] = terrain_df.iloc[79,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 624] = terrain_df.iloc[80,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 624] = terrain_df.iloc[80,4]
    df2['desert_o'][df2['iso3num_o'] == 624] = terrain_df.iloc[80,12]
    df2['desert_d'][df2['iso3num_d'] == 624] = terrain_df.iloc[80,12]
    df2['tropical_o'][df2['iso3num_o'] == 624] = terrain_df.iloc[80,13]
    df2['tropical_d'][df2['iso3num_d'] == 624] = terrain_df.iloc[80,13]
    df2['island_o'][df2['iso3num_o'] == 624] = terrain_df.iloc[80,51]
    df2['island_d'][df2['iso3num_d'] == 624] = terrain_df.iloc[80,51]
    df2['landlocked_o'][df2['iso3num_o'] == 624] = terrain_df.iloc[80,52]
    df2['landlocked_d'][df2['iso3num_d'] == 624] = terrain_df.iloc[80,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 226] = terrain_df.iloc[81,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 226] = terrain_df.iloc[81,4]
    df2['desert_o'][df2['iso3num_o'] == 226] = terrain_df.iloc[81,12]
    df2['desert_d'][df2['iso3num_d'] == 226] = terrain_df.iloc[81,12]
    df2['tropical_o'][df2['iso3num_o'] == 226] = terrain_df.iloc[81,13]
    df2['tropical_d'][df2['iso3num_d'] == 226] = terrain_df.iloc[81,13]
    df2['island_o'][df2['iso3num_o'] == 226] = terrain_df.iloc[81,51]
    df2['island_d'][df2['iso3num_d'] == 226] = terrain_df.iloc[81,51]
    df2['landlocked_o'][df2['iso3num_o'] == 226] = terrain_df.iloc[81,52]
    df2['landlocked_d'][df2['iso3num_d'] == 226] = terrain_df.iloc[81,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 300] = terrain_df.iloc[82,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 300] = terrain_df.iloc[82,4]
    df2['desert_o'][df2['iso3num_o'] == 300] = terrain_df.iloc[82,12]
    df2['desert_d'][df2['iso3num_d'] == 300] = terrain_df.iloc[82,12]
    df2['tropical_o'][df2['iso3num_o'] == 300] = terrain_df.iloc[82,13]
    df2['tropical_d'][df2['iso3num_d'] == 300] = terrain_df.iloc[82,13]
    df2['island_o'][df2['iso3num_o'] == 300] = terrain_df.iloc[82,51]
    df2['island_d'][df2['iso3num_d'] == 300] = terrain_df.iloc[82,51]
    df2['landlocked_o'][df2['iso3num_o'] == 300] = terrain_df.iloc[82,52]
    df2['landlocked_d'][df2['iso3num_d'] == 300] = terrain_df.iloc[82,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 308] = terrain_df.iloc[83,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 308] = terrain_df.iloc[83,4]
    df2['desert_o'][df2['iso3num_o'] == 308] = terrain_df.iloc[83,12]
    df2['desert_d'][df2['iso3num_d'] == 308] = terrain_df.iloc[83,12]
    df2['tropical_o'][df2['iso3num_o'] == 308] = terrain_df.iloc[83,13]
    df2['tropical_d'][df2['iso3num_d'] == 308] = terrain_df.iloc[83,13]
    df2['island_o'][df2['iso3num_o'] == 308] = terrain_df.iloc[83,51]
    df2['island_d'][df2['iso3num_d'] == 308] = terrain_df.iloc[83,51]
    df2['landlocked_o'][df2['iso3num_o'] == 308] = terrain_df.iloc[83,52]
    df2['landlocked_d'][df2['iso3num_d'] == 308] = terrain_df.iloc[83,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 304] = terrain_df.iloc[84,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 304] = terrain_df.iloc[84,4]
    df2['desert_o'][df2['iso3num_o'] == 304] = terrain_df.iloc[84,12]
    df2['desert_d'][df2['iso3num_d'] == 304] = terrain_df.iloc[84,12]
    df2['tropical_o'][df2['iso3num_o'] == 304] = terrain_df.iloc[84,13]
    df2['tropical_d'][df2['iso3num_d'] == 304] = terrain_df.iloc[84,13]
    df2['island_o'][df2['iso3num_o'] == 304] = terrain_df.iloc[84,51]
    df2['island_d'][df2['iso3num_d'] == 304] = terrain_df.iloc[84,51]
    df2['landlocked_o'][df2['iso3num_o'] == 304] = terrain_df.iloc[84,52]
    df2['landlocked_d'][df2['iso3num_d'] == 304] = terrain_df.iloc[84,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 320] = terrain_df.iloc[85,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 320] = terrain_df.iloc[85,4]
    df2['desert_o'][df2['iso3num_o'] == 320] = terrain_df.iloc[85,12]
    df2['desert_d'][df2['iso3num_d'] == 320] = terrain_df.iloc[85,12]
    df2['tropical_o'][df2['iso3num_o'] == 320] = terrain_df.iloc[85,13]
    df2['tropical_d'][df2['iso3num_d'] == 320] = terrain_df.iloc[85,13]
    df2['island_o'][df2['iso3num_o'] == 320] = terrain_df.iloc[85,51]
    df2['island_d'][df2['iso3num_d'] == 320] = terrain_df.iloc[85,51]
    df2['landlocked_o'][df2['iso3num_o'] == 320] = terrain_df.iloc[85,52]
    df2['landlocked_d'][df2['iso3num_d'] == 320] = terrain_df.iloc[85,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 254] = terrain_df.iloc[86,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 254] = terrain_df.iloc[86,4]
    df2['desert_o'][df2['iso3num_o'] == 254] = terrain_df.iloc[86,12]
    df2['desert_d'][df2['iso3num_d'] == 254] = terrain_df.iloc[86,12]
    df2['tropical_o'][df2['iso3num_o'] == 254] = terrain_df.iloc[86,13]
    df2['tropical_d'][df2['iso3num_d'] == 254] = terrain_df.iloc[86,13]
    df2['island_o'][df2['iso3num_o'] == 254] = terrain_df.iloc[86,51]
    df2['island_d'][df2['iso3num_d'] == 254] = terrain_df.iloc[86,51]
    df2['landlocked_o'][df2['iso3num_o'] == 254] = terrain_df.iloc[86,52]
    df2['landlocked_d'][df2['iso3num_d'] == 254] = terrain_df.iloc[86,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 316] = terrain_df.iloc[87,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 316] = terrain_df.iloc[87,4]
    df2['desert_o'][df2['iso3num_o'] == 316] = terrain_df.iloc[87,12]
    df2['desert_d'][df2['iso3num_d'] == 316] = terrain_df.iloc[87,12]
    df2['tropical_o'][df2['iso3num_o'] == 316] = terrain_df.iloc[87,13]
    df2['tropical_d'][df2['iso3num_d'] == 316] = terrain_df.iloc[87,13]
    df2['island_o'][df2['iso3num_o'] == 316] = terrain_df.iloc[87,51]
    df2['island_d'][df2['iso3num_d'] == 316] = terrain_df.iloc[87,51]
    df2['landlocked_o'][df2['iso3num_o'] == 316] = terrain_df.iloc[87,52]
    df2['landlocked_d'][df2['iso3num_d'] == 316] = terrain_df.iloc[87,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 328] = terrain_df.iloc[88,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 328] = terrain_df.iloc[88,4]
    df2['desert_o'][df2['iso3num_o'] == 328] = terrain_df.iloc[88,12]
    df2['desert_d'][df2['iso3num_d'] == 328] = terrain_df.iloc[88,12]
    df2['tropical_o'][df2['iso3num_o'] == 328] = terrain_df.iloc[88,13]
    df2['tropical_d'][df2['iso3num_d'] == 328] = terrain_df.iloc[88,13]
    df2['island_o'][df2['iso3num_o'] == 328] = terrain_df.iloc[88,51]
    df2['island_d'][df2['iso3num_d'] == 328] = terrain_df.iloc[88,51]
    df2['landlocked_o'][df2['iso3num_o'] == 328] = terrain_df.iloc[88,52]
    df2['landlocked_d'][df2['iso3num_d'] == 328] = terrain_df.iloc[88,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 344] = terrain_df.iloc[89,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 344] = terrain_df.iloc[89,4]
    df2['desert_o'][df2['iso3num_o'] == 344] = terrain_df.iloc[89,12]
    df2['desert_d'][df2['iso3num_d'] == 344] = terrain_df.iloc[89,12]
    df2['tropical_o'][df2['iso3num_o'] == 344] = terrain_df.iloc[89,13]
    df2['tropical_d'][df2['iso3num_d'] == 344] = terrain_df.iloc[89,13]
    df2['island_o'][df2['iso3num_o'] == 344] = terrain_df.iloc[89,51]
    df2['island_d'][df2['iso3num_d'] == 344] = terrain_df.iloc[89,51]
    df2['landlocked_o'][df2['iso3num_o'] == 344] = terrain_df.iloc[89,52]
    df2['landlocked_d'][df2['iso3num_d'] == 344] = terrain_df.iloc[89,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 340] = terrain_df.iloc[90,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 340] = terrain_df.iloc[90,4]
    df2['desert_o'][df2['iso3num_o'] == 340] = terrain_df.iloc[90,12]
    df2['desert_d'][df2['iso3num_d'] == 340] = terrain_df.iloc[90,12]
    df2['tropical_o'][df2['iso3num_o'] == 340] = terrain_df.iloc[90,13]
    df2['tropical_d'][df2['iso3num_d'] == 340] = terrain_df.iloc[90,13]
    df2['island_o'][df2['iso3num_o'] == 340] = terrain_df.iloc[90,51]
    df2['island_d'][df2['iso3num_d'] == 340] = terrain_df.iloc[90,51]
    df2['landlocked_o'][df2['iso3num_o'] == 340] = terrain_df.iloc[90,52]
    df2['landlocked_d'][df2['iso3num_d'] == 340] = terrain_df.iloc[90,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 191] = terrain_df.iloc[91,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 191] = terrain_df.iloc[91,4]
    df2['desert_o'][df2['iso3num_o'] == 191] = terrain_df.iloc[91,12]
    df2['desert_d'][df2['iso3num_d'] == 191] = terrain_df.iloc[91,12]
    df2['tropical_o'][df2['iso3num_o'] == 191] = terrain_df.iloc[91,13]
    df2['tropical_d'][df2['iso3num_d'] == 191] = terrain_df.iloc[91,13]
    df2['island_o'][df2['iso3num_o'] == 191] = terrain_df.iloc[91,51]
    df2['island_d'][df2['iso3num_d'] == 191] = terrain_df.iloc[91,51]
    df2['landlocked_o'][df2['iso3num_o'] == 191] = terrain_df.iloc[91,52]
    df2['landlocked_d'][df2['iso3num_d'] == 191] = terrain_df.iloc[91,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 332] = terrain_df.iloc[92,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 332] = terrain_df.iloc[92,4]
    df2['desert_o'][df2['iso3num_o'] == 332] = terrain_df.iloc[92,12]
    df2['desert_d'][df2['iso3num_d'] == 332] = terrain_df.iloc[92,12]
    df2['tropical_o'][df2['iso3num_o'] == 332] = terrain_df.iloc[92,13]
    df2['tropical_d'][df2['iso3num_d'] == 332] = terrain_df.iloc[92,13]
    df2['island_o'][df2['iso3num_o'] == 332] = terrain_df.iloc[92,51]
    df2['island_d'][df2['iso3num_d'] == 332] = terrain_df.iloc[92,51]
    df2['landlocked_o'][df2['iso3num_o'] == 332] = terrain_df.iloc[92,52]
    df2['landlocked_d'][df2['iso3num_d'] == 332] = terrain_df.iloc[92,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 348] = terrain_df.iloc[93,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 348] = terrain_df.iloc[93,4]
    df2['desert_o'][df2['iso3num_o'] == 348] = terrain_df.iloc[93,12]
    df2['desert_d'][df2['iso3num_d'] == 348] = terrain_df.iloc[93,12]
    df2['tropical_o'][df2['iso3num_o'] == 348] = terrain_df.iloc[93,13]
    df2['tropical_d'][df2['iso3num_d'] == 348] = terrain_df.iloc[93,13]
    df2['island_o'][df2['iso3num_o'] == 348] = terrain_df.iloc[93,51]
    df2['island_d'][df2['iso3num_d'] == 348] = terrain_df.iloc[93,51]
    df2['landlocked_o'][df2['iso3num_o'] == 348] = terrain_df.iloc[93,52]
    df2['landlocked_d'][df2['iso3num_d'] == 348] = terrain_df.iloc[93,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 360] = terrain_df.iloc[94,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 360] = terrain_df.iloc[94,4]
    df2['desert_o'][df2['iso3num_o'] == 360] = terrain_df.iloc[94,12]
    df2['desert_d'][df2['iso3num_d'] == 360] = terrain_df.iloc[94,12]
    df2['tropical_o'][df2['iso3num_o'] == 360] = terrain_df.iloc[94,13]
    df2['tropical_d'][df2['iso3num_d'] == 360] = terrain_df.iloc[94,13]
    df2['island_o'][df2['iso3num_o'] == 360] = terrain_df.iloc[94,51]
    df2['island_d'][df2['iso3num_d'] == 360] = terrain_df.iloc[94,51]
    df2['landlocked_o'][df2['iso3num_o'] == 360] = terrain_df.iloc[94,52]
    df2['landlocked_d'][df2['iso3num_d'] == 360] = terrain_df.iloc[94,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 356] = terrain_df.iloc[95,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 356] = terrain_df.iloc[95,4]
    df2['desert_o'][df2['iso3num_o'] == 356] = terrain_df.iloc[95,12]
    df2['desert_d'][df2['iso3num_d'] == 356] = terrain_df.iloc[95,12]
    df2['tropical_o'][df2['iso3num_o'] == 356] = terrain_df.iloc[95,13]
    df2['tropical_d'][df2['iso3num_d'] == 356] = terrain_df.iloc[95,13]
    df2['island_o'][df2['iso3num_o'] == 356] = terrain_df.iloc[95,51]
    df2['island_d'][df2['iso3num_d'] == 356] = terrain_df.iloc[95,51]
    df2['landlocked_o'][df2['iso3num_o'] == 356] = terrain_df.iloc[95,52]
    df2['landlocked_d'][df2['iso3num_d'] == 356] = terrain_df.iloc[95,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 86] = terrain_df.iloc[96,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 86] = terrain_df.iloc[96,4]
    df2['desert_o'][df2['iso3num_o'] == 86] = terrain_df.iloc[96,12]
    df2['desert_d'][df2['iso3num_d'] == 86] = terrain_df.iloc[96,12]
    df2['tropical_o'][df2['iso3num_o'] == 86] = terrain_df.iloc[96,13]
    df2['tropical_d'][df2['iso3num_d'] == 86] = terrain_df.iloc[96,13]
    df2['island_o'][df2['iso3num_o'] == 86] = terrain_df.iloc[96,51]
    df2['island_d'][df2['iso3num_d'] == 86] = terrain_df.iloc[96,51]
    df2['landlocked_o'][df2['iso3num_o'] == 86] = terrain_df.iloc[96,52]
    df2['landlocked_d'][df2['iso3num_d'] == 86] = terrain_df.iloc[96,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 372] = terrain_df.iloc[97,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 372] = terrain_df.iloc[97,4]
    df2['desert_o'][df2['iso3num_o'] == 372] = terrain_df.iloc[97,12]
    df2['desert_d'][df2['iso3num_d'] == 372] = terrain_df.iloc[97,12]
    df2['tropical_o'][df2['iso3num_o'] == 372] = terrain_df.iloc[97,13]
    df2['tropical_d'][df2['iso3num_d'] == 372] = terrain_df.iloc[97,13]
    df2['island_o'][df2['iso3num_o'] == 372] = terrain_df.iloc[97,51]
    df2['island_d'][df2['iso3num_d'] == 372] = terrain_df.iloc[97,51]
    df2['landlocked_o'][df2['iso3num_o'] == 372] = terrain_df.iloc[97,52]
    df2['landlocked_d'][df2['iso3num_d'] == 372] = terrain_df.iloc[97,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 364] = terrain_df.iloc[98,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 364] = terrain_df.iloc[98,4]
    df2['desert_o'][df2['iso3num_o'] == 364] = terrain_df.iloc[98,12]
    df2['desert_d'][df2['iso3num_d'] == 364] = terrain_df.iloc[98,12]
    df2['tropical_o'][df2['iso3num_o'] == 364] = terrain_df.iloc[98,13]
    df2['tropical_d'][df2['iso3num_d'] == 364] = terrain_df.iloc[98,13]
    df2['island_o'][df2['iso3num_o'] == 364] = terrain_df.iloc[98,51]
    df2['island_d'][df2['iso3num_d'] == 364] = terrain_df.iloc[98,51]
    df2['landlocked_o'][df2['iso3num_o'] == 364] = terrain_df.iloc[98,52]
    df2['landlocked_d'][df2['iso3num_d'] == 364] = terrain_df.iloc[98,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 368] = terrain_df.iloc[99,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 368] = terrain_df.iloc[99,4]
    df2['desert_o'][df2['iso3num_o'] == 368] = terrain_df.iloc[99,12]
    df2['desert_d'][df2['iso3num_d'] == 368] = terrain_df.iloc[99,12]
    df2['tropical_o'][df2['iso3num_o'] == 368] = terrain_df.iloc[99,13]
    df2['tropical_d'][df2['iso3num_d'] == 368] = terrain_df.iloc[99,13]
    df2['island_o'][df2['iso3num_o'] == 368] = terrain_df.iloc[99,51]
    df2['island_d'][df2['iso3num_d'] == 368] = terrain_df.iloc[99,51]
    df2['landlocked_o'][df2['iso3num_o'] == 368] = terrain_df.iloc[99,52]
    df2['landlocked_d'][df2['iso3num_d'] == 368] = terrain_df.iloc[99,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 352] = terrain_df.iloc[100,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 352] = terrain_df.iloc[100,4]
    df2['desert_o'][df2['iso3num_o'] == 352] = terrain_df.iloc[100,12]
    df2['desert_d'][df2['iso3num_d'] == 352] = terrain_df.iloc[100,12]
    df2['tropical_o'][df2['iso3num_o'] == 352] = terrain_df.iloc[100,13]
    df2['tropical_d'][df2['iso3num_d'] == 352] = terrain_df.iloc[100,13]
    df2['island_o'][df2['iso3num_o'] == 352] = terrain_df.iloc[100,51]
    df2['island_d'][df2['iso3num_d'] == 352] = terrain_df.iloc[100,51]
    df2['landlocked_o'][df2['iso3num_o'] == 352] = terrain_df.iloc[100,52]
    df2['landlocked_d'][df2['iso3num_d'] == 352] = terrain_df.iloc[100,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 376] = terrain_df.iloc[101,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 376] = terrain_df.iloc[101,4]
    df2['desert_o'][df2['iso3num_o'] == 376] = terrain_df.iloc[101,12]
    df2['desert_d'][df2['iso3num_d'] == 376] = terrain_df.iloc[101,12]
    df2['tropical_o'][df2['iso3num_o'] == 376] = terrain_df.iloc[101,13]
    df2['tropical_d'][df2['iso3num_d'] == 376] = terrain_df.iloc[101,13]
    df2['island_o'][df2['iso3num_o'] == 376] = terrain_df.iloc[101,51]
    df2['island_d'][df2['iso3num_d'] == 376] = terrain_df.iloc[101,51]
    df2['landlocked_o'][df2['iso3num_o'] == 376] = terrain_df.iloc[101,52]
    df2['landlocked_d'][df2['iso3num_d'] == 376] = terrain_df.iloc[101,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 380] = terrain_df.iloc[102,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 380] = terrain_df.iloc[102,4]
    df2['desert_o'][df2['iso3num_o'] == 380] = terrain_df.iloc[102,12]
    df2['desert_d'][df2['iso3num_d'] == 380] = terrain_df.iloc[102,12]
    df2['tropical_o'][df2['iso3num_o'] == 380] = terrain_df.iloc[102,13]
    df2['tropical_d'][df2['iso3num_d'] == 380] = terrain_df.iloc[102,13]
    df2['island_o'][df2['iso3num_o'] == 380] = terrain_df.iloc[102,51]
    df2['island_d'][df2['iso3num_d'] == 380] = terrain_df.iloc[102,51]
    df2['landlocked_o'][df2['iso3num_o'] == 380] = terrain_df.iloc[102,52]
    df2['landlocked_d'][df2['iso3num_d'] == 380] = terrain_df.iloc[102,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 388] = terrain_df.iloc[103,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 388] = terrain_df.iloc[103,4]
    df2['desert_o'][df2['iso3num_o'] == 388] = terrain_df.iloc[103,12]
    df2['desert_d'][df2['iso3num_d'] == 388] = terrain_df.iloc[103,12]
    df2['tropical_o'][df2['iso3num_o'] == 388] = terrain_df.iloc[103,13]
    df2['tropical_d'][df2['iso3num_d'] == 388] = terrain_df.iloc[103,13]
    df2['island_o'][df2['iso3num_o'] == 388] = terrain_df.iloc[103,51]
    df2['island_d'][df2['iso3num_d'] == 388] = terrain_df.iloc[103,51]
    df2['landlocked_o'][df2['iso3num_o'] == 388] = terrain_df.iloc[103,52]
    df2['landlocked_d'][df2['iso3num_d'] == 388] = terrain_df.iloc[103,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 400] = terrain_df.iloc[104,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 400] = terrain_df.iloc[104,4]
    df2['desert_o'][df2['iso3num_o'] == 400] = terrain_df.iloc[104,12]
    df2['desert_d'][df2['iso3num_d'] == 400] = terrain_df.iloc[104,12]
    df2['tropical_o'][df2['iso3num_o'] == 400] = terrain_df.iloc[104,13]
    df2['tropical_d'][df2['iso3num_d'] == 400] = terrain_df.iloc[104,13]
    df2['island_o'][df2['iso3num_o'] == 400] = terrain_df.iloc[104,51]
    df2['island_d'][df2['iso3num_d'] == 400] = terrain_df.iloc[104,51]
    df2['landlocked_o'][df2['iso3num_o'] == 400] = terrain_df.iloc[104,52]
    df2['landlocked_d'][df2['iso3num_d'] == 400] = terrain_df.iloc[104,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 392] = terrain_df.iloc[105,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 392] = terrain_df.iloc[105,4]
    df2['desert_o'][df2['iso3num_o'] == 392] = terrain_df.iloc[105,12]
    df2['desert_d'][df2['iso3num_d'] == 392] = terrain_df.iloc[105,12]
    df2['tropical_o'][df2['iso3num_o'] == 392] = terrain_df.iloc[105,13]
    df2['tropical_d'][df2['iso3num_d'] == 392] = terrain_df.iloc[105,13]
    df2['island_o'][df2['iso3num_o'] == 392] = terrain_df.iloc[105,51]
    df2['island_d'][df2['iso3num_d'] == 392] = terrain_df.iloc[105,51]
    df2['landlocked_o'][df2['iso3num_o'] == 392] = terrain_df.iloc[105,52]
    df2['landlocked_d'][df2['iso3num_d'] == 392] = terrain_df.iloc[105,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 398] = terrain_df.iloc[106,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 398] = terrain_df.iloc[106,4]
    df2['desert_o'][df2['iso3num_o'] == 398] = terrain_df.iloc[106,12]
    df2['desert_d'][df2['iso3num_d'] == 398] = terrain_df.iloc[106,12]
    df2['tropical_o'][df2['iso3num_o'] == 398] = terrain_df.iloc[106,13]
    df2['tropical_d'][df2['iso3num_d'] == 398] = terrain_df.iloc[106,13]
    df2['island_o'][df2['iso3num_o'] == 398] = terrain_df.iloc[106,51]
    df2['island_d'][df2['iso3num_d'] == 398] = terrain_df.iloc[106,51]
    df2['landlocked_o'][df2['iso3num_o'] == 398] = terrain_df.iloc[106,52]
    df2['landlocked_d'][df2['iso3num_d'] == 398] = terrain_df.iloc[106,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 404] = terrain_df.iloc[107,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 404] = terrain_df.iloc[107,4]
    df2['desert_o'][df2['iso3num_o'] == 404] = terrain_df.iloc[107,12]
    df2['desert_d'][df2['iso3num_d'] == 404] = terrain_df.iloc[107,12]
    df2['tropical_o'][df2['iso3num_o'] == 404] = terrain_df.iloc[107,13]
    df2['tropical_d'][df2['iso3num_d'] == 404] = terrain_df.iloc[107,13]
    df2['island_o'][df2['iso3num_o'] == 404] = terrain_df.iloc[107,51]
    df2['island_d'][df2['iso3num_d'] == 404] = terrain_df.iloc[107,51]
    df2['landlocked_o'][df2['iso3num_o'] == 404] = terrain_df.iloc[107,52]
    df2['landlocked_d'][df2['iso3num_d'] == 404] = terrain_df.iloc[107,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 417] = terrain_df.iloc[108,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 417] = terrain_df.iloc[108,4]
    df2['desert_o'][df2['iso3num_o'] == 417] = terrain_df.iloc[108,12]
    df2['desert_d'][df2['iso3num_d'] == 417] = terrain_df.iloc[108,12]
    df2['tropical_o'][df2['iso3num_o'] == 417] = terrain_df.iloc[108,13]
    df2['tropical_d'][df2['iso3num_d'] == 417] = terrain_df.iloc[108,13]
    df2['island_o'][df2['iso3num_o'] == 417] = terrain_df.iloc[108,51]
    df2['island_d'][df2['iso3num_d'] == 417] = terrain_df.iloc[108,51]
    df2['landlocked_o'][df2['iso3num_o'] == 417] = terrain_df.iloc[108,52]
    df2['landlocked_d'][df2['iso3num_d'] == 417] = terrain_df.iloc[108,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 116] = terrain_df.iloc[109,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 116] = terrain_df.iloc[109,4]
    df2['desert_o'][df2['iso3num_o'] == 116] = terrain_df.iloc[109,12]
    df2['desert_d'][df2['iso3num_d'] == 116] = terrain_df.iloc[109,12]
    df2['tropical_o'][df2['iso3num_o'] == 116] = terrain_df.iloc[109,13]
    df2['tropical_d'][df2['iso3num_d'] == 116] = terrain_df.iloc[109,13]
    df2['island_o'][df2['iso3num_o'] == 116] = terrain_df.iloc[109,51]
    df2['island_d'][df2['iso3num_d'] == 116] = terrain_df.iloc[109,51]
    df2['landlocked_o'][df2['iso3num_o'] == 116] = terrain_df.iloc[109,52]
    df2['landlocked_d'][df2['iso3num_d'] == 116] = terrain_df.iloc[109,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 296] = terrain_df.iloc[110,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 296] = terrain_df.iloc[110,4]
    df2['desert_o'][df2['iso3num_o'] == 296] = terrain_df.iloc[110,12]
    df2['desert_d'][df2['iso3num_d'] == 296] = terrain_df.iloc[110,12]
    df2['tropical_o'][df2['iso3num_o'] == 296] = terrain_df.iloc[110,13]
    df2['tropical_d'][df2['iso3num_d'] == 296] = terrain_df.iloc[110,13]
    df2['island_o'][df2['iso3num_o'] == 296] = terrain_df.iloc[110,51]
    df2['island_d'][df2['iso3num_d'] == 296] = terrain_df.iloc[110,51]
    df2['landlocked_o'][df2['iso3num_o'] == 296] = terrain_df.iloc[110,52]
    df2['landlocked_d'][df2['iso3num_d'] == 296] = terrain_df.iloc[110,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 659] = terrain_df.iloc[111,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 659] = terrain_df.iloc[111,4]
    df2['desert_o'][df2['iso3num_o'] == 659] = terrain_df.iloc[111,12]
    df2['desert_d'][df2['iso3num_d'] == 659] = terrain_df.iloc[111,12]
    df2['tropical_o'][df2['iso3num_o'] == 659] = terrain_df.iloc[111,13]
    df2['tropical_d'][df2['iso3num_d'] == 659] = terrain_df.iloc[111,13]
    df2['island_o'][df2['iso3num_o'] == 659] = terrain_df.iloc[111,51]
    df2['island_d'][df2['iso3num_d'] == 659] = terrain_df.iloc[111,51]
    df2['landlocked_o'][df2['iso3num_o'] == 659] = terrain_df.iloc[111,52]
    df2['landlocked_d'][df2['iso3num_d'] == 659] = terrain_df.iloc[111,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 410] = terrain_df.iloc[112,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 410] = terrain_df.iloc[112,4]
    df2['desert_o'][df2['iso3num_o'] == 410] = terrain_df.iloc[112,12]
    df2['desert_d'][df2['iso3num_d'] == 410] = terrain_df.iloc[112,12]
    df2['tropical_o'][df2['iso3num_o'] == 410] = terrain_df.iloc[112,13]
    df2['tropical_d'][df2['iso3num_d'] == 410] = terrain_df.iloc[112,13]
    df2['island_o'][df2['iso3num_o'] == 410] = terrain_df.iloc[112,51]
    df2['island_d'][df2['iso3num_d'] == 410] = terrain_df.iloc[112,51]
    df2['landlocked_o'][df2['iso3num_o'] == 410] = terrain_df.iloc[112,52]
    df2['landlocked_d'][df2['iso3num_d'] == 410] = terrain_df.iloc[112,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 414] = terrain_df.iloc[113,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 414] = terrain_df.iloc[113,4]
    df2['desert_o'][df2['iso3num_o'] == 414] = terrain_df.iloc[113,12]
    df2['desert_d'][df2['iso3num_d'] == 414] = terrain_df.iloc[113,12]
    df2['tropical_o'][df2['iso3num_o'] == 414] = terrain_df.iloc[113,13]
    df2['tropical_d'][df2['iso3num_d'] == 414] = terrain_df.iloc[113,13]
    df2['island_o'][df2['iso3num_o'] == 414] = terrain_df.iloc[113,51]
    df2['island_d'][df2['iso3num_d'] == 414] = terrain_df.iloc[113,51]
    df2['landlocked_o'][df2['iso3num_o'] == 414] = terrain_df.iloc[113,52]
    df2['landlocked_d'][df2['iso3num_d'] == 414] = terrain_df.iloc[113,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 418] = terrain_df.iloc[114,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 418] = terrain_df.iloc[114,4]
    df2['desert_o'][df2['iso3num_o'] == 418] = terrain_df.iloc[114,12]
    df2['desert_d'][df2['iso3num_d'] == 418] = terrain_df.iloc[114,12]
    df2['tropical_o'][df2['iso3num_o'] == 418] = terrain_df.iloc[114,13]
    df2['tropical_d'][df2['iso3num_d'] == 418] = terrain_df.iloc[114,13]
    df2['island_o'][df2['iso3num_o'] == 418] = terrain_df.iloc[114,51]
    df2['island_d'][df2['iso3num_d'] == 418] = terrain_df.iloc[114,51]
    df2['landlocked_o'][df2['iso3num_o'] == 418] = terrain_df.iloc[114,52]
    df2['landlocked_d'][df2['iso3num_d'] == 418] = terrain_df.iloc[114,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 422] = terrain_df.iloc[115,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 422] = terrain_df.iloc[115,4]
    df2['desert_o'][df2['iso3num_o'] == 422] = terrain_df.iloc[115,12]
    df2['desert_d'][df2['iso3num_d'] == 422] = terrain_df.iloc[115,12]
    df2['tropical_o'][df2['iso3num_o'] == 422] = terrain_df.iloc[115,13]
    df2['tropical_d'][df2['iso3num_d'] == 422] = terrain_df.iloc[115,13]
    df2['island_o'][df2['iso3num_o'] == 422] = terrain_df.iloc[115,51]
    df2['island_d'][df2['iso3num_d'] == 422] = terrain_df.iloc[115,51]
    df2['landlocked_o'][df2['iso3num_o'] == 422] = terrain_df.iloc[115,52]
    df2['landlocked_d'][df2['iso3num_d'] == 422] = terrain_df.iloc[115,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 430] = terrain_df.iloc[116,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 430] = terrain_df.iloc[116,4]
    df2['desert_o'][df2['iso3num_o'] == 430] = terrain_df.iloc[116,12]
    df2['desert_d'][df2['iso3num_d'] == 430] = terrain_df.iloc[116,12]
    df2['tropical_o'][df2['iso3num_o'] == 430] = terrain_df.iloc[116,13]
    df2['tropical_d'][df2['iso3num_d'] == 430] = terrain_df.iloc[116,13]
    df2['island_o'][df2['iso3num_o'] == 430] = terrain_df.iloc[116,51]
    df2['island_d'][df2['iso3num_d'] == 430] = terrain_df.iloc[116,51]
    df2['landlocked_o'][df2['iso3num_o'] == 430] = terrain_df.iloc[116,52]
    df2['landlocked_d'][df2['iso3num_d'] == 430] = terrain_df.iloc[116,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 434] = terrain_df.iloc[117,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 434] = terrain_df.iloc[117,4]
    df2['desert_o'][df2['iso3num_o'] == 434] = terrain_df.iloc[117,12]
    df2['desert_d'][df2['iso3num_d'] == 434] = terrain_df.iloc[117,12]
    df2['tropical_o'][df2['iso3num_o'] == 434] = terrain_df.iloc[117,13]
    df2['tropical_d'][df2['iso3num_d'] == 434] = terrain_df.iloc[117,13]
    df2['island_o'][df2['iso3num_o'] == 434] = terrain_df.iloc[117,51]
    df2['island_d'][df2['iso3num_d'] == 434] = terrain_df.iloc[117,51]
    df2['landlocked_o'][df2['iso3num_o'] == 434] = terrain_df.iloc[117,52]
    df2['landlocked_d'][df2['iso3num_d'] == 434] = terrain_df.iloc[117,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 662] = terrain_df.iloc[118,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 662] = terrain_df.iloc[118,4]
    df2['desert_o'][df2['iso3num_o'] == 662] = terrain_df.iloc[118,12]
    df2['desert_d'][df2['iso3num_d'] == 662] = terrain_df.iloc[118,12]
    df2['tropical_o'][df2['iso3num_o'] == 662] = terrain_df.iloc[118,13]
    df2['tropical_d'][df2['iso3num_d'] == 662] = terrain_df.iloc[118,13]
    df2['island_o'][df2['iso3num_o'] == 662] = terrain_df.iloc[118,51]
    df2['island_d'][df2['iso3num_d'] == 662] = terrain_df.iloc[118,51]
    df2['landlocked_o'][df2['iso3num_o'] == 662] = terrain_df.iloc[118,52]
    df2['landlocked_d'][df2['iso3num_d'] == 662] = terrain_df.iloc[118,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 438] = terrain_df.iloc[119,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 438] = terrain_df.iloc[119,4]
    df2['desert_o'][df2['iso3num_o'] == 438] = terrain_df.iloc[119,12]
    df2['desert_d'][df2['iso3num_d'] == 438] = terrain_df.iloc[119,12]
    df2['tropical_o'][df2['iso3num_o'] == 438] = terrain_df.iloc[119,13]
    df2['tropical_d'][df2['iso3num_d'] == 438] = terrain_df.iloc[119,13]
    df2['island_o'][df2['iso3num_o'] == 438] = terrain_df.iloc[119,51]
    df2['island_d'][df2['iso3num_d'] == 438] = terrain_df.iloc[119,51]
    df2['landlocked_o'][df2['iso3num_o'] == 438] = terrain_df.iloc[119,52]
    df2['landlocked_d'][df2['iso3num_d'] == 438] = terrain_df.iloc[119,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 144] = terrain_df.iloc[120,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 144] = terrain_df.iloc[120,4]
    df2['desert_o'][df2['iso3num_o'] == 144] = terrain_df.iloc[120,12]
    df2['desert_d'][df2['iso3num_d'] == 144] = terrain_df.iloc[120,12]
    df2['tropical_o'][df2['iso3num_o'] == 144] = terrain_df.iloc[120,13]
    df2['tropical_d'][df2['iso3num_d'] == 144] = terrain_df.iloc[120,13]
    df2['island_o'][df2['iso3num_o'] == 144] = terrain_df.iloc[120,51]
    df2['island_d'][df2['iso3num_d'] == 144] = terrain_df.iloc[120,51]
    df2['landlocked_o'][df2['iso3num_o'] == 144] = terrain_df.iloc[120,52]
    df2['landlocked_d'][df2['iso3num_d'] == 144] = terrain_df.iloc[120,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 426] = terrain_df.iloc[121,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 426] = terrain_df.iloc[121,4]
    df2['desert_o'][df2['iso3num_o'] == 426] = terrain_df.iloc[121,12]
    df2['desert_d'][df2['iso3num_d'] == 426] = terrain_df.iloc[121,12]
    df2['tropical_o'][df2['iso3num_o'] == 426] = terrain_df.iloc[121,13]
    df2['tropical_d'][df2['iso3num_d'] == 426] = terrain_df.iloc[121,13]
    df2['island_o'][df2['iso3num_o'] == 426] = terrain_df.iloc[121,51]
    df2['island_d'][df2['iso3num_d'] == 426] = terrain_df.iloc[121,51]
    df2['landlocked_o'][df2['iso3num_o'] == 426] = terrain_df.iloc[121,52]
    df2['landlocked_d'][df2['iso3num_d'] == 426] = terrain_df.iloc[121,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 440] = terrain_df.iloc[122,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 440] = terrain_df.iloc[122,4]
    df2['desert_o'][df2['iso3num_o'] == 440] = terrain_df.iloc[122,12]
    df2['desert_d'][df2['iso3num_d'] == 440] = terrain_df.iloc[122,12]
    df2['tropical_o'][df2['iso3num_o'] == 440] = terrain_df.iloc[122,13]
    df2['tropical_d'][df2['iso3num_d'] == 440] = terrain_df.iloc[122,13]
    df2['island_o'][df2['iso3num_o'] == 440] = terrain_df.iloc[122,51]
    df2['island_d'][df2['iso3num_d'] == 440] = terrain_df.iloc[122,51]
    df2['landlocked_o'][df2['iso3num_o'] == 440] = terrain_df.iloc[122,52]
    df2['landlocked_d'][df2['iso3num_d'] == 440] = terrain_df.iloc[122,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 442] = terrain_df.iloc[123,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 442] = terrain_df.iloc[123,4]
    df2['desert_o'][df2['iso3num_o'] == 442] = terrain_df.iloc[123,12]
    df2['desert_d'][df2['iso3num_d'] == 442] = terrain_df.iloc[123,12]
    df2['tropical_o'][df2['iso3num_o'] == 442] = terrain_df.iloc[123,13]
    df2['tropical_d'][df2['iso3num_d'] == 442] = terrain_df.iloc[123,13]
    df2['island_o'][df2['iso3num_o'] == 442] = terrain_df.iloc[123,51]
    df2['island_d'][df2['iso3num_d'] == 442] = terrain_df.iloc[123,51]
    df2['landlocked_o'][df2['iso3num_o'] == 442] = terrain_df.iloc[123,52]
    df2['landlocked_d'][df2['iso3num_d'] == 442] = terrain_df.iloc[123,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 428] = terrain_df.iloc[124,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 428] = terrain_df.iloc[124,4]
    df2['desert_o'][df2['iso3num_o'] == 428] = terrain_df.iloc[124,12]
    df2['desert_d'][df2['iso3num_d'] == 428] = terrain_df.iloc[124,12]
    df2['tropical_o'][df2['iso3num_o'] == 428] = terrain_df.iloc[124,13]
    df2['tropical_d'][df2['iso3num_d'] == 428] = terrain_df.iloc[124,13]
    df2['island_o'][df2['iso3num_o'] == 428] = terrain_df.iloc[124,51]
    df2['island_d'][df2['iso3num_d'] == 428] = terrain_df.iloc[124,51]
    df2['landlocked_o'][df2['iso3num_o'] == 428] = terrain_df.iloc[124,52]
    df2['landlocked_d'][df2['iso3num_d'] == 428] = terrain_df.iloc[124,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 446] = terrain_df.iloc[125,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 446] = terrain_df.iloc[125,4]
    df2['desert_o'][df2['iso3num_o'] == 446] = terrain_df.iloc[125,12]
    df2['desert_d'][df2['iso3num_d'] == 446] = terrain_df.iloc[125,12]
    df2['tropical_o'][df2['iso3num_o'] == 446] = terrain_df.iloc[125,13]
    df2['tropical_d'][df2['iso3num_d'] == 446] = terrain_df.iloc[125,13]
    df2['island_o'][df2['iso3num_o'] == 446] = terrain_df.iloc[125,51]
    df2['island_d'][df2['iso3num_d'] == 446] = terrain_df.iloc[125,51]
    df2['landlocked_o'][df2['iso3num_o'] == 446] = terrain_df.iloc[125,52]
    df2['landlocked_d'][df2['iso3num_d'] == 446] = terrain_df.iloc[125,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 504] = terrain_df.iloc[126,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 504] = terrain_df.iloc[126,4]
    df2['desert_o'][df2['iso3num_o'] == 504] = terrain_df.iloc[126,12]
    df2['desert_d'][df2['iso3num_d'] == 504] = terrain_df.iloc[126,12]
    df2['tropical_o'][df2['iso3num_o'] == 504] = terrain_df.iloc[126,13]
    df2['tropical_d'][df2['iso3num_d'] == 504] = terrain_df.iloc[126,13]
    df2['island_o'][df2['iso3num_o'] == 504] = terrain_df.iloc[126,51]
    df2['island_d'][df2['iso3num_d'] == 504] = terrain_df.iloc[126,51]
    df2['landlocked_o'][df2['iso3num_o'] == 504] = terrain_df.iloc[126,52]
    df2['landlocked_d'][df2['iso3num_d'] == 504] = terrain_df.iloc[126,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 492] = terrain_df.iloc[127,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 492] = terrain_df.iloc[127,4]
    df2['desert_o'][df2['iso3num_o'] == 492] = terrain_df.iloc[127,12]
    df2['desert_d'][df2['iso3num_d'] == 492] = terrain_df.iloc[127,12]
    df2['tropical_o'][df2['iso3num_o'] == 492] = terrain_df.iloc[127,13]
    df2['tropical_d'][df2['iso3num_d'] == 492] = terrain_df.iloc[127,13]
    df2['island_o'][df2['iso3num_o'] == 492] = terrain_df.iloc[127,51]
    df2['island_d'][df2['iso3num_d'] == 492] = terrain_df.iloc[127,51]
    df2['landlocked_o'][df2['iso3num_o'] == 492] = terrain_df.iloc[127,52]
    df2['landlocked_d'][df2['iso3num_d'] == 492] = terrain_df.iloc[127,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 498] = terrain_df.iloc[128,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 498] = terrain_df.iloc[128,4]
    df2['desert_o'][df2['iso3num_o'] == 498] = terrain_df.iloc[128,12]
    df2['desert_d'][df2['iso3num_d'] == 498] = terrain_df.iloc[128,12]
    df2['tropical_o'][df2['iso3num_o'] == 498] = terrain_df.iloc[128,13]
    df2['tropical_d'][df2['iso3num_d'] == 498] = terrain_df.iloc[128,13]
    df2['island_o'][df2['iso3num_o'] == 498] = terrain_df.iloc[128,51]
    df2['island_d'][df2['iso3num_d'] == 498] = terrain_df.iloc[128,51]
    df2['landlocked_o'][df2['iso3num_o'] == 498] = terrain_df.iloc[128,52]
    df2['landlocked_d'][df2['iso3num_d'] == 498] = terrain_df.iloc[128,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 450] = terrain_df.iloc[129,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 450] = terrain_df.iloc[129,4]
    df2['desert_o'][df2['iso3num_o'] == 450] = terrain_df.iloc[129,12]
    df2['desert_d'][df2['iso3num_d'] == 450] = terrain_df.iloc[129,12]
    df2['tropical_o'][df2['iso3num_o'] == 450] = terrain_df.iloc[129,13]
    df2['tropical_d'][df2['iso3num_d'] == 450] = terrain_df.iloc[129,13]
    df2['island_o'][df2['iso3num_o'] == 450] = terrain_df.iloc[129,51]
    df2['island_d'][df2['iso3num_d'] == 450] = terrain_df.iloc[129,51]
    df2['landlocked_o'][df2['iso3num_o'] == 450] = terrain_df.iloc[129,52]
    df2['landlocked_d'][df2['iso3num_d'] == 450] = terrain_df.iloc[129,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 462] = terrain_df.iloc[130,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 462] = terrain_df.iloc[130,4]
    df2['desert_o'][df2['iso3num_o'] == 462] = terrain_df.iloc[130,12]
    df2['desert_d'][df2['iso3num_d'] == 462] = terrain_df.iloc[130,12]
    df2['tropical_o'][df2['iso3num_o'] == 462] = terrain_df.iloc[130,13]
    df2['tropical_d'][df2['iso3num_d'] == 462] = terrain_df.iloc[130,13]
    df2['island_o'][df2['iso3num_o'] == 462] = terrain_df.iloc[130,51]
    df2['island_d'][df2['iso3num_d'] == 462] = terrain_df.iloc[130,51]
    df2['landlocked_o'][df2['iso3num_o'] == 462] = terrain_df.iloc[130,52]
    df2['landlocked_d'][df2['iso3num_d'] == 462] = terrain_df.iloc[130,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 484] = terrain_df.iloc[131,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 484] = terrain_df.iloc[131,4]
    df2['desert_o'][df2['iso3num_o'] == 484] = terrain_df.iloc[131,12]
    df2['desert_d'][df2['iso3num_d'] == 484] = terrain_df.iloc[131,12]
    df2['tropical_o'][df2['iso3num_o'] == 484] = terrain_df.iloc[131,13]
    df2['tropical_d'][df2['iso3num_d'] == 484] = terrain_df.iloc[131,13]
    df2['island_o'][df2['iso3num_o'] == 484] = terrain_df.iloc[131,51]
    df2['island_d'][df2['iso3num_d'] == 484] = terrain_df.iloc[131,51]
    df2['landlocked_o'][df2['iso3num_o'] == 484] = terrain_df.iloc[131,52]
    df2['landlocked_d'][df2['iso3num_d'] == 484] = terrain_df.iloc[131,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 584] = terrain_df.iloc[132,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 584] = terrain_df.iloc[132,4]
    df2['desert_o'][df2['iso3num_o'] == 584] = terrain_df.iloc[132,12]
    df2['desert_d'][df2['iso3num_d'] == 584] = terrain_df.iloc[132,12]
    df2['tropical_o'][df2['iso3num_o'] == 584] = terrain_df.iloc[132,13]
    df2['tropical_d'][df2['iso3num_d'] == 584] = terrain_df.iloc[132,13]
    df2['island_o'][df2['iso3num_o'] == 584] = terrain_df.iloc[132,51]
    df2['island_d'][df2['iso3num_d'] == 584] = terrain_df.iloc[132,51]
    df2['landlocked_o'][df2['iso3num_o'] == 584] = terrain_df.iloc[132,52]
    df2['landlocked_d'][df2['iso3num_d'] == 584] = terrain_df.iloc[132,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 807] = terrain_df.iloc[133,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 807] = terrain_df.iloc[133,4]
    df2['desert_o'][df2['iso3num_o'] == 807] = terrain_df.iloc[133,12]
    df2['desert_d'][df2['iso3num_d'] == 807] = terrain_df.iloc[133,12]
    df2['tropical_o'][df2['iso3num_o'] == 807] = terrain_df.iloc[133,13]
    df2['tropical_d'][df2['iso3num_d'] == 807] = terrain_df.iloc[133,13]
    df2['island_o'][df2['iso3num_o'] == 807] = terrain_df.iloc[133,51]
    df2['island_d'][df2['iso3num_d'] == 807] = terrain_df.iloc[133,51]
    df2['landlocked_o'][df2['iso3num_o'] == 807] = terrain_df.iloc[133,52]
    df2['landlocked_d'][df2['iso3num_d'] == 807] = terrain_df.iloc[133,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 466] = terrain_df.iloc[134,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 466] = terrain_df.iloc[134,4]
    df2['desert_o'][df2['iso3num_o'] == 466] = terrain_df.iloc[134,12]
    df2['desert_d'][df2['iso3num_d'] == 466] = terrain_df.iloc[134,12]
    df2['tropical_o'][df2['iso3num_o'] == 466] = terrain_df.iloc[134,13]
    df2['tropical_d'][df2['iso3num_d'] == 466] = terrain_df.iloc[134,13]
    df2['island_o'][df2['iso3num_o'] == 466] = terrain_df.iloc[134,51]
    df2['island_d'][df2['iso3num_d'] == 466] = terrain_df.iloc[134,51]
    df2['landlocked_o'][df2['iso3num_o'] == 466] = terrain_df.iloc[134,52]
    df2['landlocked_d'][df2['iso3num_d'] == 466] = terrain_df.iloc[134,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 470] = terrain_df.iloc[135,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 470] = terrain_df.iloc[135,4]
    df2['desert_o'][df2['iso3num_o'] == 470] = terrain_df.iloc[135,12]
    df2['desert_d'][df2['iso3num_d'] == 470] = terrain_df.iloc[135,12]
    df2['tropical_o'][df2['iso3num_o'] == 470] = terrain_df.iloc[135,13]
    df2['tropical_d'][df2['iso3num_d'] == 470] = terrain_df.iloc[135,13]
    df2['island_o'][df2['iso3num_o'] == 470] = terrain_df.iloc[135,51]
    df2['island_d'][df2['iso3num_d'] == 470] = terrain_df.iloc[135,51]
    df2['landlocked_o'][df2['iso3num_o'] == 470] = terrain_df.iloc[135,52]
    df2['landlocked_d'][df2['iso3num_d'] == 470] = terrain_df.iloc[135,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 104] = terrain_df.iloc[136,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 104] = terrain_df.iloc[136,4]
    df2['desert_o'][df2['iso3num_o'] == 104] = terrain_df.iloc[136,12]
    df2['desert_d'][df2['iso3num_d'] == 104] = terrain_df.iloc[136,12]
    df2['tropical_o'][df2['iso3num_o'] == 104] = terrain_df.iloc[136,13]
    df2['tropical_d'][df2['iso3num_d'] == 104] = terrain_df.iloc[136,13]
    df2['island_o'][df2['iso3num_o'] == 104] = terrain_df.iloc[136,51]
    df2['island_d'][df2['iso3num_d'] == 104] = terrain_df.iloc[136,51]
    df2['landlocked_o'][df2['iso3num_o'] == 104] = terrain_df.iloc[136,52]
    df2['landlocked_d'][df2['iso3num_d'] == 104] = terrain_df.iloc[136,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 496] = terrain_df.iloc[137,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 496] = terrain_df.iloc[137,4]
    df2['desert_o'][df2['iso3num_o'] == 496] = terrain_df.iloc[137,12]
    df2['desert_d'][df2['iso3num_d'] == 496] = terrain_df.iloc[137,12]
    df2['tropical_o'][df2['iso3num_o'] == 496] = terrain_df.iloc[137,13]
    df2['tropical_d'][df2['iso3num_d'] == 496] = terrain_df.iloc[137,13]
    df2['island_o'][df2['iso3num_o'] == 496] = terrain_df.iloc[137,51]
    df2['island_d'][df2['iso3num_d'] == 496] = terrain_df.iloc[137,51]
    df2['landlocked_o'][df2['iso3num_o'] == 496] = terrain_df.iloc[137,52]
    df2['landlocked_d'][df2['iso3num_d'] == 496] = terrain_df.iloc[137,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 580] = terrain_df.iloc[138,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 580] = terrain_df.iloc[138,4]
    df2['desert_o'][df2['iso3num_o'] == 580] = terrain_df.iloc[138,12]
    df2['desert_d'][df2['iso3num_d'] == 580] = terrain_df.iloc[138,12]
    df2['tropical_o'][df2['iso3num_o'] == 580] = terrain_df.iloc[138,13]
    df2['tropical_d'][df2['iso3num_d'] == 580] = terrain_df.iloc[138,13]
    df2['island_o'][df2['iso3num_o'] == 580] = terrain_df.iloc[138,51]
    df2['island_d'][df2['iso3num_d'] == 580] = terrain_df.iloc[138,51]
    df2['landlocked_o'][df2['iso3num_o'] == 580] = terrain_df.iloc[138,52]
    df2['landlocked_d'][df2['iso3num_d'] == 580] = terrain_df.iloc[138,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 508] = terrain_df.iloc[139,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 508] = terrain_df.iloc[139,4]
    df2['desert_o'][df2['iso3num_o'] == 508] = terrain_df.iloc[139,12]
    df2['desert_d'][df2['iso3num_d'] == 508] = terrain_df.iloc[139,12]
    df2['tropical_o'][df2['iso3num_o'] == 508] = terrain_df.iloc[139,13]
    df2['tropical_d'][df2['iso3num_d'] == 508] = terrain_df.iloc[139,13]
    df2['island_o'][df2['iso3num_o'] == 508] = terrain_df.iloc[139,51]
    df2['island_d'][df2['iso3num_d'] == 508] = terrain_df.iloc[139,51]
    df2['landlocked_o'][df2['iso3num_o'] == 508] = terrain_df.iloc[139,52]
    df2['landlocked_d'][df2['iso3num_d'] == 508] = terrain_df.iloc[139,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 478] = terrain_df.iloc[140,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 478] = terrain_df.iloc[140,4]
    df2['desert_o'][df2['iso3num_o'] == 478] = terrain_df.iloc[140,12]
    df2['desert_d'][df2['iso3num_d'] == 478] = terrain_df.iloc[140,12]
    df2['tropical_o'][df2['iso3num_o'] == 478] = terrain_df.iloc[140,13]
    df2['tropical_d'][df2['iso3num_d'] == 478] = terrain_df.iloc[140,13]
    df2['island_o'][df2['iso3num_o'] == 478] = terrain_df.iloc[140,51]
    df2['island_d'][df2['iso3num_d'] == 478] = terrain_df.iloc[140,51]
    df2['landlocked_o'][df2['iso3num_o'] == 478] = terrain_df.iloc[140,52]
    df2['landlocked_d'][df2['iso3num_d'] == 478] = terrain_df.iloc[140,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 500] = terrain_df.iloc[141,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 500] = terrain_df.iloc[141,4]
    df2['desert_o'][df2['iso3num_o'] == 500] = terrain_df.iloc[141,12]
    df2['desert_d'][df2['iso3num_d'] == 500] = terrain_df.iloc[141,12]
    df2['tropical_o'][df2['iso3num_o'] == 500] = terrain_df.iloc[141,13]
    df2['tropical_d'][df2['iso3num_d'] == 500] = terrain_df.iloc[141,13]
    df2['island_o'][df2['iso3num_o'] == 500] = terrain_df.iloc[141,51]
    df2['island_d'][df2['iso3num_d'] == 500] = terrain_df.iloc[141,51]
    df2['landlocked_o'][df2['iso3num_o'] == 500] = terrain_df.iloc[141,52]
    df2['landlocked_d'][df2['iso3num_d'] == 500] = terrain_df.iloc[141,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 474] = terrain_df.iloc[142,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 474] = terrain_df.iloc[142,4]
    df2['desert_o'][df2['iso3num_o'] == 474] = terrain_df.iloc[142,12]
    df2['desert_d'][df2['iso3num_d'] == 474] = terrain_df.iloc[142,12]
    df2['tropical_o'][df2['iso3num_o'] == 474] = terrain_df.iloc[142,13]
    df2['tropical_d'][df2['iso3num_d'] == 474] = terrain_df.iloc[142,13]
    df2['island_o'][df2['iso3num_o'] == 474] = terrain_df.iloc[142,51]
    df2['island_d'][df2['iso3num_d'] == 474] = terrain_df.iloc[142,51]
    df2['landlocked_o'][df2['iso3num_o'] == 474] = terrain_df.iloc[142,52]
    df2['landlocked_d'][df2['iso3num_d'] == 474] = terrain_df.iloc[142,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 480] = terrain_df.iloc[143,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 480] = terrain_df.iloc[143,4]
    df2['desert_o'][df2['iso3num_o'] == 480] = terrain_df.iloc[143,12]
    df2['desert_d'][df2['iso3num_d'] == 480] = terrain_df.iloc[143,12]
    df2['tropical_o'][df2['iso3num_o'] == 480] = terrain_df.iloc[143,13]
    df2['tropical_d'][df2['iso3num_d'] == 480] = terrain_df.iloc[143,13]
    df2['island_o'][df2['iso3num_o'] == 480] = terrain_df.iloc[143,51]
    df2['island_d'][df2['iso3num_d'] == 480] = terrain_df.iloc[143,51]
    df2['landlocked_o'][df2['iso3num_o'] == 480] = terrain_df.iloc[143,52]
    df2['landlocked_d'][df2['iso3num_d'] == 480] = terrain_df.iloc[143,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 454] = terrain_df.iloc[144,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 454] = terrain_df.iloc[144,4]
    df2['desert_o'][df2['iso3num_o'] == 454] = terrain_df.iloc[144,12]
    df2['desert_d'][df2['iso3num_d'] == 454] = terrain_df.iloc[144,12]
    df2['tropical_o'][df2['iso3num_o'] == 454] = terrain_df.iloc[144,13]
    df2['tropical_d'][df2['iso3num_d'] == 454] = terrain_df.iloc[144,13]
    df2['island_o'][df2['iso3num_o'] == 454] = terrain_df.iloc[144,51]
    df2['island_d'][df2['iso3num_d'] == 454] = terrain_df.iloc[144,51]
    df2['landlocked_o'][df2['iso3num_o'] == 454] = terrain_df.iloc[144,52]
    df2['landlocked_d'][df2['iso3num_d'] == 454] = terrain_df.iloc[144,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 458] = terrain_df.iloc[145,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 458] = terrain_df.iloc[145,4]
    df2['desert_o'][df2['iso3num_o'] == 458] = terrain_df.iloc[145,12]
    df2['desert_d'][df2['iso3num_d'] == 458] = terrain_df.iloc[145,12]
    df2['tropical_o'][df2['iso3num_o'] == 458] = terrain_df.iloc[145,13]
    df2['tropical_d'][df2['iso3num_d'] == 458] = terrain_df.iloc[145,13]
    df2['island_o'][df2['iso3num_o'] == 458] = terrain_df.iloc[145,51]
    df2['island_d'][df2['iso3num_d'] == 458] = terrain_df.iloc[145,51]
    df2['landlocked_o'][df2['iso3num_o'] == 458] = terrain_df.iloc[145,52]
    df2['landlocked_d'][df2['iso3num_d'] == 458] = terrain_df.iloc[145,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 175] = terrain_df.iloc[146,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 175] = terrain_df.iloc[146,4]
    df2['desert_o'][df2['iso3num_o'] == 175] = terrain_df.iloc[146,12]
    df2['desert_d'][df2['iso3num_d'] == 175] = terrain_df.iloc[146,12]
    df2['tropical_o'][df2['iso3num_o'] == 175] = terrain_df.iloc[146,13]
    df2['tropical_d'][df2['iso3num_d'] == 175] = terrain_df.iloc[146,13]
    df2['island_o'][df2['iso3num_o'] == 175] = terrain_df.iloc[146,51]
    df2['island_d'][df2['iso3num_d'] == 175] = terrain_df.iloc[146,51]
    df2['landlocked_o'][df2['iso3num_o'] == 175] = terrain_df.iloc[146,52]
    df2['landlocked_d'][df2['iso3num_d'] == 175] = terrain_df.iloc[146,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 516] = terrain_df.iloc[147,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 516] = terrain_df.iloc[147,4]
    df2['desert_o'][df2['iso3num_o'] == 516] = terrain_df.iloc[147,12]
    df2['desert_d'][df2['iso3num_d'] == 516] = terrain_df.iloc[147,12]
    df2['tropical_o'][df2['iso3num_o'] == 516] = terrain_df.iloc[147,13]
    df2['tropical_d'][df2['iso3num_d'] == 516] = terrain_df.iloc[147,13]
    df2['island_o'][df2['iso3num_o'] == 516] = terrain_df.iloc[147,51]
    df2['island_d'][df2['iso3num_d'] == 516] = terrain_df.iloc[147,51]
    df2['landlocked_o'][df2['iso3num_o'] == 516] = terrain_df.iloc[147,52]
    df2['landlocked_d'][df2['iso3num_d'] == 516] = terrain_df.iloc[147,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 540] = terrain_df.iloc[148,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 540] = terrain_df.iloc[148,4]
    df2['desert_o'][df2['iso3num_o'] == 540] = terrain_df.iloc[148,12]
    df2['desert_d'][df2['iso3num_d'] == 540] = terrain_df.iloc[148,12]
    df2['tropical_o'][df2['iso3num_o'] == 540] = terrain_df.iloc[148,13]
    df2['tropical_d'][df2['iso3num_d'] == 540] = terrain_df.iloc[148,13]
    df2['island_o'][df2['iso3num_o'] == 540] = terrain_df.iloc[148,51]
    df2['island_d'][df2['iso3num_d'] == 540] = terrain_df.iloc[148,51]
    df2['landlocked_o'][df2['iso3num_o'] == 540] = terrain_df.iloc[148,52]
    df2['landlocked_d'][df2['iso3num_d'] == 540] = terrain_df.iloc[148,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 562] = terrain_df.iloc[149,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 562] = terrain_df.iloc[149,4]
    df2['desert_o'][df2['iso3num_o'] == 562] = terrain_df.iloc[149,12]
    df2['desert_d'][df2['iso3num_d'] == 562] = terrain_df.iloc[149,12]
    df2['tropical_o'][df2['iso3num_o'] == 562] = terrain_df.iloc[149,13]
    df2['tropical_d'][df2['iso3num_d'] == 562] = terrain_df.iloc[149,13]
    df2['island_o'][df2['iso3num_o'] == 562] = terrain_df.iloc[149,51]
    df2['island_d'][df2['iso3num_d'] == 562] = terrain_df.iloc[149,51]
    df2['landlocked_o'][df2['iso3num_o'] == 562] = terrain_df.iloc[149,52]
    df2['landlocked_d'][df2['iso3num_d'] == 562] = terrain_df.iloc[149,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 574] = terrain_df.iloc[150,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 574] = terrain_df.iloc[150,4]
    df2['desert_o'][df2['iso3num_o'] == 574] = terrain_df.iloc[150,12]
    df2['desert_d'][df2['iso3num_d'] == 574] = terrain_df.iloc[150,12]
    df2['tropical_o'][df2['iso3num_o'] == 574] = terrain_df.iloc[150,13]
    df2['tropical_d'][df2['iso3num_d'] == 574] = terrain_df.iloc[150,13]
    df2['island_o'][df2['iso3num_o'] == 574] = terrain_df.iloc[150,51]
    df2['island_d'][df2['iso3num_d'] == 574] = terrain_df.iloc[150,51]
    df2['landlocked_o'][df2['iso3num_o'] == 574] = terrain_df.iloc[150,52]
    df2['landlocked_d'][df2['iso3num_d'] == 574] = terrain_df.iloc[150,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 566] = terrain_df.iloc[151,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 566] = terrain_df.iloc[151,4]
    df2['desert_o'][df2['iso3num_o'] == 566] = terrain_df.iloc[151,12]
    df2['desert_d'][df2['iso3num_d'] == 566] = terrain_df.iloc[151,12]
    df2['tropical_o'][df2['iso3num_o'] == 566] = terrain_df.iloc[151,13]
    df2['tropical_d'][df2['iso3num_d'] == 566] = terrain_df.iloc[151,13]
    df2['island_o'][df2['iso3num_o'] == 566] = terrain_df.iloc[151,51]
    df2['island_d'][df2['iso3num_d'] == 566] = terrain_df.iloc[151,51]
    df2['landlocked_o'][df2['iso3num_o'] == 566] = terrain_df.iloc[151,52]
    df2['landlocked_d'][df2['iso3num_d'] == 566] = terrain_df.iloc[151,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 558] = terrain_df.iloc[152,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 558] = terrain_df.iloc[152,4]
    df2['desert_o'][df2['iso3num_o'] == 558] = terrain_df.iloc[152,12]
    df2['desert_d'][df2['iso3num_d'] == 558] = terrain_df.iloc[152,12]
    df2['tropical_o'][df2['iso3num_o'] == 558] = terrain_df.iloc[152,13]
    df2['tropical_d'][df2['iso3num_d'] == 558] = terrain_df.iloc[152,13]
    df2['island_o'][df2['iso3num_o'] == 558] = terrain_df.iloc[152,51]
    df2['island_d'][df2['iso3num_d'] == 558] = terrain_df.iloc[152,51]
    df2['landlocked_o'][df2['iso3num_o'] == 558] = terrain_df.iloc[152,52]
    df2['landlocked_d'][df2['iso3num_d'] == 558] = terrain_df.iloc[152,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 570] = terrain_df.iloc[153,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 570] = terrain_df.iloc[153,4]
    df2['desert_o'][df2['iso3num_o'] == 570] = terrain_df.iloc[153,12]
    df2['desert_d'][df2['iso3num_d'] == 570] = terrain_df.iloc[153,12]
    df2['tropical_o'][df2['iso3num_o'] == 570] = terrain_df.iloc[153,13]
    df2['tropical_d'][df2['iso3num_d'] == 570] = terrain_df.iloc[153,13]
    df2['island_o'][df2['iso3num_o'] == 570] = terrain_df.iloc[153,51]
    df2['island_d'][df2['iso3num_d'] == 570] = terrain_df.iloc[153,51]
    df2['landlocked_o'][df2['iso3num_o'] == 570] = terrain_df.iloc[153,52]
    df2['landlocked_d'][df2['iso3num_d'] == 570] = terrain_df.iloc[153,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 528] = terrain_df.iloc[154,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 528] = terrain_df.iloc[154,4]
    df2['desert_o'][df2['iso3num_o'] == 528] = terrain_df.iloc[154,12]
    df2['desert_d'][df2['iso3num_d'] == 528] = terrain_df.iloc[154,12]
    df2['tropical_o'][df2['iso3num_o'] == 528] = terrain_df.iloc[154,13]
    df2['tropical_d'][df2['iso3num_d'] == 528] = terrain_df.iloc[154,13]
    df2['island_o'][df2['iso3num_o'] == 528] = terrain_df.iloc[154,51]
    df2['island_d'][df2['iso3num_d'] == 528] = terrain_df.iloc[154,51]
    df2['landlocked_o'][df2['iso3num_o'] == 528] = terrain_df.iloc[154,52]
    df2['landlocked_d'][df2['iso3num_d'] == 528] = terrain_df.iloc[154,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 578] = terrain_df.iloc[155,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 578] = terrain_df.iloc[155,4]
    df2['desert_o'][df2['iso3num_o'] == 578] = terrain_df.iloc[155,12]
    df2['desert_d'][df2['iso3num_d'] == 578] = terrain_df.iloc[155,12]
    df2['tropical_o'][df2['iso3num_o'] == 578] = terrain_df.iloc[155,13]
    df2['tropical_d'][df2['iso3num_d'] == 578] = terrain_df.iloc[155,13]
    df2['island_o'][df2['iso3num_o'] == 578] = terrain_df.iloc[155,51]
    df2['island_d'][df2['iso3num_d'] == 578] = terrain_df.iloc[155,51]
    df2['landlocked_o'][df2['iso3num_o'] == 578] = terrain_df.iloc[155,52]
    df2['landlocked_d'][df2['iso3num_d'] == 578] = terrain_df.iloc[155,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 524] = terrain_df.iloc[156,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 524] = terrain_df.iloc[156,4]
    df2['desert_o'][df2['iso3num_o'] == 524] = terrain_df.iloc[156,12]
    df2['desert_d'][df2['iso3num_d'] == 524] = terrain_df.iloc[156,12]
    df2['tropical_o'][df2['iso3num_o'] == 524] = terrain_df.iloc[156,13]
    df2['tropical_d'][df2['iso3num_d'] == 524] = terrain_df.iloc[156,13]
    df2['island_o'][df2['iso3num_o'] == 524] = terrain_df.iloc[156,51]
    df2['island_d'][df2['iso3num_d'] == 524] = terrain_df.iloc[156,51]
    df2['landlocked_o'][df2['iso3num_o'] == 524] = terrain_df.iloc[156,52]
    df2['landlocked_d'][df2['iso3num_d'] == 524] = terrain_df.iloc[156,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 520] = terrain_df.iloc[157,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 520] = terrain_df.iloc[157,4]
    df2['desert_o'][df2['iso3num_o'] == 520] = terrain_df.iloc[157,12]
    df2['desert_d'][df2['iso3num_d'] == 520] = terrain_df.iloc[157,12]
    df2['tropical_o'][df2['iso3num_o'] == 520] = terrain_df.iloc[157,13]
    df2['tropical_d'][df2['iso3num_d'] == 520] = terrain_df.iloc[157,13]
    df2['island_o'][df2['iso3num_o'] == 520] = terrain_df.iloc[157,51]
    df2['island_d'][df2['iso3num_d'] == 520] = terrain_df.iloc[157,51]
    df2['landlocked_o'][df2['iso3num_o'] == 520] = terrain_df.iloc[157,52]
    df2['landlocked_d'][df2['iso3num_d'] == 520] = terrain_df.iloc[157,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 554] = terrain_df.iloc[158,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 554] = terrain_df.iloc[158,4]
    df2['desert_o'][df2['iso3num_o'] == 554] = terrain_df.iloc[158,12]
    df2['desert_d'][df2['iso3num_d'] == 554] = terrain_df.iloc[158,12]
    df2['tropical_o'][df2['iso3num_o'] == 554] = terrain_df.iloc[158,13]
    df2['tropical_d'][df2['iso3num_d'] == 554] = terrain_df.iloc[158,13]
    df2['island_o'][df2['iso3num_o'] == 554] = terrain_df.iloc[158,51]
    df2['island_d'][df2['iso3num_d'] == 554] = terrain_df.iloc[158,51]
    df2['landlocked_o'][df2['iso3num_o'] == 554] = terrain_df.iloc[158,52]
    df2['landlocked_d'][df2['iso3num_d'] == 554] = terrain_df.iloc[158,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 512] = terrain_df.iloc[159,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 512] = terrain_df.iloc[159,4]
    df2['desert_o'][df2['iso3num_o'] == 512] = terrain_df.iloc[159,12]
    df2['desert_d'][df2['iso3num_d'] == 512] = terrain_df.iloc[159,12]
    df2['tropical_o'][df2['iso3num_o'] == 512] = terrain_df.iloc[159,13]
    df2['tropical_d'][df2['iso3num_d'] == 512] = terrain_df.iloc[159,13]
    df2['island_o'][df2['iso3num_o'] == 512] = terrain_df.iloc[159,51]
    df2['island_d'][df2['iso3num_d'] == 512] = terrain_df.iloc[159,51]
    df2['landlocked_o'][df2['iso3num_o'] == 512] = terrain_df.iloc[159,52]
    df2['landlocked_d'][df2['iso3num_d'] == 512] = terrain_df.iloc[159,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 586] = terrain_df.iloc[160,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 586] = terrain_df.iloc[160,4]
    df2['desert_o'][df2['iso3num_o'] == 586] = terrain_df.iloc[160,12]
    df2['desert_d'][df2['iso3num_d'] == 586] = terrain_df.iloc[160,12]
    df2['tropical_o'][df2['iso3num_o'] == 586] = terrain_df.iloc[160,13]
    df2['tropical_d'][df2['iso3num_d'] == 586] = terrain_df.iloc[160,13]
    df2['island_o'][df2['iso3num_o'] == 586] = terrain_df.iloc[160,51]
    df2['island_d'][df2['iso3num_d'] == 586] = terrain_df.iloc[160,51]
    df2['landlocked_o'][df2['iso3num_o'] == 586] = terrain_df.iloc[160,52]
    df2['landlocked_d'][df2['iso3num_d'] == 586] = terrain_df.iloc[160,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 591] = terrain_df.iloc[161,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 591] = terrain_df.iloc[161,4]
    df2['desert_o'][df2['iso3num_o'] == 591] = terrain_df.iloc[161,12]
    df2['desert_d'][df2['iso3num_d'] == 591] = terrain_df.iloc[161,12]
    df2['tropical_o'][df2['iso3num_o'] == 591] = terrain_df.iloc[161,13]
    df2['tropical_d'][df2['iso3num_d'] == 591] = terrain_df.iloc[161,13]
    df2['island_o'][df2['iso3num_o'] == 591] = terrain_df.iloc[161,51]
    df2['island_d'][df2['iso3num_d'] == 591] = terrain_df.iloc[161,51]
    df2['landlocked_o'][df2['iso3num_o'] == 591] = terrain_df.iloc[161,52]
    df2['landlocked_d'][df2['iso3num_d'] == 591] = terrain_df.iloc[161,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 612] = terrain_df.iloc[162,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 612] = terrain_df.iloc[162,4]
    df2['desert_o'][df2['iso3num_o'] == 612] = terrain_df.iloc[162,12]
    df2['desert_d'][df2['iso3num_d'] == 612] = terrain_df.iloc[162,12]
    df2['tropical_o'][df2['iso3num_o'] == 612] = terrain_df.iloc[162,13]
    df2['tropical_d'][df2['iso3num_d'] == 612] = terrain_df.iloc[162,13]
    df2['island_o'][df2['iso3num_o'] == 612] = terrain_df.iloc[162,51]
    df2['island_d'][df2['iso3num_d'] == 612] = terrain_df.iloc[162,51]
    df2['landlocked_o'][df2['iso3num_o'] == 612] = terrain_df.iloc[162,52]
    df2['landlocked_d'][df2['iso3num_d'] == 612] = terrain_df.iloc[162,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 604] = terrain_df.iloc[163,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 604] = terrain_df.iloc[163,4]
    df2['desert_o'][df2['iso3num_o'] == 604] = terrain_df.iloc[163,12]
    df2['desert_d'][df2['iso3num_d'] == 604] = terrain_df.iloc[163,12]
    df2['tropical_o'][df2['iso3num_o'] == 604] = terrain_df.iloc[163,13]
    df2['tropical_d'][df2['iso3num_d'] == 604] = terrain_df.iloc[163,13]
    df2['island_o'][df2['iso3num_o'] == 604] = terrain_df.iloc[163,51]
    df2['island_d'][df2['iso3num_d'] == 604] = terrain_df.iloc[163,51]
    df2['landlocked_o'][df2['iso3num_o'] == 604] = terrain_df.iloc[163,52]
    df2['landlocked_d'][df2['iso3num_d'] == 604] = terrain_df.iloc[163,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 608] = terrain_df.iloc[164,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 608] = terrain_df.iloc[164,4]
    df2['desert_o'][df2['iso3num_o'] == 608] = terrain_df.iloc[164,12]
    df2['desert_d'][df2['iso3num_d'] == 608] = terrain_df.iloc[164,12]
    df2['tropical_o'][df2['iso3num_o'] == 608] = terrain_df.iloc[164,13]
    df2['tropical_d'][df2['iso3num_d'] == 608] = terrain_df.iloc[164,13]
    df2['island_o'][df2['iso3num_o'] == 608] = terrain_df.iloc[164,51]
    df2['island_d'][df2['iso3num_d'] == 608] = terrain_df.iloc[164,51]
    df2['landlocked_o'][df2['iso3num_o'] == 608] = terrain_df.iloc[164,52]
    df2['landlocked_d'][df2['iso3num_d'] == 608] = terrain_df.iloc[164,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 585] = terrain_df.iloc[165,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 585] = terrain_df.iloc[165,4]
    df2['desert_o'][df2['iso3num_o'] == 585] = terrain_df.iloc[165,12]
    df2['desert_d'][df2['iso3num_d'] == 585] = terrain_df.iloc[165,12]
    df2['tropical_o'][df2['iso3num_o'] == 585] = terrain_df.iloc[165,13]
    df2['tropical_d'][df2['iso3num_d'] == 585] = terrain_df.iloc[165,13]
    df2['island_o'][df2['iso3num_o'] == 585] = terrain_df.iloc[165,51]
    df2['island_d'][df2['iso3num_d'] == 585] = terrain_df.iloc[165,51]
    df2['landlocked_o'][df2['iso3num_o'] == 585] = terrain_df.iloc[165,52]
    df2['landlocked_d'][df2['iso3num_d'] == 585] = terrain_df.iloc[165,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 598] = terrain_df.iloc[166,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 598] = terrain_df.iloc[166,4]
    df2['desert_o'][df2['iso3num_o'] == 598] = terrain_df.iloc[166,12]
    df2['desert_d'][df2['iso3num_d'] == 598] = terrain_df.iloc[166,12]
    df2['tropical_o'][df2['iso3num_o'] == 598] = terrain_df.iloc[166,13]
    df2['tropical_d'][df2['iso3num_d'] == 598] = terrain_df.iloc[166,13]
    df2['island_o'][df2['iso3num_o'] == 598] = terrain_df.iloc[166,51]
    df2['island_d'][df2['iso3num_d'] == 598] = terrain_df.iloc[166,51]
    df2['landlocked_o'][df2['iso3num_o'] == 598] = terrain_df.iloc[166,52]
    df2['landlocked_d'][df2['iso3num_d'] == 598] = terrain_df.iloc[166,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 616] = terrain_df.iloc[167,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 616] = terrain_df.iloc[167,4]
    df2['desert_o'][df2['iso3num_o'] == 616] = terrain_df.iloc[167,12]
    df2['desert_d'][df2['iso3num_d'] == 616] = terrain_df.iloc[167,12]
    df2['tropical_o'][df2['iso3num_o'] == 616] = terrain_df.iloc[167,13]
    df2['tropical_d'][df2['iso3num_d'] == 616] = terrain_df.iloc[167,13]
    df2['island_o'][df2['iso3num_o'] == 616] = terrain_df.iloc[167,51]
    df2['island_d'][df2['iso3num_d'] == 616] = terrain_df.iloc[167,51]
    df2['landlocked_o'][df2['iso3num_o'] == 616] = terrain_df.iloc[167,52]
    df2['landlocked_d'][df2['iso3num_d'] == 616] = terrain_df.iloc[167,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 630] = terrain_df.iloc[168,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 630] = terrain_df.iloc[168,4]
    df2['desert_o'][df2['iso3num_o'] == 630] = terrain_df.iloc[168,12]
    df2['desert_d'][df2['iso3num_d'] == 630] = terrain_df.iloc[168,12]
    df2['tropical_o'][df2['iso3num_o'] == 630] = terrain_df.iloc[168,13]
    df2['tropical_d'][df2['iso3num_d'] == 630] = terrain_df.iloc[168,13]
    df2['island_o'][df2['iso3num_o'] == 630] = terrain_df.iloc[168,51]
    df2['island_d'][df2['iso3num_d'] == 630] = terrain_df.iloc[168,51]
    df2['landlocked_o'][df2['iso3num_o'] == 630] = terrain_df.iloc[168,52]
    df2['landlocked_d'][df2['iso3num_d'] == 630] = terrain_df.iloc[168,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 408] = terrain_df.iloc[169,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 408] = terrain_df.iloc[169,4]
    df2['desert_o'][df2['iso3num_o'] == 408] = terrain_df.iloc[169,12]
    df2['desert_d'][df2['iso3num_d'] == 408] = terrain_df.iloc[169,12]
    df2['tropical_o'][df2['iso3num_o'] == 408] = terrain_df.iloc[169,13]
    df2['tropical_d'][df2['iso3num_d'] == 408] = terrain_df.iloc[169,13]
    df2['island_o'][df2['iso3num_o'] == 408] = terrain_df.iloc[169,51]
    df2['island_d'][df2['iso3num_d'] == 408] = terrain_df.iloc[169,51]
    df2['landlocked_o'][df2['iso3num_o'] == 408] = terrain_df.iloc[169,52]
    df2['landlocked_d'][df2['iso3num_d'] == 408] = terrain_df.iloc[169,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 620] = terrain_df.iloc[170,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 620] = terrain_df.iloc[170,4]
    df2['desert_o'][df2['iso3num_o'] == 620] = terrain_df.iloc[170,12]
    df2['desert_d'][df2['iso3num_d'] == 620] = terrain_df.iloc[170,12]
    df2['tropical_o'][df2['iso3num_o'] == 620] = terrain_df.iloc[170,13]
    df2['tropical_d'][df2['iso3num_d'] == 620] = terrain_df.iloc[170,13]
    df2['island_o'][df2['iso3num_o'] == 620] = terrain_df.iloc[170,51]
    df2['island_d'][df2['iso3num_d'] == 620] = terrain_df.iloc[170,51]
    df2['landlocked_o'][df2['iso3num_o'] == 620] = terrain_df.iloc[170,52]
    df2['landlocked_d'][df2['iso3num_d'] == 620] = terrain_df.iloc[170,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 600] = terrain_df.iloc[171,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 600] = terrain_df.iloc[171,4]
    df2['desert_o'][df2['iso3num_o'] == 600] = terrain_df.iloc[171,12]
    df2['desert_d'][df2['iso3num_d'] == 600] = terrain_df.iloc[171,12]
    df2['tropical_o'][df2['iso3num_o'] == 600] = terrain_df.iloc[171,13]
    df2['tropical_d'][df2['iso3num_d'] == 600] = terrain_df.iloc[171,13]
    df2['island_o'][df2['iso3num_o'] == 600] = terrain_df.iloc[171,51]
    df2['island_d'][df2['iso3num_d'] == 600] = terrain_df.iloc[171,51]
    df2['landlocked_o'][df2['iso3num_o'] == 600] = terrain_df.iloc[171,52]
    df2['landlocked_d'][df2['iso3num_d'] == 600] = terrain_df.iloc[171,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 275] = terrain_df.iloc[172,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 275] = terrain_df.iloc[172,4]
    df2['desert_o'][df2['iso3num_o'] == 275] = terrain_df.iloc[172,12]
    df2['desert_d'][df2['iso3num_d'] == 275] = terrain_df.iloc[172,12]
    df2['tropical_o'][df2['iso3num_o'] == 275] = terrain_df.iloc[172,13]
    df2['tropical_d'][df2['iso3num_d'] == 275] = terrain_df.iloc[172,13]
    df2['island_o'][df2['iso3num_o'] == 275] = terrain_df.iloc[172,51]
    df2['island_d'][df2['iso3num_d'] == 275] = terrain_df.iloc[172,51]
    df2['landlocked_o'][df2['iso3num_o'] == 275] = terrain_df.iloc[172,52]
    df2['landlocked_d'][df2['iso3num_d'] == 275] = terrain_df.iloc[172,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 258] = terrain_df.iloc[173,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 258] = terrain_df.iloc[173,4]
    df2['desert_o'][df2['iso3num_o'] == 258] = terrain_df.iloc[173,12]
    df2['desert_d'][df2['iso3num_d'] == 258] = terrain_df.iloc[173,12]
    df2['tropical_o'][df2['iso3num_o'] == 258] = terrain_df.iloc[173,13]
    df2['tropical_d'][df2['iso3num_d'] == 258] = terrain_df.iloc[173,13]
    df2['island_o'][df2['iso3num_o'] == 258] = terrain_df.iloc[173,51]
    df2['island_d'][df2['iso3num_d'] == 258] = terrain_df.iloc[173,51]
    df2['landlocked_o'][df2['iso3num_o'] == 258] = terrain_df.iloc[173,52]
    df2['landlocked_d'][df2['iso3num_d'] == 258] = terrain_df.iloc[173,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 634] = terrain_df.iloc[174,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 634] = terrain_df.iloc[174,4]
    df2['desert_o'][df2['iso3num_o'] == 634] = terrain_df.iloc[174,12]
    df2['desert_d'][df2['iso3num_d'] == 634] = terrain_df.iloc[174,12]
    df2['tropical_o'][df2['iso3num_o'] == 634] = terrain_df.iloc[174,13]
    df2['tropical_d'][df2['iso3num_d'] == 634] = terrain_df.iloc[174,13]
    df2['island_o'][df2['iso3num_o'] == 634] = terrain_df.iloc[174,51]
    df2['island_d'][df2['iso3num_d'] == 634] = terrain_df.iloc[174,51]
    df2['landlocked_o'][df2['iso3num_o'] == 634] = terrain_df.iloc[174,52]
    df2['landlocked_d'][df2['iso3num_d'] == 634] = terrain_df.iloc[174,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 638] = terrain_df.iloc[175,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 638] = terrain_df.iloc[175,4]
    df2['desert_o'][df2['iso3num_o'] == 638] = terrain_df.iloc[175,12]
    df2['desert_d'][df2['iso3num_d'] == 638] = terrain_df.iloc[175,12]
    df2['tropical_o'][df2['iso3num_o'] == 638] = terrain_df.iloc[175,13]
    df2['tropical_d'][df2['iso3num_d'] == 638] = terrain_df.iloc[175,13]
    df2['island_o'][df2['iso3num_o'] == 638] = terrain_df.iloc[175,51]
    df2['island_d'][df2['iso3num_d'] == 638] = terrain_df.iloc[175,51]
    df2['landlocked_o'][df2['iso3num_o'] == 638] = terrain_df.iloc[175,52]
    df2['landlocked_d'][df2['iso3num_d'] == 638] = terrain_df.iloc[175,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 642] = terrain_df.iloc[176,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 642] = terrain_df.iloc[176,4]
    df2['desert_o'][df2['iso3num_o'] == 642] = terrain_df.iloc[176,12]
    df2['desert_d'][df2['iso3num_d'] == 642] = terrain_df.iloc[176,12]
    df2['tropical_o'][df2['iso3num_o'] == 642] = terrain_df.iloc[176,13]
    df2['tropical_d'][df2['iso3num_d'] == 642] = terrain_df.iloc[176,13]
    df2['island_o'][df2['iso3num_o'] == 642] = terrain_df.iloc[176,51]
    df2['island_d'][df2['iso3num_d'] == 642] = terrain_df.iloc[176,51]
    df2['landlocked_o'][df2['iso3num_o'] == 642] = terrain_df.iloc[176,52]
    df2['landlocked_d'][df2['iso3num_d'] == 642] = terrain_df.iloc[176,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 643] = terrain_df.iloc[177,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 643] = terrain_df.iloc[177,4]
    df2['desert_o'][df2['iso3num_o'] == 643] = terrain_df.iloc[177,12]
    df2['desert_d'][df2['iso3num_d'] == 643] = terrain_df.iloc[177,12]
    df2['tropical_o'][df2['iso3num_o'] == 643] = terrain_df.iloc[177,13]
    df2['tropical_d'][df2['iso3num_d'] == 643] = terrain_df.iloc[177,13]
    df2['island_o'][df2['iso3num_o'] == 643] = terrain_df.iloc[177,51]
    df2['island_d'][df2['iso3num_d'] == 643] = terrain_df.iloc[177,51]
    df2['landlocked_o'][df2['iso3num_o'] == 643] = terrain_df.iloc[177,52]
    df2['landlocked_d'][df2['iso3num_d'] == 643] = terrain_df.iloc[177,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 646] = terrain_df.iloc[178,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 646] = terrain_df.iloc[178,4]
    df2['desert_o'][df2['iso3num_o'] == 646] = terrain_df.iloc[178,12]
    df2['desert_d'][df2['iso3num_d'] == 646] = terrain_df.iloc[178,12]
    df2['tropical_o'][df2['iso3num_o'] == 646] = terrain_df.iloc[178,13]
    df2['tropical_d'][df2['iso3num_d'] == 646] = terrain_df.iloc[178,13]
    df2['island_o'][df2['iso3num_o'] == 646] = terrain_df.iloc[178,51]
    df2['island_d'][df2['iso3num_d'] == 646] = terrain_df.iloc[178,51]
    df2['landlocked_o'][df2['iso3num_o'] == 646] = terrain_df.iloc[178,52]
    df2['landlocked_d'][df2['iso3num_d'] == 646] = terrain_df.iloc[178,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 682] = terrain_df.iloc[179,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 682] = terrain_df.iloc[179,4]
    df2['desert_o'][df2['iso3num_o'] == 682] = terrain_df.iloc[179,12]
    df2['desert_d'][df2['iso3num_d'] == 682] = terrain_df.iloc[179,12]
    df2['tropical_o'][df2['iso3num_o'] == 682] = terrain_df.iloc[179,13]
    df2['tropical_d'][df2['iso3num_d'] == 682] = terrain_df.iloc[179,13]
    df2['island_o'][df2['iso3num_o'] == 682] = terrain_df.iloc[179,51]
    df2['island_d'][df2['iso3num_d'] == 682] = terrain_df.iloc[179,51]
    df2['landlocked_o'][df2['iso3num_o'] == 682] = terrain_df.iloc[179,52]
    df2['landlocked_d'][df2['iso3num_d'] == 682] = terrain_df.iloc[179,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 891] = terrain_df.iloc[180,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 891] = terrain_df.iloc[180,4]
    df2['desert_o'][df2['iso3num_o'] == 891] = terrain_df.iloc[180,12]
    df2['desert_d'][df2['iso3num_d'] == 891] = terrain_df.iloc[180,12]
    df2['tropical_o'][df2['iso3num_o'] == 891] = terrain_df.iloc[180,13]
    df2['tropical_d'][df2['iso3num_d'] == 891] = terrain_df.iloc[180,13]
    df2['island_o'][df2['iso3num_o'] == 891] = terrain_df.iloc[180,51]
    df2['island_d'][df2['iso3num_d'] == 891] = terrain_df.iloc[180,51]
    df2['landlocked_o'][df2['iso3num_o'] == 891] = terrain_df.iloc[180,52]
    df2['landlocked_d'][df2['iso3num_d'] == 891] = terrain_df.iloc[180,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 736] = terrain_df.iloc[181,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 736] = terrain_df.iloc[181,4]
    df2['desert_o'][df2['iso3num_o'] == 736] = terrain_df.iloc[181,12]
    df2['desert_d'][df2['iso3num_d'] == 736] = terrain_df.iloc[181,12]
    df2['tropical_o'][df2['iso3num_o'] == 736] = terrain_df.iloc[181,13]
    df2['tropical_d'][df2['iso3num_d'] == 736] = terrain_df.iloc[181,13]
    df2['island_o'][df2['iso3num_o'] == 736] = terrain_df.iloc[181,51]
    df2['island_d'][df2['iso3num_d'] == 736] = terrain_df.iloc[181,51]
    df2['landlocked_o'][df2['iso3num_o'] == 736] = terrain_df.iloc[181,52]
    df2['landlocked_d'][df2['iso3num_d'] == 736] = terrain_df.iloc[181,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 686] = terrain_df.iloc[182,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 686] = terrain_df.iloc[182,4]
    df2['desert_o'][df2['iso3num_o'] == 686] = terrain_df.iloc[182,12]
    df2['desert_d'][df2['iso3num_d'] == 686] = terrain_df.iloc[182,12]
    df2['tropical_o'][df2['iso3num_o'] == 686] = terrain_df.iloc[182,13]
    df2['tropical_d'][df2['iso3num_d'] == 686] = terrain_df.iloc[182,13]
    df2['island_o'][df2['iso3num_o'] == 686] = terrain_df.iloc[182,51]
    df2['island_d'][df2['iso3num_d'] == 686] = terrain_df.iloc[182,51]
    df2['landlocked_o'][df2['iso3num_o'] == 686] = terrain_df.iloc[182,52]
    df2['landlocked_d'][df2['iso3num_d'] == 686] = terrain_df.iloc[182,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 702] = terrain_df.iloc[183,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 702] = terrain_df.iloc[183,4]
    df2['desert_o'][df2['iso3num_o'] == 702] = terrain_df.iloc[183,12]
    df2['desert_d'][df2['iso3num_d'] == 702] = terrain_df.iloc[183,12]
    df2['tropical_o'][df2['iso3num_o'] == 702] = terrain_df.iloc[183,13]
    df2['tropical_d'][df2['iso3num_d'] == 702] = terrain_df.iloc[183,13]
    df2['island_o'][df2['iso3num_o'] == 702] = terrain_df.iloc[183,51]
    df2['island_d'][df2['iso3num_d'] == 702] = terrain_df.iloc[183,51]
    df2['landlocked_o'][df2['iso3num_o'] == 702] = terrain_df.iloc[183,52]
    df2['landlocked_d'][df2['iso3num_d'] == 702] = terrain_df.iloc[183,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 654] = terrain_df.iloc[184,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 654] = terrain_df.iloc[184,4]
    df2['desert_o'][df2['iso3num_o'] == 654] = terrain_df.iloc[184,12]
    df2['desert_d'][df2['iso3num_d'] == 654] = terrain_df.iloc[184,12]
    df2['tropical_o'][df2['iso3num_o'] == 654] = terrain_df.iloc[184,13]
    df2['tropical_d'][df2['iso3num_d'] == 654] = terrain_df.iloc[184,13]
    df2['island_o'][df2['iso3num_o'] == 654] = terrain_df.iloc[184,51]
    df2['island_d'][df2['iso3num_d'] == 654] = terrain_df.iloc[184,51]
    df2['landlocked_o'][df2['iso3num_o'] == 654] = terrain_df.iloc[184,52]
    df2['landlocked_d'][df2['iso3num_d'] == 654] = terrain_df.iloc[184,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 744] = terrain_df.iloc[185,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 744] = terrain_df.iloc[185,4]
    df2['desert_o'][df2['iso3num_o'] == 744] = terrain_df.iloc[185,12]
    df2['desert_d'][df2['iso3num_d'] == 744] = terrain_df.iloc[185,12]
    df2['tropical_o'][df2['iso3num_o'] == 744] = terrain_df.iloc[185,13]
    df2['tropical_d'][df2['iso3num_d'] == 744] = terrain_df.iloc[185,13]
    df2['island_o'][df2['iso3num_o'] == 744] = terrain_df.iloc[185,51]
    df2['island_d'][df2['iso3num_d'] == 744] = terrain_df.iloc[185,51]
    df2['landlocked_o'][df2['iso3num_o'] == 744] = terrain_df.iloc[185,52]
    df2['landlocked_d'][df2['iso3num_d'] == 744] = terrain_df.iloc[185,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 90] = terrain_df.iloc[186,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 90] = terrain_df.iloc[186,4]
    df2['desert_o'][df2['iso3num_o'] == 90] = terrain_df.iloc[186,12]
    df2['desert_d'][df2['iso3num_d'] == 90] = terrain_df.iloc[186,12]
    df2['tropical_o'][df2['iso3num_o'] == 90] = terrain_df.iloc[186,13]
    df2['tropical_d'][df2['iso3num_d'] == 90] = terrain_df.iloc[186,13]
    df2['island_o'][df2['iso3num_o'] == 90] = terrain_df.iloc[186,51]
    df2['island_d'][df2['iso3num_d'] == 90] = terrain_df.iloc[186,51]
    df2['landlocked_o'][df2['iso3num_o'] == 90] = terrain_df.iloc[186,52]
    df2['landlocked_d'][df2['iso3num_d'] == 90] = terrain_df.iloc[186,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 694] = terrain_df.iloc[187,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 694] = terrain_df.iloc[187,4]
    df2['desert_o'][df2['iso3num_o'] == 694] = terrain_df.iloc[187,12]
    df2['desert_d'][df2['iso3num_d'] == 694] = terrain_df.iloc[187,12]
    df2['tropical_o'][df2['iso3num_o'] == 694] = terrain_df.iloc[187,13]
    df2['tropical_d'][df2['iso3num_d'] == 694] = terrain_df.iloc[187,13]
    df2['island_o'][df2['iso3num_o'] == 694] = terrain_df.iloc[187,51]
    df2['island_d'][df2['iso3num_d'] == 694] = terrain_df.iloc[187,51]
    df2['landlocked_o'][df2['iso3num_o'] == 694] = terrain_df.iloc[187,52]
    df2['landlocked_d'][df2['iso3num_d'] == 694] = terrain_df.iloc[187,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 222] = terrain_df.iloc[188,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 222] = terrain_df.iloc[188,4]
    df2['desert_o'][df2['iso3num_o'] == 222] = terrain_df.iloc[188,12]
    df2['desert_d'][df2['iso3num_d'] == 222] = terrain_df.iloc[188,12]
    df2['tropical_o'][df2['iso3num_o'] == 222] = terrain_df.iloc[188,13]
    df2['tropical_d'][df2['iso3num_d'] == 222] = terrain_df.iloc[188,13]
    df2['island_o'][df2['iso3num_o'] == 222] = terrain_df.iloc[188,51]
    df2['island_d'][df2['iso3num_d'] == 222] = terrain_df.iloc[188,51]
    df2['landlocked_o'][df2['iso3num_o'] == 222] = terrain_df.iloc[188,52]
    df2['landlocked_d'][df2['iso3num_d'] == 222] = terrain_df.iloc[188,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 674] = terrain_df.iloc[189,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 674] = terrain_df.iloc[189,4]
    df2['desert_o'][df2['iso3num_o'] == 674] = terrain_df.iloc[189,12]
    df2['desert_d'][df2['iso3num_d'] == 674] = terrain_df.iloc[189,12]
    df2['tropical_o'][df2['iso3num_o'] == 674] = terrain_df.iloc[189,13]
    df2['tropical_d'][df2['iso3num_d'] == 674] = terrain_df.iloc[189,13]
    df2['island_o'][df2['iso3num_o'] == 674] = terrain_df.iloc[189,51]
    df2['island_d'][df2['iso3num_d'] == 674] = terrain_df.iloc[189,51]
    df2['landlocked_o'][df2['iso3num_o'] == 674] = terrain_df.iloc[189,52]
    df2['landlocked_d'][df2['iso3num_d'] == 674] = terrain_df.iloc[189,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 706] = terrain_df.iloc[190,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 706] = terrain_df.iloc[190,4]
    df2['desert_o'][df2['iso3num_o'] == 706] = terrain_df.iloc[190,12]
    df2['desert_d'][df2['iso3num_d'] == 706] = terrain_df.iloc[190,12]
    df2['tropical_o'][df2['iso3num_o'] == 706] = terrain_df.iloc[190,13]
    df2['tropical_d'][df2['iso3num_d'] == 706] = terrain_df.iloc[190,13]
    df2['island_o'][df2['iso3num_o'] == 706] = terrain_df.iloc[190,51]
    df2['island_d'][df2['iso3num_d'] == 706] = terrain_df.iloc[190,51]
    df2['landlocked_o'][df2['iso3num_o'] == 706] = terrain_df.iloc[190,52]
    df2['landlocked_d'][df2['iso3num_d'] == 706] = terrain_df.iloc[190,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 666] = terrain_df.iloc[191,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 666] = terrain_df.iloc[191,4]
    df2['desert_o'][df2['iso3num_o'] == 666] = terrain_df.iloc[191,12]
    df2['desert_d'][df2['iso3num_d'] == 666] = terrain_df.iloc[191,12]
    df2['tropical_o'][df2['iso3num_o'] == 666] = terrain_df.iloc[191,13]
    df2['tropical_d'][df2['iso3num_d'] == 666] = terrain_df.iloc[191,13]
    df2['island_o'][df2['iso3num_o'] == 666] = terrain_df.iloc[191,51]
    df2['island_d'][df2['iso3num_d'] == 666] = terrain_df.iloc[191,51]
    df2['landlocked_o'][df2['iso3num_o'] == 666] = terrain_df.iloc[191,52]
    df2['landlocked_d'][df2['iso3num_d'] == 666] = terrain_df.iloc[191,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 678] = terrain_df.iloc[192,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 678] = terrain_df.iloc[192,4]
    df2['desert_o'][df2['iso3num_o'] == 678] = terrain_df.iloc[192,12]
    df2['desert_d'][df2['iso3num_d'] == 678] = terrain_df.iloc[192,12]
    df2['tropical_o'][df2['iso3num_o'] == 678] = terrain_df.iloc[192,13]
    df2['tropical_d'][df2['iso3num_d'] == 678] = terrain_df.iloc[192,13]
    df2['island_o'][df2['iso3num_o'] == 678] = terrain_df.iloc[192,51]
    df2['island_d'][df2['iso3num_d'] == 678] = terrain_df.iloc[192,51]
    df2['landlocked_o'][df2['iso3num_o'] == 678] = terrain_df.iloc[192,52]
    df2['landlocked_d'][df2['iso3num_d'] == 678] = terrain_df.iloc[192,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 740] = terrain_df.iloc[193,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 740] = terrain_df.iloc[193,4]
    df2['desert_o'][df2['iso3num_o'] == 740] = terrain_df.iloc[193,12]
    df2['desert_d'][df2['iso3num_d'] == 740] = terrain_df.iloc[193,12]
    df2['tropical_o'][df2['iso3num_o'] == 740] = terrain_df.iloc[193,13]
    df2['tropical_d'][df2['iso3num_d'] == 740] = terrain_df.iloc[193,13]
    df2['island_o'][df2['iso3num_o'] == 740] = terrain_df.iloc[193,51]
    df2['island_d'][df2['iso3num_d'] == 740] = terrain_df.iloc[193,51]
    df2['landlocked_o'][df2['iso3num_o'] == 740] = terrain_df.iloc[193,52]
    df2['landlocked_d'][df2['iso3num_d'] == 740] = terrain_df.iloc[193,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 703] = terrain_df.iloc[194,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 703] = terrain_df.iloc[194,4]
    df2['desert_o'][df2['iso3num_o'] == 703] = terrain_df.iloc[194,12]
    df2['desert_d'][df2['iso3num_d'] == 703] = terrain_df.iloc[194,12]
    df2['tropical_o'][df2['iso3num_o'] == 703] = terrain_df.iloc[194,13]
    df2['tropical_d'][df2['iso3num_d'] == 703] = terrain_df.iloc[194,13]
    df2['island_o'][df2['iso3num_o'] == 703] = terrain_df.iloc[194,51]
    df2['island_d'][df2['iso3num_d'] == 703] = terrain_df.iloc[194,51]
    df2['landlocked_o'][df2['iso3num_o'] == 703] = terrain_df.iloc[194,52]
    df2['landlocked_d'][df2['iso3num_d'] == 703] = terrain_df.iloc[194,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 705] = terrain_df.iloc[195,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 705] = terrain_df.iloc[195,4]
    df2['desert_o'][df2['iso3num_o'] == 705] = terrain_df.iloc[195,12]
    df2['desert_d'][df2['iso3num_d'] == 705] = terrain_df.iloc[195,12]
    df2['tropical_o'][df2['iso3num_o'] == 705] = terrain_df.iloc[195,13]
    df2['tropical_d'][df2['iso3num_d'] == 705] = terrain_df.iloc[195,13]
    df2['island_o'][df2['iso3num_o'] == 705] = terrain_df.iloc[195,51]
    df2['island_d'][df2['iso3num_d'] == 705] = terrain_df.iloc[195,51]
    df2['landlocked_o'][df2['iso3num_o'] == 705] = terrain_df.iloc[195,52]
    df2['landlocked_d'][df2['iso3num_d'] == 705] = terrain_df.iloc[195,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 752] = terrain_df.iloc[196,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 752] = terrain_df.iloc[196,4]
    df2['desert_o'][df2['iso3num_o'] == 752] = terrain_df.iloc[196,12]
    df2['desert_d'][df2['iso3num_d'] == 752] = terrain_df.iloc[196,12]
    df2['tropical_o'][df2['iso3num_o'] == 752] = terrain_df.iloc[196,13]
    df2['tropical_d'][df2['iso3num_d'] == 752] = terrain_df.iloc[196,13]
    df2['island_o'][df2['iso3num_o'] == 752] = terrain_df.iloc[196,51]
    df2['island_d'][df2['iso3num_d'] == 752] = terrain_df.iloc[196,51]
    df2['landlocked_o'][df2['iso3num_o'] == 752] = terrain_df.iloc[196,52]
    df2['landlocked_d'][df2['iso3num_d'] == 752] = terrain_df.iloc[196,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 748] = terrain_df.iloc[197,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 748] = terrain_df.iloc[197,4]
    df2['desert_o'][df2['iso3num_o'] == 748] = terrain_df.iloc[197,12]
    df2['desert_d'][df2['iso3num_d'] == 748] = terrain_df.iloc[197,12]
    df2['tropical_o'][df2['iso3num_o'] == 748] = terrain_df.iloc[197,13]
    df2['tropical_d'][df2['iso3num_d'] == 748] = terrain_df.iloc[197,13]
    df2['island_o'][df2['iso3num_o'] == 748] = terrain_df.iloc[197,51]
    df2['island_d'][df2['iso3num_d'] == 748] = terrain_df.iloc[197,51]
    df2['landlocked_o'][df2['iso3num_o'] == 748] = terrain_df.iloc[197,52]
    df2['landlocked_d'][df2['iso3num_d'] == 748] = terrain_df.iloc[197,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 690] = terrain_df.iloc[198,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 690] = terrain_df.iloc[198,4]
    df2['desert_o'][df2['iso3num_o'] == 690] = terrain_df.iloc[198,12]
    df2['desert_d'][df2['iso3num_d'] == 690] = terrain_df.iloc[198,12]
    df2['tropical_o'][df2['iso3num_o'] == 690] = terrain_df.iloc[198,13]
    df2['tropical_d'][df2['iso3num_d'] == 690] = terrain_df.iloc[198,13]
    df2['island_o'][df2['iso3num_o'] == 690] = terrain_df.iloc[198,51]
    df2['island_d'][df2['iso3num_d'] == 690] = terrain_df.iloc[198,51]
    df2['landlocked_o'][df2['iso3num_o'] == 690] = terrain_df.iloc[198,52]
    df2['landlocked_d'][df2['iso3num_d'] == 690] = terrain_df.iloc[198,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 760] = terrain_df.iloc[199,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 760] = terrain_df.iloc[199,4]
    df2['desert_o'][df2['iso3num_o'] == 760] = terrain_df.iloc[199,12]
    df2['desert_d'][df2['iso3num_d'] == 760] = terrain_df.iloc[199,12]
    df2['tropical_o'][df2['iso3num_o'] == 760] = terrain_df.iloc[199,13]
    df2['tropical_d'][df2['iso3num_d'] == 760] = terrain_df.iloc[199,13]
    df2['island_o'][df2['iso3num_o'] == 760] = terrain_df.iloc[199,51]
    df2['island_d'][df2['iso3num_d'] == 760] = terrain_df.iloc[199,51]
    df2['landlocked_o'][df2['iso3num_o'] == 760] = terrain_df.iloc[199,52]
    df2['landlocked_d'][df2['iso3num_d'] == 760] = terrain_df.iloc[199,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 796] = terrain_df.iloc[200,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 796] = terrain_df.iloc[200,4]
    df2['desert_o'][df2['iso3num_o'] == 796] = terrain_df.iloc[200,12]
    df2['desert_d'][df2['iso3num_d'] == 796] = terrain_df.iloc[200,12]
    df2['tropical_o'][df2['iso3num_o'] == 796] = terrain_df.iloc[200,13]
    df2['tropical_d'][df2['iso3num_d'] == 796] = terrain_df.iloc[200,13]
    df2['island_o'][df2['iso3num_o'] == 796] = terrain_df.iloc[200,51]
    df2['island_d'][df2['iso3num_d'] == 796] = terrain_df.iloc[200,51]
    df2['landlocked_o'][df2['iso3num_o'] == 796] = terrain_df.iloc[200,52]
    df2['landlocked_d'][df2['iso3num_d'] == 796] = terrain_df.iloc[200,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 148] = terrain_df.iloc[201,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 148] = terrain_df.iloc[201,4]
    df2['desert_o'][df2['iso3num_o'] == 148] = terrain_df.iloc[201,12]
    df2['desert_d'][df2['iso3num_d'] == 148] = terrain_df.iloc[201,12]
    df2['tropical_o'][df2['iso3num_o'] == 148] = terrain_df.iloc[201,13]
    df2['tropical_d'][df2['iso3num_d'] == 148] = terrain_df.iloc[201,13]
    df2['island_o'][df2['iso3num_o'] == 148] = terrain_df.iloc[201,51]
    df2['island_d'][df2['iso3num_d'] == 148] = terrain_df.iloc[201,51]
    df2['landlocked_o'][df2['iso3num_o'] == 148] = terrain_df.iloc[201,52]
    df2['landlocked_d'][df2['iso3num_d'] == 148] = terrain_df.iloc[201,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 768] = terrain_df.iloc[202,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 768] = terrain_df.iloc[202,4]
    df2['desert_o'][df2['iso3num_o'] == 768] = terrain_df.iloc[202,12]
    df2['desert_d'][df2['iso3num_d'] == 768] = terrain_df.iloc[202,12]
    df2['tropical_o'][df2['iso3num_o'] == 768] = terrain_df.iloc[202,13]
    df2['tropical_d'][df2['iso3num_d'] == 768] = terrain_df.iloc[202,13]
    df2['island_o'][df2['iso3num_o'] == 768] = terrain_df.iloc[202,51]
    df2['island_d'][df2['iso3num_d'] == 768] = terrain_df.iloc[202,51]
    df2['landlocked_o'][df2['iso3num_o'] == 768] = terrain_df.iloc[202,52]
    df2['landlocked_d'][df2['iso3num_d'] == 768] = terrain_df.iloc[202,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 764] = terrain_df.iloc[203,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 764] = terrain_df.iloc[203,4]
    df2['desert_o'][df2['iso3num_o'] == 764] = terrain_df.iloc[203,12]
    df2['desert_d'][df2['iso3num_d'] == 764] = terrain_df.iloc[203,12]
    df2['tropical_o'][df2['iso3num_o'] == 764] = terrain_df.iloc[203,13]
    df2['tropical_d'][df2['iso3num_d'] == 764] = terrain_df.iloc[203,13]
    df2['island_o'][df2['iso3num_o'] == 764] = terrain_df.iloc[203,51]
    df2['island_d'][df2['iso3num_d'] == 764] = terrain_df.iloc[203,51]
    df2['landlocked_o'][df2['iso3num_o'] == 764] = terrain_df.iloc[203,52]
    df2['landlocked_d'][df2['iso3num_d'] == 764] = terrain_df.iloc[203,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 762] = terrain_df.iloc[204,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 762] = terrain_df.iloc[204,4]
    df2['desert_o'][df2['iso3num_o'] == 762] = terrain_df.iloc[204,12]
    df2['desert_d'][df2['iso3num_d'] == 762] = terrain_df.iloc[204,12]
    df2['tropical_o'][df2['iso3num_o'] == 762] = terrain_df.iloc[204,13]
    df2['tropical_d'][df2['iso3num_d'] == 762] = terrain_df.iloc[204,13]
    df2['island_o'][df2['iso3num_o'] == 762] = terrain_df.iloc[204,51]
    df2['island_d'][df2['iso3num_d'] == 762] = terrain_df.iloc[204,51]
    df2['landlocked_o'][df2['iso3num_o'] == 762] = terrain_df.iloc[204,52]
    df2['landlocked_d'][df2['iso3num_d'] == 762] = terrain_df.iloc[204,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 772] = terrain_df.iloc[205,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 772] = terrain_df.iloc[205,4]
    df2['desert_o'][df2['iso3num_o'] == 772] = terrain_df.iloc[205,12]
    df2['desert_d'][df2['iso3num_d'] == 772] = terrain_df.iloc[205,12]
    df2['tropical_o'][df2['iso3num_o'] == 772] = terrain_df.iloc[205,13]
    df2['tropical_d'][df2['iso3num_d'] == 772] = terrain_df.iloc[205,13]
    df2['island_o'][df2['iso3num_o'] == 772] = terrain_df.iloc[205,51]
    df2['island_d'][df2['iso3num_d'] == 772] = terrain_df.iloc[205,51]
    df2['landlocked_o'][df2['iso3num_o'] == 772] = terrain_df.iloc[205,52]
    df2['landlocked_d'][df2['iso3num_d'] == 772] = terrain_df.iloc[205,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 795] = terrain_df.iloc[206,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 795] = terrain_df.iloc[206,4]
    df2['desert_o'][df2['iso3num_o'] == 795] = terrain_df.iloc[206,12]
    df2['desert_d'][df2['iso3num_d'] == 795] = terrain_df.iloc[206,12]
    df2['tropical_o'][df2['iso3num_o'] == 795] = terrain_df.iloc[206,13]
    df2['tropical_d'][df2['iso3num_d'] == 795] = terrain_df.iloc[206,13]
    df2['island_o'][df2['iso3num_o'] == 795] = terrain_df.iloc[206,51]
    df2['island_d'][df2['iso3num_d'] == 795] = terrain_df.iloc[206,51]
    df2['landlocked_o'][df2['iso3num_o'] == 795] = terrain_df.iloc[206,52]
    df2['landlocked_d'][df2['iso3num_d'] == 795] = terrain_df.iloc[206,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 626] = terrain_df.iloc[207,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 626] = terrain_df.iloc[207,4]
    df2['desert_o'][df2['iso3num_o'] == 626] = terrain_df.iloc[207,12]
    df2['desert_d'][df2['iso3num_d'] == 626] = terrain_df.iloc[207,12]
    df2['tropical_o'][df2['iso3num_o'] == 626] = terrain_df.iloc[207,13]
    df2['tropical_d'][df2['iso3num_d'] == 626] = terrain_df.iloc[207,13]
    df2['island_o'][df2['iso3num_o'] == 626] = terrain_df.iloc[207,51]
    df2['island_d'][df2['iso3num_d'] == 626] = terrain_df.iloc[207,51]
    df2['landlocked_o'][df2['iso3num_o'] == 626] = terrain_df.iloc[207,52]
    df2['landlocked_d'][df2['iso3num_d'] == 626] = terrain_df.iloc[207,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 776] = terrain_df.iloc[208,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 776] = terrain_df.iloc[208,4]
    df2['desert_o'][df2['iso3num_o'] == 776] = terrain_df.iloc[208,12]
    df2['desert_d'][df2['iso3num_d'] == 776] = terrain_df.iloc[208,12]
    df2['tropical_o'][df2['iso3num_o'] == 776] = terrain_df.iloc[208,13]
    df2['tropical_d'][df2['iso3num_d'] == 776] = terrain_df.iloc[208,13]
    df2['island_o'][df2['iso3num_o'] == 776] = terrain_df.iloc[208,51]
    df2['island_d'][df2['iso3num_d'] == 776] = terrain_df.iloc[208,51]
    df2['landlocked_o'][df2['iso3num_o'] == 776] = terrain_df.iloc[208,52]
    df2['landlocked_d'][df2['iso3num_d'] == 776] = terrain_df.iloc[208,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 780] = terrain_df.iloc[209,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 780] = terrain_df.iloc[209,4]
    df2['desert_o'][df2['iso3num_o'] == 780] = terrain_df.iloc[209,12]
    df2['desert_d'][df2['iso3num_d'] == 780] = terrain_df.iloc[209,12]
    df2['tropical_o'][df2['iso3num_o'] == 780] = terrain_df.iloc[209,13]
    df2['tropical_d'][df2['iso3num_d'] == 780] = terrain_df.iloc[209,13]
    df2['island_o'][df2['iso3num_o'] == 780] = terrain_df.iloc[209,51]
    df2['island_d'][df2['iso3num_d'] == 780] = terrain_df.iloc[209,51]
    df2['landlocked_o'][df2['iso3num_o'] == 780] = terrain_df.iloc[209,52]
    df2['landlocked_d'][df2['iso3num_d'] == 780] = terrain_df.iloc[209,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 788] = terrain_df.iloc[210,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 788] = terrain_df.iloc[210,4]
    df2['desert_o'][df2['iso3num_o'] == 788] = terrain_df.iloc[210,12]
    df2['desert_d'][df2['iso3num_d'] == 788] = terrain_df.iloc[210,12]
    df2['tropical_o'][df2['iso3num_o'] == 788] = terrain_df.iloc[210,13]
    df2['tropical_d'][df2['iso3num_d'] == 788] = terrain_df.iloc[210,13]
    df2['island_o'][df2['iso3num_o'] == 788] = terrain_df.iloc[210,51]
    df2['island_d'][df2['iso3num_d'] == 788] = terrain_df.iloc[210,51]
    df2['landlocked_o'][df2['iso3num_o'] == 788] = terrain_df.iloc[210,52]
    df2['landlocked_d'][df2['iso3num_d'] == 788] = terrain_df.iloc[210,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 792] = terrain_df.iloc[211,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 792] = terrain_df.iloc[211,4]
    df2['desert_o'][df2['iso3num_o'] == 792] = terrain_df.iloc[211,12]
    df2['desert_d'][df2['iso3num_d'] == 792] = terrain_df.iloc[211,12]
    df2['tropical_o'][df2['iso3num_o'] == 792] = terrain_df.iloc[211,13]
    df2['tropical_d'][df2['iso3num_d'] == 792] = terrain_df.iloc[211,13]
    df2['island_o'][df2['iso3num_o'] == 792] = terrain_df.iloc[211,51]
    df2['island_d'][df2['iso3num_d'] == 792] = terrain_df.iloc[211,51]
    df2['landlocked_o'][df2['iso3num_o'] == 792] = terrain_df.iloc[211,52]
    df2['landlocked_d'][df2['iso3num_d'] == 792] = terrain_df.iloc[211,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 798] = terrain_df.iloc[212,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 798] = terrain_df.iloc[212,4]
    df2['desert_o'][df2['iso3num_o'] == 798] = terrain_df.iloc[212,12]
    df2['desert_d'][df2['iso3num_d'] == 798] = terrain_df.iloc[212,12]
    df2['tropical_o'][df2['iso3num_o'] == 798] = terrain_df.iloc[212,13]
    df2['tropical_d'][df2['iso3num_d'] == 798] = terrain_df.iloc[212,13]
    df2['island_o'][df2['iso3num_o'] == 798] = terrain_df.iloc[212,51]
    df2['island_d'][df2['iso3num_d'] == 798] = terrain_df.iloc[212,51]
    df2['landlocked_o'][df2['iso3num_o'] == 798] = terrain_df.iloc[212,52]
    df2['landlocked_d'][df2['iso3num_d'] == 798] = terrain_df.iloc[212,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 158] = terrain_df.iloc[213,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 158] = terrain_df.iloc[213,4]
    df2['desert_o'][df2['iso3num_o'] == 158] = terrain_df.iloc[213,12]
    df2['desert_d'][df2['iso3num_d'] == 158] = terrain_df.iloc[213,12]
    df2['tropical_o'][df2['iso3num_o'] == 158] = terrain_df.iloc[213,13]
    df2['tropical_d'][df2['iso3num_d'] == 158] = terrain_df.iloc[213,13]
    df2['island_o'][df2['iso3num_o'] == 158] = terrain_df.iloc[213,51]
    df2['island_d'][df2['iso3num_d'] == 158] = terrain_df.iloc[213,51]
    df2['landlocked_o'][df2['iso3num_o'] == 158] = terrain_df.iloc[213,52]
    df2['landlocked_d'][df2['iso3num_d'] == 158] = terrain_df.iloc[213,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 834] = terrain_df.iloc[214,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 834] = terrain_df.iloc[214,4]
    df2['desert_o'][df2['iso3num_o'] == 834] = terrain_df.iloc[214,12]
    df2['desert_d'][df2['iso3num_d'] == 834] = terrain_df.iloc[214,12]
    df2['tropical_o'][df2['iso3num_o'] == 834] = terrain_df.iloc[214,13]
    df2['tropical_d'][df2['iso3num_d'] == 834] = terrain_df.iloc[214,13]
    df2['island_o'][df2['iso3num_o'] == 834] = terrain_df.iloc[214,51]
    df2['island_d'][df2['iso3num_d'] == 834] = terrain_df.iloc[214,51]
    df2['landlocked_o'][df2['iso3num_o'] == 834] = terrain_df.iloc[214,52]
    df2['landlocked_d'][df2['iso3num_d'] == 834] = terrain_df.iloc[214,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 800] = terrain_df.iloc[215,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 800] = terrain_df.iloc[215,4]
    df2['desert_o'][df2['iso3num_o'] == 800] = terrain_df.iloc[215,12]
    df2['desert_d'][df2['iso3num_d'] == 800] = terrain_df.iloc[215,12]
    df2['tropical_o'][df2['iso3num_o'] == 800] = terrain_df.iloc[215,13]
    df2['tropical_d'][df2['iso3num_d'] == 800] = terrain_df.iloc[215,13]
    df2['island_o'][df2['iso3num_o'] == 800] = terrain_df.iloc[215,51]
    df2['island_d'][df2['iso3num_d'] == 800] = terrain_df.iloc[215,51]
    df2['landlocked_o'][df2['iso3num_o'] == 800] = terrain_df.iloc[215,52]
    df2['landlocked_d'][df2['iso3num_d'] == 800] = terrain_df.iloc[215,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 804] = terrain_df.iloc[216,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 804] = terrain_df.iloc[216,4]
    df2['desert_o'][df2['iso3num_o'] == 804] = terrain_df.iloc[216,12]
    df2['desert_d'][df2['iso3num_d'] == 804] = terrain_df.iloc[216,12]
    df2['tropical_o'][df2['iso3num_o'] == 804] = terrain_df.iloc[216,13]
    df2['tropical_d'][df2['iso3num_d'] == 804] = terrain_df.iloc[216,13]
    df2['island_o'][df2['iso3num_o'] == 804] = terrain_df.iloc[216,51]
    df2['island_d'][df2['iso3num_d'] == 804] = terrain_df.iloc[216,51]
    df2['landlocked_o'][df2['iso3num_o'] == 804] = terrain_df.iloc[216,52]
    df2['landlocked_d'][df2['iso3num_d'] == 804] = terrain_df.iloc[216,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 581] = terrain_df.iloc[217,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 581] = terrain_df.iloc[217,4]
    df2['desert_o'][df2['iso3num_o'] == 581] = terrain_df.iloc[217,12]
    df2['desert_d'][df2['iso3num_d'] == 581] = terrain_df.iloc[217,12]
    df2['tropical_o'][df2['iso3num_o'] == 581] = terrain_df.iloc[217,13]
    df2['tropical_d'][df2['iso3num_d'] == 581] = terrain_df.iloc[217,13]
    df2['island_o'][df2['iso3num_o'] == 581] = terrain_df.iloc[217,51]
    df2['island_d'][df2['iso3num_d'] == 581] = terrain_df.iloc[217,51]
    df2['landlocked_o'][df2['iso3num_o'] == 581] = terrain_df.iloc[217,52]
    df2['landlocked_d'][df2['iso3num_d'] == 581] = terrain_df.iloc[217,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 858] = terrain_df.iloc[218,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 858] = terrain_df.iloc[218,4]
    df2['desert_o'][df2['iso3num_o'] == 858] = terrain_df.iloc[218,12]
    df2['desert_d'][df2['iso3num_d'] == 858] = terrain_df.iloc[218,12]
    df2['tropical_o'][df2['iso3num_o'] == 858] = terrain_df.iloc[218,13]
    df2['tropical_d'][df2['iso3num_d'] == 858] = terrain_df.iloc[218,13]
    df2['island_o'][df2['iso3num_o'] == 858] = terrain_df.iloc[218,51]
    df2['island_d'][df2['iso3num_d'] == 858] = terrain_df.iloc[218,51]
    df2['landlocked_o'][df2['iso3num_o'] == 858] = terrain_df.iloc[218,52]
    df2['landlocked_d'][df2['iso3num_d'] == 858] = terrain_df.iloc[218,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 840] = terrain_df.iloc[219,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 840] = terrain_df.iloc[219,4]
    df2['desert_o'][df2['iso3num_o'] == 840] = terrain_df.iloc[219,12]
    df2['desert_d'][df2['iso3num_d'] == 840] = terrain_df.iloc[219,12]
    df2['tropical_o'][df2['iso3num_o'] == 840] = terrain_df.iloc[219,13]
    df2['tropical_d'][df2['iso3num_d'] == 840] = terrain_df.iloc[219,13]
    df2['island_o'][df2['iso3num_o'] == 840] = terrain_df.iloc[219,51]
    df2['island_d'][df2['iso3num_d'] == 840] = terrain_df.iloc[219,51]
    df2['landlocked_o'][df2['iso3num_o'] == 840] = terrain_df.iloc[219,52]
    df2['landlocked_d'][df2['iso3num_d'] == 840] = terrain_df.iloc[219,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 860] = terrain_df.iloc[220,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 860] = terrain_df.iloc[220,4]
    df2['desert_o'][df2['iso3num_o'] == 860] = terrain_df.iloc[220,12]
    df2['desert_d'][df2['iso3num_d'] == 860] = terrain_df.iloc[220,12]
    df2['tropical_o'][df2['iso3num_o'] == 860] = terrain_df.iloc[220,13]
    df2['tropical_d'][df2['iso3num_d'] == 860] = terrain_df.iloc[220,13]
    df2['island_o'][df2['iso3num_o'] == 860] = terrain_df.iloc[220,51]
    df2['island_d'][df2['iso3num_d'] == 860] = terrain_df.iloc[220,51]
    df2['landlocked_o'][df2['iso3num_o'] == 860] = terrain_df.iloc[220,52]
    df2['landlocked_d'][df2['iso3num_d'] == 860] = terrain_df.iloc[220,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 336] = terrain_df.iloc[221,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 336] = terrain_df.iloc[221,4]
    df2['desert_o'][df2['iso3num_o'] == 336] = terrain_df.iloc[221,12]
    df2['desert_d'][df2['iso3num_d'] == 336] = terrain_df.iloc[221,12]
    df2['tropical_o'][df2['iso3num_o'] == 336] = terrain_df.iloc[221,13]
    df2['tropical_d'][df2['iso3num_d'] == 336] = terrain_df.iloc[221,13]
    df2['island_o'][df2['iso3num_o'] == 336] = terrain_df.iloc[221,51]
    df2['island_d'][df2['iso3num_d'] == 336] = terrain_df.iloc[221,51]
    df2['landlocked_o'][df2['iso3num_o'] == 336] = terrain_df.iloc[221,52]
    df2['landlocked_d'][df2['iso3num_d'] == 336] = terrain_df.iloc[221,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 670] = terrain_df.iloc[222,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 670] = terrain_df.iloc[222,4]
    df2['desert_o'][df2['iso3num_o'] == 670] = terrain_df.iloc[222,12]
    df2['desert_d'][df2['iso3num_d'] == 670] = terrain_df.iloc[222,12]
    df2['tropical_o'][df2['iso3num_o'] == 670] = terrain_df.iloc[222,13]
    df2['tropical_d'][df2['iso3num_d'] == 670] = terrain_df.iloc[222,13]
    df2['island_o'][df2['iso3num_o'] == 670] = terrain_df.iloc[222,51]
    df2['island_d'][df2['iso3num_d'] == 670] = terrain_df.iloc[222,51]
    df2['landlocked_o'][df2['iso3num_o'] == 670] = terrain_df.iloc[222,52]
    df2['landlocked_d'][df2['iso3num_d'] == 670] = terrain_df.iloc[222,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 862] = terrain_df.iloc[223,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 862] = terrain_df.iloc[223,4]
    df2['desert_o'][df2['iso3num_o'] == 862] = terrain_df.iloc[223,12]
    df2['desert_d'][df2['iso3num_d'] == 862] = terrain_df.iloc[223,12]
    df2['tropical_o'][df2['iso3num_o'] == 862] = terrain_df.iloc[223,13]
    df2['tropical_d'][df2['iso3num_d'] == 862] = terrain_df.iloc[223,13]
    df2['island_o'][df2['iso3num_o'] == 862] = terrain_df.iloc[223,51]
    df2['island_d'][df2['iso3num_d'] == 862] = terrain_df.iloc[223,51]
    df2['landlocked_o'][df2['iso3num_o'] == 862] = terrain_df.iloc[223,52]
    df2['landlocked_d'][df2['iso3num_d'] == 862] = terrain_df.iloc[223,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 92] = terrain_df.iloc[224,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 92] = terrain_df.iloc[224,4]
    df2['desert_o'][df2['iso3num_o'] == 92] = terrain_df.iloc[224,12]
    df2['desert_d'][df2['iso3num_d'] == 92] = terrain_df.iloc[224,12]
    df2['tropical_o'][df2['iso3num_o'] == 92] = terrain_df.iloc[224,13]
    df2['tropical_d'][df2['iso3num_d'] == 92] = terrain_df.iloc[224,13]
    df2['island_o'][df2['iso3num_o'] == 92] = terrain_df.iloc[224,51]
    df2['island_d'][df2['iso3num_d'] == 92] = terrain_df.iloc[224,51]
    df2['landlocked_o'][df2['iso3num_o'] == 92] = terrain_df.iloc[224,52]
    df2['landlocked_d'][df2['iso3num_d'] == 92] = terrain_df.iloc[224,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 850] = terrain_df.iloc[225,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 850] = terrain_df.iloc[225,4]
    df2['desert_o'][df2['iso3num_o'] == 850] = terrain_df.iloc[225,12]
    df2['desert_d'][df2['iso3num_d'] == 850] = terrain_df.iloc[225,12]
    df2['tropical_o'][df2['iso3num_o'] == 850] = terrain_df.iloc[225,13]
    df2['tropical_d'][df2['iso3num_d'] == 850] = terrain_df.iloc[225,13]
    df2['island_o'][df2['iso3num_o'] == 850] = terrain_df.iloc[225,51]
    df2['island_d'][df2['iso3num_d'] == 850] = terrain_df.iloc[225,51]
    df2['landlocked_o'][df2['iso3num_o'] == 850] = terrain_df.iloc[225,52]
    df2['landlocked_d'][df2['iso3num_d'] == 850] = terrain_df.iloc[225,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 704] = terrain_df.iloc[226,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 704] = terrain_df.iloc[226,4]
    df2['desert_o'][df2['iso3num_o'] == 704] = terrain_df.iloc[226,12]
    df2['desert_d'][df2['iso3num_d'] == 704] = terrain_df.iloc[226,12]
    df2['tropical_o'][df2['iso3num_o'] == 704] = terrain_df.iloc[226,13]
    df2['tropical_d'][df2['iso3num_d'] == 704] = terrain_df.iloc[226,13]
    df2['island_o'][df2['iso3num_o'] == 704] = terrain_df.iloc[226,51]
    df2['island_d'][df2['iso3num_d'] == 704] = terrain_df.iloc[226,51]
    df2['landlocked_o'][df2['iso3num_o'] == 704] = terrain_df.iloc[226,52]
    df2['landlocked_d'][df2['iso3num_d'] == 704] = terrain_df.iloc[226,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 548] = terrain_df.iloc[227,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 548] = terrain_df.iloc[227,4]
    df2['desert_o'][df2['iso3num_o'] == 548] = terrain_df.iloc[227,12]
    df2['desert_d'][df2['iso3num_d'] == 548] = terrain_df.iloc[227,12]
    df2['tropical_o'][df2['iso3num_o'] == 548] = terrain_df.iloc[227,13]
    df2['tropical_d'][df2['iso3num_d'] == 548] = terrain_df.iloc[227,13]
    df2['island_o'][df2['iso3num_o'] == 548] = terrain_df.iloc[227,51]
    df2['island_d'][df2['iso3num_d'] == 548] = terrain_df.iloc[227,51]
    df2['landlocked_o'][df2['iso3num_o'] == 548] = terrain_df.iloc[227,52]
    df2['landlocked_d'][df2['iso3num_d'] == 548] = terrain_df.iloc[227,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 876] = terrain_df.iloc[228,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 876] = terrain_df.iloc[228,4]
    df2['desert_o'][df2['iso3num_o'] == 876] = terrain_df.iloc[228,12]
    df2['desert_d'][df2['iso3num_d'] == 876] = terrain_df.iloc[228,12]
    df2['tropical_o'][df2['iso3num_o'] == 876] = terrain_df.iloc[228,13]
    df2['tropical_d'][df2['iso3num_d'] == 876] = terrain_df.iloc[228,13]
    df2['island_o'][df2['iso3num_o'] == 876] = terrain_df.iloc[228,51]
    df2['island_d'][df2['iso3num_d'] == 876] = terrain_df.iloc[228,51]
    df2['landlocked_o'][df2['iso3num_o'] == 876] = terrain_df.iloc[228,52]
    df2['landlocked_d'][df2['iso3num_d'] == 876] = terrain_df.iloc[228,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 882] = terrain_df.iloc[229,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 882] = terrain_df.iloc[229,4]
    df2['desert_o'][df2['iso3num_o'] == 882] = terrain_df.iloc[229,12]
    df2['desert_d'][df2['iso3num_d'] == 882] = terrain_df.iloc[229,12]
    df2['tropical_o'][df2['iso3num_o'] == 882] = terrain_df.iloc[229,13]
    df2['tropical_d'][df2['iso3num_d'] == 882] = terrain_df.iloc[229,13]
    df2['island_o'][df2['iso3num_o'] == 882] = terrain_df.iloc[229,51]
    df2['island_d'][df2['iso3num_d'] == 882] = terrain_df.iloc[229,51]
    df2['landlocked_o'][df2['iso3num_o'] == 882] = terrain_df.iloc[229,52]
    df2['landlocked_d'][df2['iso3num_d'] == 882] = terrain_df.iloc[229,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 887] = terrain_df.iloc[230,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 887] = terrain_df.iloc[230,4]
    df2['desert_o'][df2['iso3num_o'] == 887] = terrain_df.iloc[230,12]
    df2['desert_d'][df2['iso3num_d'] == 887] = terrain_df.iloc[230,12]
    df2['tropical_o'][df2['iso3num_o'] == 887] = terrain_df.iloc[230,13]
    df2['tropical_d'][df2['iso3num_d'] == 887] = terrain_df.iloc[230,13]
    df2['island_o'][df2['iso3num_o'] == 887] = terrain_df.iloc[230,51]
    df2['island_d'][df2['iso3num_d'] == 887] = terrain_df.iloc[230,51]
    df2['landlocked_o'][df2['iso3num_o'] == 887] = terrain_df.iloc[230,52]
    df2['landlocked_d'][df2['iso3num_d'] == 887] = terrain_df.iloc[230,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 710] = terrain_df.iloc[231,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 710] = terrain_df.iloc[231,4]
    df2['desert_o'][df2['iso3num_o'] == 710] = terrain_df.iloc[231,12]
    df2['desert_d'][df2['iso3num_d'] == 710] = terrain_df.iloc[231,12]
    df2['tropical_o'][df2['iso3num_o'] == 710] = terrain_df.iloc[231,13]
    df2['tropical_d'][df2['iso3num_d'] == 710] = terrain_df.iloc[231,13]
    df2['island_o'][df2['iso3num_o'] == 710] = terrain_df.iloc[231,51]
    df2['island_d'][df2['iso3num_d'] == 710] = terrain_df.iloc[231,51]
    df2['landlocked_o'][df2['iso3num_o'] == 710] = terrain_df.iloc[231,52]
    df2['landlocked_d'][df2['iso3num_d'] == 710] = terrain_df.iloc[231,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 894] = terrain_df.iloc[232,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 894] = terrain_df.iloc[232,4]
    df2['desert_o'][df2['iso3num_o'] == 894] = terrain_df.iloc[232,12]
    df2['desert_d'][df2['iso3num_d'] == 894] = terrain_df.iloc[232,12]
    df2['tropical_o'][df2['iso3num_o'] == 894] = terrain_df.iloc[232,13]
    df2['tropical_d'][df2['iso3num_d'] == 894] = terrain_df.iloc[232,13]
    df2['island_o'][df2['iso3num_o'] == 894] = terrain_df.iloc[232,51]
    df2['island_d'][df2['iso3num_d'] == 894] = terrain_df.iloc[232,51]
    df2['landlocked_o'][df2['iso3num_o'] == 894] = terrain_df.iloc[232,52]
    df2['landlocked_d'][df2['iso3num_d'] == 894] = terrain_df.iloc[232,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 716] = terrain_df.iloc[233,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 716] = terrain_df.iloc[233,4]
    df2['desert_o'][df2['iso3num_o'] == 716] = terrain_df.iloc[233,12]
    df2['desert_d'][df2['iso3num_d'] == 716] = terrain_df.iloc[233,12]
    df2['tropical_o'][df2['iso3num_o'] == 716] = terrain_df.iloc[233,13]
    df2['tropical_d'][df2['iso3num_d'] == 716] = terrain_df.iloc[233,13]
    df2['island_o'][df2['iso3num_o'] == 716] = terrain_df.iloc[233,51]
    df2['island_d'][df2['iso3num_d'] == 716] = terrain_df.iloc[233,51]
    df2['landlocked_o'][df2['iso3num_o'] == 716] = terrain_df.iloc[233,52]
    df2['landlocked_d'][df2['iso3num_d'] == 716] = terrain_df.iloc[233,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 200] = terrain_df.iloc[234,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 200] = terrain_df.iloc[234,4]
    df2['desert_o'][df2['iso3num_o'] == 200] = terrain_df.iloc[234,12]
    df2['desert_d'][df2['iso3num_d'] == 200] = terrain_df.iloc[234,12]
    df2['tropical_o'][df2['iso3num_o'] == 200] = terrain_df.iloc[234,13]
    df2['tropical_d'][df2['iso3num_d'] == 200] = terrain_df.iloc[234,13]
    df2['island_o'][df2['iso3num_o'] == 200] = terrain_df.iloc[234,51]
    df2['island_d'][df2['iso3num_d'] == 200] = terrain_df.iloc[234,51]
    df2['landlocked_o'][df2['iso3num_o'] == 200] = terrain_df.iloc[234,52]
    df2['landlocked_d'][df2['iso3num_d'] == 200] = terrain_df.iloc[234,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 278] = terrain_df.iloc[235,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 278] = terrain_df.iloc[235,4]
    df2['desert_o'][df2['iso3num_o'] == 278] = terrain_df.iloc[235,12]
    df2['desert_d'][df2['iso3num_d'] == 278] = terrain_df.iloc[235,12]
    df2['tropical_o'][df2['iso3num_o'] == 278] = terrain_df.iloc[235,13]
    df2['tropical_d'][df2['iso3num_d'] == 278] = terrain_df.iloc[235,13]
    df2['island_o'][df2['iso3num_o'] == 278] = terrain_df.iloc[235,51]
    df2['island_d'][df2['iso3num_d'] == 278] = terrain_df.iloc[235,51]
    df2['landlocked_o'][df2['iso3num_o'] == 278] = terrain_df.iloc[235,52]
    df2['landlocked_d'][df2['iso3num_d'] == 278] = terrain_df.iloc[235,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 280] = terrain_df.iloc[236,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 280] = terrain_df.iloc[236,4]
    df2['desert_o'][df2['iso3num_o'] == 280] = terrain_df.iloc[236,12]
    df2['desert_d'][df2['iso3num_d'] == 280] = terrain_df.iloc[236,12]
    df2['tropical_o'][df2['iso3num_o'] == 280] = terrain_df.iloc[236,13]
    df2['tropical_d'][df2['iso3num_d'] == 280] = terrain_df.iloc[236,13]
    df2['island_o'][df2['iso3num_o'] == 280] = terrain_df.iloc[236,51]
    df2['island_d'][df2['iso3num_d'] == 280] = terrain_df.iloc[236,51]
    df2['landlocked_o'][df2['iso3num_o'] == 280] = terrain_df.iloc[236,52]
    df2['landlocked_d'][df2['iso3num_d'] == 280] = terrain_df.iloc[236,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 230] = terrain_df.iloc[237,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 230] = terrain_df.iloc[237,4]
    df2['desert_o'][df2['iso3num_o'] == 230] = terrain_df.iloc[237,12]
    df2['desert_d'][df2['iso3num_d'] == 230] = terrain_df.iloc[237,12]
    df2['tropical_o'][df2['iso3num_o'] == 230] = terrain_df.iloc[237,13]
    df2['tropical_d'][df2['iso3num_d'] == 230] = terrain_df.iloc[237,13]
    df2['island_o'][df2['iso3num_o'] == 230] = terrain_df.iloc[237,51]
    df2['island_d'][df2['iso3num_d'] == 230] = terrain_df.iloc[237,51]
    df2['landlocked_o'][df2['iso3num_o'] == 230] = terrain_df.iloc[237,52]
    df2['landlocked_d'][df2['iso3num_d'] == 230] = terrain_df.iloc[237,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 499] = terrain_df.iloc[238,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 499] = terrain_df.iloc[238,4]
    df2['desert_o'][df2['iso3num_o'] == 499] = terrain_df.iloc[238,12]
    df2['desert_d'][df2['iso3num_d'] == 499] = terrain_df.iloc[238,12]
    df2['tropical_o'][df2['iso3num_o'] == 499] = terrain_df.iloc[238,13]
    df2['tropical_d'][df2['iso3num_d'] == 499] = terrain_df.iloc[238,13]
    df2['island_o'][df2['iso3num_o'] == 499] = terrain_df.iloc[238,51]
    df2['island_d'][df2['iso3num_d'] == 499] = terrain_df.iloc[238,51]
    df2['landlocked_o'][df2['iso3num_o'] == 499] = terrain_df.iloc[238,52]
    df2['landlocked_d'][df2['iso3num_d'] == 499] = terrain_df.iloc[238,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 590] = terrain_df.iloc[239,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 590] = terrain_df.iloc[239,4]
    df2['desert_o'][df2['iso3num_o'] == 590] = terrain_df.iloc[239,12]
    df2['desert_d'][df2['iso3num_d'] == 590] = terrain_df.iloc[239,12]
    df2['tropical_o'][df2['iso3num_o'] == 590] = terrain_df.iloc[239,13]
    df2['tropical_d'][df2['iso3num_d'] == 590] = terrain_df.iloc[239,13]
    df2['island_o'][df2['iso3num_o'] == 590] = terrain_df.iloc[239,51]
    df2['island_d'][df2['iso3num_d'] == 590] = terrain_df.iloc[239,51]
    df2['landlocked_o'][df2['iso3num_o'] == 590] = terrain_df.iloc[239,52]
    df2['landlocked_d'][df2['iso3num_d'] == 590] = terrain_df.iloc[239,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 729] = terrain_df.iloc[240,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 729] = terrain_df.iloc[240,4]
    df2['desert_o'][df2['iso3num_o'] == 729] = terrain_df.iloc[240,12]
    df2['desert_d'][df2['iso3num_d'] == 729] = terrain_df.iloc[240,12]
    df2['tropical_o'][df2['iso3num_o'] == 729] = terrain_df.iloc[240,13]
    df2['tropical_d'][df2['iso3num_d'] == 729] = terrain_df.iloc[240,13]
    df2['island_o'][df2['iso3num_o'] == 729] = terrain_df.iloc[240,51]
    df2['island_d'][df2['iso3num_d'] == 729] = terrain_df.iloc[240,51]
    df2['landlocked_o'][df2['iso3num_o'] == 729] = terrain_df.iloc[240,52]
    df2['landlocked_d'][df2['iso3num_d'] == 729] = terrain_df.iloc[240,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 688] = terrain_df.iloc[241,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 688] = terrain_df.iloc[241,4]
    df2['desert_o'][df2['iso3num_o'] == 688] = terrain_df.iloc[241,12]
    df2['desert_d'][df2['iso3num_d'] == 688] = terrain_df.iloc[241,12]
    df2['tropical_o'][df2['iso3num_o'] == 688] = terrain_df.iloc[241,13]
    df2['tropical_d'][df2['iso3num_d'] == 688] = terrain_df.iloc[241,13]
    df2['island_o'][df2['iso3num_o'] == 688] = terrain_df.iloc[241,51]
    df2['island_d'][df2['iso3num_d'] == 688] = terrain_df.iloc[241,51]
    df2['landlocked_o'][df2['iso3num_o'] == 688] = terrain_df.iloc[241,52]
    df2['landlocked_d'][df2['iso3num_d'] == 688] = terrain_df.iloc[241,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 728] = terrain_df.iloc[242,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 728] = terrain_df.iloc[242,4]
    df2['desert_o'][df2['iso3num_o'] == 728] = terrain_df.iloc[242,12]
    df2['desert_d'][df2['iso3num_d'] == 728] = terrain_df.iloc[242,12]
    df2['tropical_o'][df2['iso3num_o'] == 728] = terrain_df.iloc[242,13]
    df2['tropical_d'][df2['iso3num_d'] == 728] = terrain_df.iloc[242,13]
    df2['island_o'][df2['iso3num_o'] == 728] = terrain_df.iloc[242,51]
    df2['island_d'][df2['iso3num_d'] == 728] = terrain_df.iloc[242,51]
    df2['landlocked_o'][df2['iso3num_o'] == 728] = terrain_df.iloc[242,52]
    df2['landlocked_d'][df2['iso3num_d'] == 728] = terrain_df.iloc[242,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 810] = terrain_df.iloc[243,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 810] = terrain_df.iloc[243,4]
    df2['desert_o'][df2['iso3num_o'] == 810] = terrain_df.iloc[243,12]
    df2['desert_d'][df2['iso3num_d'] == 810] = terrain_df.iloc[243,12]
    df2['tropical_o'][df2['iso3num_o'] == 810] = terrain_df.iloc[243,13]
    df2['tropical_d'][df2['iso3num_d'] == 810] = terrain_df.iloc[243,13]
    df2['island_o'][df2['iso3num_o'] == 810] = terrain_df.iloc[243,51]
    df2['island_d'][df2['iso3num_d'] == 810] = terrain_df.iloc[243,51]
    df2['landlocked_o'][df2['iso3num_o'] == 810] = terrain_df.iloc[243,52]
    df2['landlocked_d'][df2['iso3num_d'] == 810] = terrain_df.iloc[243,52]
    df2['rugged_popw_o'][df2['iso3num_o'] == 890] = terrain_df.iloc[244,4]
    df2['rugged_popw_d'][df2['iso3num_d'] == 890] = terrain_df.iloc[244,4]
    df2['desert_o'][df2['iso3num_o'] == 890] = terrain_df.iloc[244,12]
    df2['desert_d'][df2['iso3num_d'] == 890] = terrain_df.iloc[244,12]
    df2['tropical_o'][df2['iso3num_o'] == 890] = terrain_df.iloc[244,13]
    df2['tropical_d'][df2['iso3num_d'] == 890] = terrain_df.iloc[244,13]
    df2['island_o'][df2['iso3num_o'] == 890] = terrain_df.iloc[244,51]
    df2['island_d'][df2['iso3num_d'] == 890] = terrain_df.iloc[244,51]
    df2['landlocked_o'][df2['iso3num_o'] == 890] = terrain_df.iloc[244,52]
    df2['landlocked_d'][df2['iso3num_d'] == 890] = terrain_df.iloc[244,52]    

    df2 = df2.loc[df2['geoscheme_o'].notna()]
    df2 = df2.loc[df2['geoscheme_d'].notna()]
    print(i)
    
    df2.to_csv (r'Gravity-%s.csv' % i, index = False, header=True)
    i+=1 

1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
